# 最適化ケーススタディ

> 実践的なケーススタディによって、自分で最適化モデリングをする力をつける。以下のケースは、特定の実問題に基づくものではない。

[![](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/scmopt/moai-manual/blob/main/optseq-trial.ipynb)


## ローカルで amplpyを動かす方法

AMPLをインストールしてampl.exeがある場所をEnvironmentで指定する。


In [1]:
#| hide
from amplpy import AMPL, Environment, tools
ampl = AMPL(Environment("/Users/mikiokubo/Documents/ampl/"))

## Google Colab.で使う場合

ampl_notebook関数で、使うソルバー(modules)とlicense_uuidを設定する（空白でも動く）。

In [1]:
!pip install amplpy
from amplpy import ampl_notebook
ampl = ampl_notebook(
    modules=["highs","gurobi","cbc","scip","coin"]
            #coin includes ipopt, couenne, bonmin
    license_uuid=None)

## 1. ナースプラクティショナーの人員配置

[original case (English)](https://neos-guide.org/case-studies/sc/la/nurse-practitioner-staffing/)

**問題の背景**

最近、ウィスコンシン大学（UW）ヘルスは、患者の病院再入院を減らすための取り組みを進めています。広範なデータ分析の結果、高齢者層は再入院の可能性が高いと結論付けられました。データによると、高齢患者の再入院の大部分は、病院を退院してから最初の1週間以内に発生しています。この層の再入院を減らす試みとして、UWヘルスは、専門看護施設（skilled nursing facilities）の患者を訪問し、必要に応じて適切なフォローアップケアを提供するためにナースプラクティショナー（NP）を雇用するという独自のプログラムに投資しました。

専門看護施設に退院したすべての患者を訪問することが極めて重要であるため、UWヘルスは最適な人数のナースプラクティショナーを確実に雇用する必要があります。この移行期ケアプログラムの下では、デーン郡地域にナースプラクティショナーが配置される9つの施設があり、合計で815床あります。各施設は民間によって運営されており、それぞれ固有のベッド数を持っています。下の図は施設の位置を示しています。各施設間の車での移動距離は表に記載されています。

---

| 場所 (距離 マイル) | Capital Lakes | Karmenta | Oakwood West | Oakwood East | Sunny Hill | Attic Angels | St. Mary’s | Oak Park | Belmont |
|--------------------|---------------|----------|--------------|--------------|------------|--------------|------------|----------|---------|
| Capitol Lakes      | 0             | 5.2      | 6.4          | 8.8          | 5.6        | 10.9         | 8.9        | 7        | 5       |
| Karmenta           | 5.2           | 0        | 15.8         | 5.5          | 11.9       | 17.2         | 15.2       | 2.6      | 0.5     |
| Oakwood West       | 6.4           | 15.8     | 0            | 20.9         | 3.4        | 3.4          | 5.4        | 15.5     | 14.6    |
| Oakwood East       | 8.8           | 5.5      | 20.9         | 0            | 17.4       | 22.7         | 20.7       | 7        | 5.5     |
| Sunny Hill         | 5.6           | 11.9     | 3.4          | 17.4         | 0          | 6            | 3.9        | 13       | 12.1    |
| Attic Angels       | 10.9          | 17.2     | 3.4          | 22.7         | 6          | 0            | 6.3        | 17.9     | 17      |
| St. Mary’s         | 8.9           | 15.2     | 5.4          | 20.7         | 3.9        | 6.3          | 0          | 15.9     | 15      |
| Oak Park           | 7             | 2.6      | 15.5         | 7            | 13         | 17.9         | 15.9       | 0        | 2.9     |
| Belmont            | 5             | 0.5      | 14.6         | 5.5          | 12.1       | 17           | 15         | 2.9      | 0       |

[download csv file](http://github.com/scmopt/scmopt_data/blob/main/case/case1-distance-table.txt?raw=true)


このプログラムでは、ナースプラクティショナーは特定の施設に配置されます。私たちの最適化問題は、配置されたナースプラクティショナーがいる施設から半径3マイル以内に入る患者数を最大化するために必要なナースプラクティショナーの最小数を決定することと定義します。

この問題をモデル化するために、いくつかの仮定を置きます。

1. 各ナースプラクティショナーは同じペースで働き、同じ数の患者を訪問できる。
2. 移動時間ではなく、施設間の距離のみを考慮する。

<!-- **モデル**

以下にモデルの構成要素を説明します。

*   `NP`: ナースプラクティショナーの数を表す。
*   `Z`: ナースプラクティショナーによってカバーされる患者数を表す。
*   各専門看護施設には整数のIDが割り当てられる: `SNF=1, 2, ..., 9`
*   バイナリ変数 `X(i)`: 施設 `i` にナースプラクティショナーが配置されている場合は `X(i)=1`、そうでなければ `X(i)=0`。
*   `distance(i, j)`: 施設 `i` と施設 `j` の間の車での移動距離を表す。
*   `cover(i, j)=1`: `distance(i, j) < 3` の場合。すなわち、施設 `j` が、その施設から3マイル以内にいるナースプラクティショナーによってカバーされていることを示す。
*   `patients(i)`: 施設 `i` のベッド数を表す。

与えられたナースプラクティショナーの数に対して、問題の目的は `Z` を最大化することです。`Z` は `Z := ∑𝑖,𝑗 𝑝𝑎𝑡𝑖𝑒𝑛𝑡𝑠(𝑗) * 𝑥(𝑖)` と定義されます。
モデルは以下の制約に従います。(1) `∑𝑖 𝑥(𝑖) = NP`（配置されるNPの総数は指定された `NP` と等しい）、(2) 各ナースプラクティショナーは一度に一つの施設にのみ配置される。 -->


## 2. シフト最適化

[original case(English)](https://ampl.com/mo-book/notebooks/03/shift-scheduling.html)

大学のキャンパスに新しい食品店がオープンしました。この店は年中無休、24時間営業となります。
毎日、3つの8時間シフトがあります。
朝番は6:00から14:00、夕番は14:00から22:00、夜勤は22:00から翌日の6:00までです。
夜間（夜勤）は従業員1名のみですが、日中（朝番と夕番）は2名です。ただし、日曜日は各シフトとも1名のみとなります。
各従業員の週労働時間は最大40時間を超えず、シフト間には12時間の休息を取らなければなりません。
週休日に関しては、ある日曜日に休む従業員は、その週の土曜日も同様に休むことが望ましいです。
原則として、利用可能な従業員は10名いますが、これは明らかに過剰人員です。
必要とされる従業員が少なければ少ないほど、他の店舗のためのリソースが多くなります。
１週間の最適なシフトを作成してください。



In [21]:
#| hide
%%ampl_eval
reset;
model;
# ======== Sets ========
set EMPLOYEES = 1..10; # 利用可能な従業員の集合 (最大10人)
set DAYS = 1..7;       # 曜日の集合 (例: 1=月曜, ..., 7=日曜)
set SHIFTS = {"Morning", "Evening", "Night", "OFF"}; # シフトの種類 (OFFは休日)

set WORKING_SHIFTS = SHIFTS diff {"OFF"}; # 勤務シフトの集合

# ======== Parameters ========
param ShiftHours{SHIFTS} default 8; # 各シフトの労働時間 (OFFは0時間にすべきだが、今回は使わない)
param MaxWeeklyHours = 40;          # 週の最大労働時間
param MinRestHours = 12;            # シフト間の最低休息時間

param SUNDAY integer default 7;     # 日曜日を表す数値
param SATURDAY integer default 6;   # 土曜日を表す数値

# 日ごとのシフトに必要な従業員数
param RequiredStaff{DAYS, WORKING_SHIFTS};

# 次の日を計算するためのパラメータ (ラップアラウンド対応)
param NextDay{d in DAYS} = (d mod card(DAYS)) + 1;

# ======== Variables ========
# Assign[e, d, s] = 1 iff employee e works shift s on day d
var Assign{EMPLOYEES, DAYS, SHIFTS} binary;

# IsUsed[e] = 1 iff employee e is used in the schedule
var IsUsed{EMPLOYEES} binary;

var SoftViolation{EMPLOYEES} binary;

# ======== Objective Function ========
# 目的：スケジュールに必要な従業員数を最小化する
minimize TotalEmployeesUsed:
  sum {e in EMPLOYEES} (IsUsed[e] +  0.01 * SoftViolation[e]);

# ======== Constraints ========

# --- 1. Staffing Coverage Constraint ---
# 各日の各勤務シフトで必要な人数を確保する
subject to StaffingRequirement {d in DAYS, s in WORKING_SHIFTS}:
  sum {e in EMPLOYEES} Assign[e, d, s] = RequiredStaff[d, s];

# --- 2. One Assignment Per Day Constraint ---
# 各従業員は、各日に必ず1つの状態（勤務または休日）を持つ
subject to OneAssignmentPerDay {e in EMPLOYEES, d in DAYS}:
  sum {s in SHIFTS} Assign[e, d, s] = 1;

# --- 3. Maximum Weekly Hours Constraint ---
# 各従業員の週労働時間はMaxWeeklyHoursを超えない (全シフト8時間なので、最大5勤務)
subject to MaxHoursPerWeek {e in EMPLOYEES}:
  sum {d in DAYS, s in WORKING_SHIFTS} Assign[e, d, s] * ShiftHours[s] <= MaxWeeklyHours;
  # Alternatively, since all shifts are 8 hours:
  # sum {d in DAYS, s in WORKING_SHIFTS} Assign[e, d, s] <= MaxWeeklyHours / 8; # (<= 5)

# --- 4. Minimum Rest Between Shifts Constraint (12 hours) ---
# 特定のシフトの組み合わせを禁止する
# 終了時刻と次の開始時刻の間が12時間未満になる組み合わせ:
# - Evening (ends 22:00) -> Morning (starts 6:00 next day) : 8 hours rest -> FORBIDDEN
# - Night (ends 6:00) -> Evening (starts 14:00 next day) : 8 hours rest -> FORBIDDEN
# - Morning (ends 14:00) -> Night (starts 22:00 same day) : 8 hours rest -> IMPOSSIBLE due to OneAssignmentPerDay
# - Night (ends 6:00) -> Morning (starts 6:00 next day) : 24 hours rest -> OK
# - Other combinations have >= 16 hours rest -> OK

subject to MinRest_Eve_Morn {e in EMPLOYEES, d in DAYS}:
  Assign[e, d, "Evening"] + Assign[e, NextDay[d], "Morning"] <= 1;

subject to MinRest_Ngt_Eve {e in EMPLOYEES, d in DAYS}:
  Assign[e, d, "Night"] + Assign[e, NextDay[d], "Evening"] <= 1;

# --- 5. Weekend Rest Preference (soft constraint) ---
# 日曜日に休む従業員は、土曜日も（できれば）休まなければならない
subject to WeekendRest {e in EMPLOYEES}:
  Assign[e, SATURDAY, "OFF"] + SoftViolation[e] >= Assign[e, SUNDAY, "OFF"];
  # If Assign[e, SUNDAY, "OFF"] is 1, then Assign[e, SATURDAY, "OFF"] must also be 1.

# --- 6. Link IsUsed Variable ---
# 従業員が少なくとも1つの勤務シフトを担当する場合、IsUsed変数を1にする
# (Mは大きな数だが、ここでは単純に Assign <= IsUsed で十分)
subject to LinkEmployeeUsage {e in EMPLOYEES, d in DAYS, s in WORKING_SHIFTS}:
  IsUsed[e] >= Assign[e, d, s];

# --- Optional: Ensure IsUsed is 0 if no shifts assigned ---
# If sum of Assign is 0, IsUsed must be 0. This is usually implicitly handled
# by the minimization objective, but can be made explicit if needed.
# subject to LinkEmployeeUsageZero {e in EMPLOYEES}:
#   IsUsed[e] * card(DAYS) * card(WORKING_SHIFTS) >= sum {d in DAYS, s in WORKING_SHIFTS} Assign[e, d, s];

data;
# データファイル for store_schedule.mod

# 曜日とシフトごとの必要人数データ
param RequiredStaff : "Morning" "Evening" "Night" :=
  1  2 2 1  # Monday
  2  2 2 1  # Tuesday
  3  2 2 1  # Wednesday
  4  2 2 1  # Thursday
  5  2 2 1  # Friday
  6  2 2 1  # Saturday (Day 6)
  7  1 1 1  # Sunday   (Day 7)
;

# Optional: Specify shift hours if they differ or if OFF needs 0
# param ShiftHours :=
#  "Morning" 8
#  "Evening" 8
#  "Night"   8
#  "OFF"     0
# ;

# Specify Sunday and Saturday numbers if different from default
param SUNDAY := 7;
param SATURDAY := 6;

option solver highs;
solve;

display TotalEmployeesUsed;

# スケジュールを表示 (見やすいように整形)
printf "\n=== Schedule ===\n";
printf "%-10s", "Employee";
for {d in DAYS} {
  printf " %3s", d; # Display day number header
}
printf "\n";
printf "%-10s", "----------";
for {d in DAYS} {
  printf " ---";
}
printf "\n";

for {e in EMPLOYEES: IsUsed[e] > 0.5} { # Only display used employees
  printf "%-10s", e;
  for {d in DAYS} {
    for {s in SHIFTS} {
      if Assign[e, d, s] > 0.5 then {
         # Display first letter of shift or OFF
         if s == "Morning" then {printf " %3s", "M"}
         if s == "Evening" then {printf " %3s", "E"}
         if s == "Night" then {printf " %3s", "N"}
         if s == "OFF" then {printf " %3s", "OFF"};
      }
    }
  }
  printf "\n";
}

# 勤務時間や休日数の確認 (オプション)
display {e in EMPLOYEES: IsUsed[e] > 0.5} (sum {d in DAYS, s in WORKING_SHIFTS} Assign[e,d,s] * ShiftHours[s]);
display {e in EMPLOYEES: IsUsed[e] > 0.5} (sum {d in DAYS} Assign[e,d,"OFF"]);

HiGHS 1.10.0: optimal solution; objective 7.02
15008 simplex iterations
29 branching nodes
TotalEmployeesUsed = 7.02


=== Schedule ===
Employee     1   2   3   4   5   6   7
---------- --- --- --- --- --- --- ---
1            E   E   N   M   M OFF OFF
2          OFF   N OFF   E   E   N   N
4          OFF   M   E OFF OFF   M   M
5            E   E OFF OFF   N   M   E
7            M   M   M   M OFF OFF OFF
8            M OFF   M   N   M   E OFF
10           N OFF   E   E   E   E OFF
sum{d in DAYS, s in WORKING_SHIFTS} Assign[e,d,s]*ShiftHours[s] [*] :=
 1  40
 2  40
 4  32
 5  40
 7  32
 8  40
10  40
;

sum{d in DAYS} Assign[e,d,'OFF'] [*] :=
 1  2
 2  2
 4  3
 5  2
 7  3
 8  2
10  2
;



## 3. 中規模食品卸売業者の店舗配送ルート最適化


**1. 背景と課題**

「フレッシュデリバリー株式会社」は、地域のレストランや小規模小売店へ生鮮食品や加工品を卸している中規模の卸売業者です。自社の物流センター（デポ）から、保有する複数の保冷トラック（能力が異なる）を使って毎日配送を行っています。

**現状の課題:**

*   **配送コストの増大:** 燃料費の高騰に加え、ベテラン担当者の経験に頼った非効率なルート設定により、人件費（残業代含む）や車両維持費がかさんでいる。
*   **時間指定の厳守:** 顧客（店舗）からはランチやディナーの仕込みに間に合わせるため、厳しい納品時間帯（タイムウィンドウ）が指定されていることが多いが、遅延が発生することもある。
*   **車両積載率のばらつき:** 荷量の少ないルートに大型トラックを割り当ててしまうなど、車両の能力を有効活用できていない場合がある。

これらの課題を解決するため、数理最適化を用いて、**総配送コスト（車両固定費＋変動費）を最小化**し、かつ**全ての顧客の時間指定を守る**配送計画を作成することを目指します。

**2. データ**

*   **物流センター（デポ）:**
    *   所在地: (座標 0, 0)
    *   稼働開始可能時刻: 8:00

*   **配送先顧客（店舗）:**

    | 顧客ID | 所在地 (X, Y) | 需要量 (ケース) | 納品時間窓      | 荷降ろし時間 (分) |
    | :----- | :------------ | :-------------- | :-------------- | :---------------- |
    | A      | (15, 25)      | 30              | 9:00 - 10:00    | 15                |
    | B      | (30, 10)      | 45              | 9:30 - 11:00    | 20                |
    | C      | (-10, 20)     | 25              | 10:00 - 12:00   | 15                |
    | D      | (5, -15)      | 50              | 9:00 - 11:30    | 25                |
    | E      | (-20, -10)    | 35              | 10:30 - 12:30   | 15                |
    | F      | (20, -5)      | 40              | 11:00 - 13:00   | 20                |

*   **配送トラック:**

    | トラックID | 積載容量 (ケース) | 固定費 (円/日) | 変動費 (円/km) | 平均速度 (km/h) |
    | :--------- | :---------------- | :------------- | :------------- | :-------------- |
    | V1         | 100               | 6000           | 120            | 40              |
    | V2         | 80                | 5000           | 110            | 45              |
    | V3         | 80                | 5000           | 110            | 45              |

*   **地点間距離 (km) と移動時間 (分):**
 
    **距離行列 (km)**

    
    |      | デポ | A    | B    | C    | D    | E    | F    |
    | :--- | :--- | :--- | :--- | :--- | :--- | :--- | :--- |
    | デポ | 0    | 32   | 35   | 25   | 18   | 25   | 23   |
    | A    | 32   | 0    | 30   | 28   | 45   | 55   | 40   |
    | B    | 35   | 30   | 0    | 48   | 30   | 58   | 15   |
    | C    | 25   | 28   | 48   | 0    | 35   | 20   | 45   |
    | D    | 18   | 45   | 30   | 35   | 0    | 30   | 15   |
    | E    | 25   | 55   | 58   | 20   | 30   | 0    | 40   |
    | F    | 23   | 40   | 15   | 45   | 15   | 40   | 0    |

    **移動時間行列 (分) - トラックの平均時速 (40km/h) をもとに計算 **


**3. 最適化モデルの目標と制約**

*   **目的関数:**
    *   最小化: Σ (使用するトラックの固定費) + Σ (各トラックの総走行距離 × 変動費)

*   **主な制約条件:**
    1.  **車両割り当て:** 各顧客は、いずれか1台のトラックによって、ちょうど1回訪問される。
    2.  **デポ発着:** 使用される各トラックは、デポを出発し、全ての担当顧客を訪問後、デポに戻る。
    3.  **積載容量:** 各トラックに積み込まれる荷物の総量は、そのトラックの積載容量を超えない。
    4.  **時間窓:** 各顧客へのトラック到着時刻は、指定された納品時間窓内であること。
        *   `到着時刻 >= 時間窓開始時刻`
        *   `到着時刻 <= 時間窓終了時刻`
        *   早く着きすぎた場合は、時間窓開始時刻まで待機する。
    5.  **時間連続性:** ある地点への到着時刻、荷降ろし時間、次の地点への移動時間を考慮し、次の地点への到着時刻を計算する。
        *   `次の地点への到着時刻 = max(現地点到着時刻, 時間窓開始時刻) + 荷降ろし時間 + 移動時間`
    6.  **トラック稼働時間:** (オプション) ドライバーの最大労働時間などを考慮することも可能。

**4. 期待されるアウトプット (最適化ソルバーによる結果)**

*   **使用トラック:** 最適なコストを実現するために使用するトラック (V1, V2, V3 のうちどれか、または全部)。
*   **各トラックの担当顧客と訪問順序 (ルート):**
    *   例:
        *   トラックV1: デポ → C → A → デポ
        *   トラックV2: デポ → D → F → B → デポ
        *   トラックV3: デポ → E → デポ
*   **各地点への到着・出発予定時刻:**
    *   例: トラックV1 - C店到着 10:15, C店出発 10:30, A店到着 11:12 (時間窓違反!), ... のような詳細スケジュール。 
*   **総配送コスト:** 最小化された総コスト (円)。
*   **各トラックの走行距離と積載率:**

**5. ケーススタディの意義と実務への応用**

このケーススタディは、複数の制約（時間、容量、コスト）が絡み合う複雑な配送計画問題を、数理最適化によってどのように解決できるかを示しています。

*   **コスト削減:** 燃料費、人件費、車両固定費を最適化し、具体的な削減額を提示できる。
*   **サービス品質向上:** 時間指定遵守により、顧客満足度を高める。
*   **業務効率化:** 配車計画作成にかかる時間を大幅に短縮し、属人化を解消できる。
*   **資源の有効活用:** トラックの積載率を向上させ、遊休車両を減らす。
*   **意思決定支援:** 新規顧客獲得時や車両増減時の影響をシミュレーションし、戦略的な意思決定を支援する。



#| hide

これは、**タイムウィンドウ付き異種車両配送計画問題 (Heterogeneous Fleet Vehicle Routing Problem with Time Windows - HFVRPTW)** と呼ばれる典型的な数理最適化問題です。

### 問題の定義と定式化

この問題を数理モデルとして定式化するために、以下の集合、パラメータ、変数を定義します。

**1. 集合 (Sets)**

* $V$: トラックの集合 (e.g., V1, V2, V3)
* $C$: 顧客（店舗）の集合 (e.g., A, B, C, D, E, F)
* $N$: デポを含む全地点の集合 ($N = C \cup \{\text{Depot}\}$)

**2. パラメータ (Parameters)**

* $Q_k$: トラック $k \in V$ の積載容量
* $F_k$: トラック $k \in V$ の固定費
* $C_k^V$: トラック $k \in V$ の変動費 (距離あたり)
* $d_i$: 顧客 $i \in C$ の需要量
* $e_i$: 顧客 $i \in C$ の納品時間窓の開始時刻
* $l_i$: 顧客 $i \in C$ の納品時間窓の終了時刻
* $s_i$: 顧客 $i \in C$ での荷降ろし時間
* $D_{ij}$: 地点 $i \in N$ から地点 $j \in N$ への距離
* $T_{ijk}$: トラック $k \in V$ で地点 $i \in N$ から地点 $j \in N$ へ移動する時間
* $T_{start}$: デポの稼働開始時刻

**3. 決定変数 (Decision Variables)**

* $x_{ijk}$: トラック $k \in V$ が地点 $i \in N$ から地点 $j \in N$ へ直接移動する場合に $1$、さもなければ $0$ となるバイナリ変数。
* $y_k$: トラック $k \in V$ を使用する場合に $1$、さもなければ $0$ となるバイナリ変数。
* $t_{ik}$: トラック $k \in V$ が地点 $i \in N$ に到着する時刻を表す連続変数。

**4. 定式化**

**目的関数**
総配送コスト（全使用トラックの固定費と全トラックの総変動費の合計）を最小化します。
$$\text{Minimize} \quad \sum_{k \in V} F_k \cdot y_k + \sum_{i \in N} \sum_{j \in N} \sum_{k \in V} C_k^V \cdot D_{ij} \cdot x_{ijk}$$

**制約条件**

1.  **顧客訪問制約:** 全ての顧客は、いずれかのトラックによって、ちょうど1回訪問されなければなりません。
    $$
    \sum_{i \in N, i \neq j} \sum_{k \in V} x_{ijk} = 1 \quad (\forall j \in C)
    $$
2.  **フロー保存則:** トラックがある顧客を訪問した場合、必ずその顧客から次の地点へ出発しなければなりません。
    $$
    \sum_{i \in N, i \neq h} x_{ihk} = \sum_{j \in N, j \neq h} x_{hjk} \quad (\forall h \in C, \forall k \in V)
    $$
3.  **デポ出発・帰着制約:** 使用されるトラックは、必ずデポから出発し、1つのルートを担当した後、デポに戻ります。
    $$
    \sum_{j \in C} x_{\text{Depot}, j, k} = y_k \quad (\forall k \in V)
    $$   $$
    \sum_{i \in C} x_{i, \text{Depot}, k} = y_k \quad (\forall k \in V)
    $$
4.  **積載容量制約:** 各トラックが配送する荷物の総需要量は、そのトラックの積載容量を超えてはなりません。
    $$
    \sum_{i \in C} d_i \left( \sum_{j \in N, j \neq i} x_{ji k} \right) \le Q_k \cdot y_k \quad (\forall k \in V)
    $$
5.  **時間窓制約:** 各顧客への到着時刻は、指定された時間窓内でなければなりません。
    $$
    e_i \le t_{ik} \le l_i \quad (\text{if customer } i \text{ is visited by truck } k)
    $$
6.  **時間連続性と部分巡回路除去:** ある顧客への到着時刻は、前の顧客でのサービス時間と移動時間を考慮して計算されます。早く到着した場合は待機します。この制約は、同時に非合理な部分巡回路（デポを含まないループ）の発生を防ぎます。
    $$
    t_{jk} \ge (\max(t_{ik}, e_i) + s_i + T_{ijk}) \cdot x_{ijk} \quad (\forall i \in C, \forall j \in N, i \neq j, \forall k \in V)
    $$
    この`max`を含む非線形制約は、線形のBig-M法を用いて以下のように表現されます。$M$は十分に大きな定数です。
    $$
    t_{jk} \ge t_{ik} + s_i + T_{ijk} - M(1-x_{ijk})
    $$   $$
    t_{jk} \ge e_i + s_i + T_{ijk} - M(1-x_{ijk})
    $$


In [12]:
#| hide

%%ampl_eval
# ------------------------------------------------------------------
# モデル定義 (.mod)
# ------------------------------------------------------------------
reset;
model;

# ===== 1. 集合 (Sets) =====
# トラック、顧客、およびデポを含む全地点の集合
set TRUCKS;
set CUSTOMERS;
set LOCATIONS = CUSTOMERS union {'Depot'};


# ===== 2. パラメータ (Parameters) =====
# トラック関連のパラメータ
param capacity {TRUCKS} > 0;
param fixed_cost {TRUCKS} >= 0;
param variable_cost {TRUCKS} >= 0; # 円/km
param speed {TRUCKS} > 0; # km/h

# 顧客関連のパラメータ (デポのデフォルト値は0)
param demand {CUSTOMERS} > 0;
param earliest {LOCATIONS} default 0; # 時間窓開始時刻 (分)
param latest {LOCATIONS} default 9999;  # 時間窓終了時刻 (分)
param service_time {LOCATIONS} default 0; # 荷降ろし時間 (分)

# 地点間と時間に関するパラメータ
param dist {LOCATIONS, LOCATIONS} >= 0, default 0;
param depot_start_time; # デポの稼働開始時刻 (分)
param M; # Big-M法で使用する大きな数

# モデル内で計算されるパラメータ (地点間の移動時間)
param travel_time {i in LOCATIONS, j in LOCATIONS, k in TRUCKS} =
    if i <> j then (dist[i,j] / speed[k]) * 60 else 0;


# ===== 3. 変数 (Variables) =====
# x[i,j,k] = 1: トラックkが地点iからjへ移動する
var x {LOCATIONS, LOCATIONS, TRUCKS} binary;
# y[k] = 1: トラックkを使用する
var y {TRUCKS} binary;
# t[i,k]: トラックkが地点iに到着する時刻
var t {LOCATIONS, TRUCKS} >= 0;


# ===== 4. 目的関数 (Objective Function) =====
# 総配送コスト(固定費 + 変動費)を最小化
minimize Total_Cost:
    sum {k in TRUCKS} fixed_cost[k] * y[k] +
    sum {i in LOCATIONS, j in LOCATIONS, k in TRUCKS} variable_cost[k] * dist[i,j] * x[i,j,k];


# ===== 5. 制約 (Constraints) =====
# 自己ループの禁止
subject to No_Self_Loop {i in LOCATIONS, k in TRUCKS}:
    x[i,i,k] = 0;

# 各顧客はちょうど1回訪問される
subject to Assign_Customer {c in CUSTOMERS}:
    sum {i in LOCATIONS, k in TRUCKS: i <> c} x[i,c,k] = 1;

# トラックのフロー保存則 (入るアークと出るアークは同数)
subject to Flow_Conservation {h in CUSTOMERS, k in TRUCKS}:
    sum {i in LOCATIONS: i <> h} x[i,h,k] = sum {j in LOCATIONS: j <> h} x[h,j,k];

# 使用されるトラックはデポから出発する
subject to Depot_Out {k in TRUCKS}:
    sum {j in CUSTOMERS} x['Depot',j,k] = y[k];

# 使用されるトラックはデポへ帰着する
subject to Depot_In {k in TRUCKS}:
    sum {i in CUSTOMERS} x[i,'Depot',k] = y[k];

# トラックの積載容量制約
subject to Capacity {k in TRUCKS}:
    sum {c in CUSTOMERS} demand[c] * (sum {i in LOCATIONS: i <> c} x[i,c,k]) <= capacity[k];

# デポの出発時刻をセット
subject to Set_Depot_Time {k in TRUCKS}:
    t['Depot', k] = depot_start_time;

# 時間連続性制約 (部分巡回路除去を含む)
# max(到着時刻, 顧客の時間窓開始時刻) + サービス時間 + 移動時間 を線形化
subject to Time_Continuity_1 {i in CUSTOMERS, j in LOCATIONS, k in TRUCKS: i <> j}:
    t[j,k] >= (t[i,k] + service_time[i] + travel_time[i,j,k]) - M * (1 - x[i,j,k]);

subject to Time_Continuity_2 {i in CUSTOMERS, j in LOCATIONS, k in TRUCKS: i <> j}:
    t[j,k] >= (earliest[i] + service_time[i] + travel_time[i,j,k]) - M * (1 - x[i,j,k]);

# デポからの出発時刻の制約
subject to Time_From_Depot {j in CUSTOMERS, k in TRUCKS}:
    t[j,k] >= (depot_start_time + travel_time['Depot',j,k]) - M * (1 - x['Depot',j,k]);

# 顧客の時間窓制約
subject to Time_Window_Lower {c in CUSTOMERS, k in TRUCKS}:
    t[c,k] >= earliest[c] * (sum {i in LOCATIONS: i <> c} x[i,c,k]);

subject to Time_Window_Upper {c in CUSTOMERS, k in TRUCKS}:
    t[c,k] <= latest[c] + M * (1 - (sum {i in LOCATIONS: i <> c} x[i,c,k]));

# ------------------------------------------------------------------
# データ定義 (.dat)
# ------------------------------------------------------------------
data;

# 集合データ
set TRUCKS := V1 V2 V3;
set CUSTOMERS := A B C D E F;

# パラメータデータ
param depot_start_time := 480; # 8:00
param M := 99999;

# トラックの仕様
param capacity :=
    V1 100
    V2 80
    V3 80;

param fixed_cost :=
    V1 6000
    V2 5000
    V3 5000;

param variable_cost :=
    V1 120
    V2 110
    V3 110;

param speed :=
    V1 40
    V2 45
    V3 45;

# 顧客の仕様
param demand :=
    A 30
    B 45
    C 25
    D 50
    E 35
    F 40;

param earliest :=
    A  540 # 09:00
    B  570 # 09:30
    C  600 # 10:00
    D  540 # 09:00
    E  630 # 10:30
    F  660; # 11:00

param latest :=
    A  600 # 10:00
    B  660 # 11:00
    C  720 # 12:00
    D  690 # 11:30
    E  750 # 12:30
    F  780; # 13:00

param service_time :=
    A  15
    B  20
    C  15
    D  25
    E  15
    F  20;

# 距離行列データ (km)
param dist :=
    Depot A 32   Depot B 35   Depot C 25   Depot D 18   Depot E 25   Depot F 23
    A Depot 32   A B 30       A C 28       A D 45       A E 55       A F 40
    B Depot 35   B A 30       B C 48       B D 30       B E 58       B F 15
    C Depot 25   C A 28       C B 48       C D 35       C E 20       C F 45
    D Depot 18   D A 45       D B 30       D C 35       D E 30       D F 15
    E Depot 25   E A 55       E B 58       E C 20       E D 30       E F 40
    F Depot 23   F A 40       F B 15       F C 45       F D 15       F E 40;

# ------------------------------------------------------------------
# 実行コマンド (.run)
# ------------------------------------------------------------------
option solver highs;
solve;

# 結果の表示 (ルート、到着時刻、使用トラック)
display x, t, y;

HiGHS 1.10.0: infeasible problem
0 simplex iterations
0 branching nodes
x [*,*,V1]
:       A      B      C      D    Depot      E      F    :=
A       0   0         0   0          0    0         0
B       0   0         0   0          0    0.54717   0
C       0   0         0   0          0    0         0
D       0   0.54717   0   0          0    0         0
Depot   0   0         0   0          0    0         0
E       0   0         0   0.54717    0    0         0
F       0   0         0   0          0    0         0

 [*,*,V2]
:          A          B          C          D    Depot      E          F     :=
A       0          0          0.179245   0.45283    0    0          0
B       0          0          0          0          0    0.179245   0
C       0          0.179245   0          0          0    0          0
D       0.45283    0          0          0          0    0          0
Depot   0          0          0          0          0    0          0
E       0.179245   0          0       

## 4. 部品メーカーにおける月次生産計画最適化

**1. 背景と課題**

「テクノパーツ工業」は、自動車産業向けに2種類の精密部品（部品X、部品Y）を製造・販売している中小企業です。部品Xと部品Yは、それぞれ異なる原材料を使用し、工場内の2つの主要な工程（切断工程、組立工程）を経て完成します。

**現状の課題:**

*   **利益最大化の難しさ:** 部品XとYでは利益率が異なり、また各工程での加工時間も異なります。どの製品をどれだけ生産すれば、限られた資源（機械稼働時間、原材料在庫）の中で月間の総利益を最大化できるか、経験と勘に頼っており最適化されていない。
*   **ボトルネック工程の存在:** 特定の工程（特に組立工程の機械）の稼働率が高く、生産全体の制約となっている可能性があるが、定量的に把握できていない。
*   **原材料の過不足:** 需要予測に基づいて原材料を発注しているが、生産計画の変動により、月末に特定の原材料が不足したり、逆に過剰在庫になったりすることがある。

これらの課題に対し、数理最適化を用いて、**月間総利益を最大化**する最適な部品Xと部品Yの生産量を決定することを目指します。

**2. データ (月間)**

*   **製品情報:**

    | 製品名 | 販売単価 (円/個) | 月間最大需要 (個) |
    | :----- | :--------------- | :---------------- |
    | 部品X  | 5,000            | 800               |
    | 部品Y  | 6,500            | 600               |

*   **原材料情報:**

    | 原材料名 | 単価 (円/kg) | 月間利用可能量 (kg) | 部品X必要量 (kg/個) | 部品Y必要量 (kg/個) |
    | :------- | :----------- | :------------------ | :------------------ | :------------------ |
    | 材料P    | 300          | 3,000               | 2.0                 | 1.0                 |
    | 材料Q    | 500          | 4,000               | 1.5                 | 3.0                 |

*   **工程・機械情報:**

    | 工程名 | 使用機械 | 利用可能時間 (時間/月) | 部品X処理時間 (時間/個) | 部品Y処理時間 (時間/個) | 加工費 (円/時間) |
    | :----- | :------- | :--------------------- | :---------------------- | :---------------------- | :--------------- |
    | 切断   | 機械A    | 400                    | 0.15 (9分)              | 0.20 (12分)             | 2,000            |
    | 組立   | 機械B    | 350                    | 0.25 (15分)             | 0.30 (18分)             | 3,000            |

*   **その他:**
    *   生産量に関わらず発生する固定費は考慮しない（利益最大化の判断には影響しないため）。

<!-- **3. 最適化モデルの目標と制約**

*   **決定変数:**
    *   `x`: 部品Xの月間生産量 (個)
    *   `y`: 部品Yの月間生産量 (個)

*   **目的関数 (総利益の最大化):**
    *   最大化: (部品Xの売上 + 部品Yの売上) - (総原材料費) - (総加工費)
    *   売上 = `5000 * x + 6500 * y`
    *   原材料費 = `(2.0 * x + 1.0 * y) * 300 + (1.5 * x + 3.0 * y) * 500`
    *   加工費 = `(0.15 * x + 0.20 * y) * 2000 + (0.25 * x + 0.30 * y) * 3000`
    *   整理すると:
        `Maximize: (5000x + 6500y) - ( (600x + 300y) + (750x + 1500y) ) - ( (300x + 400y) + (750x + 900y) )`
        `Maximize: (5000x + 6500y) - (1350x + 1800y) - (1050x + 1300y)`
        `Maximize: 2600x + 3400y`

*   **制約条件:**
    1.  **最大需要制約:**
        *   `x <= 800`
        *   `y <= 600`
    2.  **原材料制約:**
        *   材料P: `2.0 * x + 1.0 * y <= 3000`
        *   材料Q: `1.5 * x + 3.0 * y <= 4000`
    3.  **機械稼働時間制約:**
        *   機械A (切断): `0.15 * x + 0.20 * y <= 400`
        *   機械B (組立): `0.25 * x + 0.30 * y <= 350`
    4.  **非負制約:**
        *   `x >= 0`
        *   `y >= 0`

**4. 期待されるアウトプット (最適化ソルバーによる結果)**

最適化ソルバー（線形計画法ソルバー）を用いてこの問題を解くと、以下の情報が得られます。

*   **最適生産量:**
    *   部品X (`x`): 例えば 560 個
    *   部品Y (`y`): 例えば 520 個
    *(上記は仮の最適解です。実際にソルバーで解く必要があります)*
*   **最大総利益:**
    *   `2600 * 560 + 3400 * 520 = 1,456,000 + 1,768,000 = 3,224,000` 円 (月間)
*   **資源利用状況:**
    *   材料P使用量: `2.0 * 560 + 1.0 * 520 = 1120 + 520 = 1640 kg` (利用可能量 3000 kg) -> 余剰あり
    *   材料Q使用量: `1.5 * 560 + 3.0 * 520 = 840 + 1560 = 2400 kg` (利用可能量 4000 kg) -> 余剰あり
    *   機械A稼働時間: `0.15 * 560 + 0.20 * 520 = 84 + 104 = 188 時間` (利用可能時間 400 時間) -> 余剰あり
    *   機械B稼働時間: `0.25 * 560 + 0.30 * 520 = 140 + 156 = 296 時間` (利用可能時間 350 時間) -> 余剰あり
    *(この仮の解では、どの資源も上限に達していません。実際の最適解では、いずれかの資源（多くの場合、ボトルネック工程の機械時間や利益率の高い製品の需要上限）が制約上限に達することが一般的です)* -->

**3. ケーススタディの意義と実務への応用**

このケーススタディは、数理最適化が工場運営において以下の価値を提供することを示しています。

*   **収益性の向上:** 利益貢献度と資源消費を考慮した最適な製品ミックスを決定し、工場全体の収益性を最大化する。
*   **資源の有効活用:** ボトルネックとなっている資源（機械、原材料など）を特定し、その利用率を最大化する計画を立てる。また、余剰がある資源も可視化される。
*   **データに基づいた意思決定:** 経験や勘ではなく、定量的なデータ分析に基づいて生産計画を立案できる。これにより、計画の客観性と再現性が向上する。
*   **感度分析による柔軟な対応:** 原材料価格の変動、機械の故障、需要の変化などが起きた場合に、最適化モデルを再実行することで、変化に対応した新しい最適計画を迅速に作成できる（What-if分析）。



## 5. 部品メーカーにおける3ヶ月生産・在庫最適化計画

**1. 背景と課題の進化**

「テクノパーツ工業」では、単月での生産計画最適化により一定の成果を上げましたが、新たな課題が浮上しました。

*   **需要の季節変動:** 部品Xは春先に、部品Yは初夏に需要が高まる傾向があり、単月計画では需要ピーク時に生産能力が追いつかず機会損失が発生したり、逆に需要閑散期に過剰生産してしまうリスクがあります。
*   **原材料調達の変動:** 主要な原材料Pのサプライヤーが、特定の月（例: 2月）にメンテナンスのため供給量を減らすと通知してきました。
*   **在庫コスト:** 製品在庫や原材料在庫を持つことには、倉庫スペースや管理のためのコストが発生します。

これらの課題に対応するため、今後3ヶ月間を見通した**生産計画と在庫計画を同時に最適化**し、**3ヶ月間の総利益（売上 - 原材料費 - 加工費 - 在庫保管費）を最大化**することを目指します。

**2. データ (3ヶ月計画: 1月、2月、3月)**

*   **製品情報:**

    | 製品名 | 販売単価 (円/個) | 1月需要 (個) | 2月需要 (個) | 3月需要 (個) | 在庫保管費 (円/個/月) |
    | :----- | :--------------- | :----------- | :----------- | :----------- | :-------------------- |
    | 部品X  | 5,000            | 700          | 600          | 900          | 50                    |
    | 部品Y  | 6,500            | 500          | 700          | 650          | 60                    |
    *   *初期在庫 (1月初め):* 部品X: 50個, 部品Y: 30個
    *   *目標期末在庫 (3月末):* 部品X: 80個, 部品Y: 50個 (安全在庫として)

*   **原材料情報:**

    | 原材料名 | 単価 (円/kg) | 1月調達量(kg) | 2月調達量(kg) | 3月調達量(kg) | 在庫保管費 (円/kg/月) | 部品X必要量(kg/個) | 部品Y必要量(kg/個) |
    | :------- | :----------- | :------------ | :------------ | :------------ | :-------------------- | :----------------- | :----------------- |
    | 材料P    | 300          | 3,000         | 2,000         | 3,000         | 10                    | 2.0                | 1.0                |
    | 材料Q    | 500          | 4,000         | 4,000         | 4,000         | 15                    | 1.5                | 3.0                |
    *   *初期在庫 (1月初め):* 材料P: 200kg, 材料Q: 300kg

*   **工程・機械情報 (各月共通):**

    | 工程名 | 使用機械 | 月間利用可能時間(時間) | 部品X処理時間(時間/個) | 部品Y処理時間(時間/個) | 加工費 (円/時間) |
    | :----- | :------- | :--------------------- | :--------------------- | :--------------------- | :--------------- |
    | 切断   | 機械A    | 400                    | 0.15                   | 0.20                   | 2,000            |
    | 組立   | 機械B    | 350                    | 0.25                   | 0.30                   | 3,000            |

<!-- **3. 最適化モデルの目標と制約**

*   **決定変数 (例):**
    *   `Prod[p, t]`: 製品 `p` (X or Y) の期間 `t` (1, 2, 3) における生産量 (個)
    *   `Sell[p, t]`: 製品 `p` の期間 `t` における販売量 (個)
    *   `InvP[p, t]`: 製品 `p` の期間 `t` の期末在庫量 (個)
    *   `InvR[r, t]`: 原材料 `r` (P or Q) の期間 `t` の期末在庫量 (kg)

*   **目的関数 (3ヶ月間の総利益最大化):**
    *   最大化: Σ(t=1 to 3) [ Σ(p) (販売単価[p] * Sell[p, t]) - Σ(r) (原材料単価[r] * 使用量[r, t]) - Σ(m) (加工費[m] * 稼働時間[m, t]) - Σ(p) (製品保管費[p] * InvP[p, t]) - Σ(r) (原材料保管費[r] * InvR[r, t]) ]
        *   *原材料使用量[r, t]* と *機械稼働時間[m, t]* は生産量 `Prod[p, t]` から計算される。

*   **主な制約条件 (各期間 t=1, 2, 3 で成立):**
    1.  **製品在庫バランス:**
        *   `InvP[p, t-1] + Prod[p, t] = Sell[p, t] + InvP[p, t]`
        *   (t=1 の時、`InvP[p, 0]` は初期在庫)
    2.  **販売量上限 (需要制約):**
        *   `Sell[p, t] <= 需要[p, t]`
    3.  **原材料在庫バランス:**
        *   `InvR[r, t-1] + 調達量[r, t] = Σ(p) (部品p必要量[r] * Prod[p, t]) + InvR[r, t]`
        *   (t=1 の時、`InvR[r, 0]` は初期在庫)
    4.  **機械稼働時間制約:**
        *   機械A: `Σ(p) (部品p処理時間[A] * Prod[p, t]) <= 利用可能時間[A]`
        *   機械B: `Σ(p) (部品p処理時間[B] * Prod[p, t]) <= 利用可能時間[B]`
    5.  **目標期末在庫制約 (期間 t=3 のみ):**
        *   `InvP[p, 3] >= 目標期末在庫[p]`
    6.  **非負制約:** 全ての変数は 0 以上。

**4. 期待されるアウトプット (最適化ソルバーによる結果)**

*   **各月の最適生産計画:**
    *   1月: 部品X 生産量 `Prod[X, 1]`, 部品Y 生産量 `Prod[Y, 1]`
    *   2月: 部品X 生産量 `Prod[X, 2]`, 部品Y 生産量 `Prod[Y, 2]`
    *   3月: 部品X 生産量 `Prod[X, 3]`, 部品Y 生産量 `Prod[Y, 3]`
*   **各月の最適販売計画:** (`Sell[p, t]` - 需要を満たす範囲で決定)
*   **各月末の製品・原材料在庫レベル:** (`InvP[p, t]`, `InvR[r, t]`)
*   **各月の機械稼働率:**
*   **3ヶ月間の最大総利益:** (円)
*   **ボトルネックの特定:** どの期間のどの資源（機械時間、原材料、需要）が計画の制約となっているか。

**例えば、以下のような計画が出力される可能性があります:**

*   需要が少ない1月に、翌月以降の需要増に備えて部品Yを多めに生産し、在庫として持つ（先行生産）。
*   原材料Pの供給が減る2月を見越して、1月に材料Pを多く使う部品Xの生産を一部行い、原材料Pの在庫も確保しておく。
*   需要がピークとなる3月には、前期からの在庫を活用しつつ、フル稼働に近い状態で部品Xを中心に生産する。 -->

**3. ケーススタディの意義と実務への応用**

この多期間モデルは、単期間モデルよりも現実に近い状況を扱えます。

*   **需要変動への戦略的対応:** 先行生産や在庫調整により、需要の波に柔軟に対応し、機会損失を最小化する。
*   **サプライチェーンリスクへの備え:** 原材料供給の変動などを事前に計画に織り込むことで、生産停止リスクを低減する。
*   **在庫コストの最適化:** 過剰な在庫や欠品を防ぎ、製品・原材料の両方で最適な在庫レベルを維持する。
*   **キャッシュフローの改善:** 生産、販売、在庫のバランスを取ることで、資金繰りを安定させる計画を立てられる。
*   **中期的な視点での資源配分:** 複数期間にわたる機械稼働や原材料調達を最適化し、工場全体の生産性を高める。

このような中期的な生産・在庫最適化計画は、企業の収益性向上と安定的な操業に不可欠であり、多くの製造業でSCM (サプライチェーンマネジメント) の一環として導入されています。


#| hide

## 複数期間にわたる生産・在庫最適化計画

提示された課題は、複数期間（今回は3ヶ月）にわたる需要やコスト、供給の変動を考慮し、全体の利益を最大化する生産計画および在庫計画を同時に策定する問題です。このような問題は、数理最適化の分野で「多期間計画問題」として知られており、線形計画法（Linear Programming）を用いて最適化することが可能です。

### 1. 問題の定義と定式化

この問題の目的は、3ヶ月間の総利益（総売上 - 総費用）を最大化することです。総費用は、原材料費、加工費、製品在庫保管費、原材料在庫保管費の合計です。これを達成するために、各月にどの製品をどれだけ生産し、どれだけ販売し、どれだけ在庫として保有するかを決定します。

#### 集合とインデックス
* $P$: 製品の集合 ($p \in P$)
* $M$: 原材料の集合 ($m \in M$)
* $C$: 工程（機械）の集合 ($c \in C$)
* $T$: 計画期間（月）の集合 ($t \in T = \{1, 2, 3\}$)
* $T_0$: 初期在庫を含む期間の集合 ($t \in T_0 = \{0, 1, 2, 3\}$)

#### パラメータ
* $SalesPrice_p$: 製品 $p$ の販売単価
* $Demand_{pt}$: 月 $t$ における製品 $p$ の需要量
* $ProdHoldingCost_p$: 製品 $p$ の在庫保管費（/個・月）
* $InitialProdInv_p$: 製品 $p$ の初期在庫量（0ヶ月目末）
* $TargetProdInv_p$: 製品 $p$ の目標期末在庫量（最終月）
* $MaterialPrice_m$: 原材料 $m$ の調達単価
* $Procurement_{mt}$: 月 $t$ における原材料 $m$ の調達量
* $MatHoldingCost_m$: 原材料 $m$ の在庫保管費（/kg・月）
* $InitialMatInv_m$: 原材料 $m$ の初期在庫量（0ヶ月目末）
* $MaterialNeed_{mp}$: 製品 $p$ を1個生産するのに必要な原材料 $m$ の量
* $MachineHours_c$: 機械 $c$ の月間利用可能時間
* $ProcessTime_{cp}$: 製品 $p$ の生産に機械 $c$ が要する時間
* $ProcessingCost_c$: 機械 $c$ の加工費（/時間）

#### 決定変数
* $Produce_{pt} \ge 0$: 月 $t$ に製品 $p$ を生産する量
* $Sell_{pt} \ge 0$: 月 $t$ に製品 $p$ を販売する量
* $ProdInv_{pt_0} \ge 0$: 月 $t_0$ 末時点での製品 $p$ の在庫量 ($t_0 \in T_0$)
* $MatInv_{mt_0} \ge 0$: 月 $t_0$ 末時点での原材料 $m$ の在庫量 ($t_0 \in T_0$)

#### 目的関数
総利益 = (総売上) - (原材料費 + 加工費 + 製品在庫費 + 原材料在庫費) を最大化します。

$$\text{maximize} \quad \sum_{p \in P, t \in T} SalesPrice_p \cdot Sell_{pt} - \left( \sum_{m \in M, t \in T} MaterialPrice_m \cdot Procurement_{mt} + \dots \right)$$

#### 制約条件
1.  **製品の在庫バランス:** (前月の在庫) + (当月の生産量) - (当月の販売量) = (当月の在庫)
    $$ProdInv_{p, t-1} + Produce_{pt} - Sell_{pt} = ProdInv_{pt} \quad (\forall p \in P, \forall t \in T)$$
2.  **原材料の在庫バランス:** (前月の在庫) + (当月の調達量) - (当月の消費量) = (当月の在庫)
    $$MatInv_{m, t-1} + Procurement_{mt} - \sum_{p \in P} MaterialNeed_{mp} \cdot Produce_{pt} = MatInv_{mt} \quad (\forall m \in M, \forall t \in T)$$
3.  **販売量の上限:** 各月の販売量は、その月の需要量を超えることはできません。
    $$Sell_{pt} \le Demand_{pt} \quad (\forall p \in P, \forall t \in T)$$
4.  **機械の稼働時間制約:** 各月の各機械の総稼働時間は、利用可能時間以下でなければなりません。
    $$\sum_{p \in P} ProcessTime_{cp} \cdot Produce_{pt} \le MachineHours_c \quad (\forall c \in C, \forall t \in T)$$
5.  **初期在庫・期末在庫の制約:**
    * $ProdInv_{p,0} = InitialProdInv_p \quad (\forall p \in P)$
    * $MatInv_{m,0} = InitialMatInv_m \quad (\forall m \in M)$
    * $ProdInv_{p, |T|} \ge TargetProdInv_p \quad (\forall p \in P)$



In [2]:
#| hide
%%ampl_eval
# -----------------------------------------------
# モデル (multi_period_planning.mod)
# -----------------------------------------------
reset; model;

# --- 集合 ---
set PRODUCTS;
set MATERIALS;
set MACHINES;

param num_months > 0;
set MONTHS = 1..num_months;
set MONTHS0 = 0..num_months; # 初期在庫(0ヶ月目)を含む期間

# --- パラメータ ---
# 製品関連
param SalesPrice{PRODUCTS} >= 0;
param Demand{PRODUCTS, MONTHS} >= 0;
param ProdHoldingCost{PRODUCTS} >= 0;
param InitialProdInv{PRODUCTS} >= 0;
param TargetProdInv{PRODUCTS} >= 0;

# 原材料関連
param MaterialPrice{MATERIALS} >= 0;
param Procurement{MATERIALS, MONTHS} >= 0;
param MatHoldingCost{MATERIALS} >= 0;
param InitialMatInv{MATERIALS} >= 0;
param MaterialNeed{MATERIALS, PRODUCTS} >= 0;

# 工程・機械関連
param MachineHours{MACHINES} >= 0;
param ProcessTime{MACHINES, PRODUCTS} >= 0;
param ProcessingCost{MACHINES} >= 0;

# --- 変数 ---
var Produce{PRODUCTS, MONTHS} >= 0;   # 生産量
var Sell{PRODUCTS, MONTHS} >= 0;        # 販売量
var ProdInv{PRODUCTS, MONTHS0} >= 0;  # 製品在庫量
var MatInv{MATERIALS, MONTHS0} >= 0;  # 原材料在庫量

# --- 目的関数 ---
# 総利益 = 売上 - (原材料費 + 加工費 + 製品在庫費 + 材料在庫費)
maximize TotalProfit:
    # 売上
    sum{p in PRODUCTS, t in MONTHS} SalesPrice[p] * Sell[p,t]
    # 原材料費 (今回はパラメータとして与えられている調達量に基づく)
    - sum{m in MATERIALS, t in MONTHS} MaterialPrice[m] * Procurement[m,t]
    # 加工費
    - sum{c in MACHINES, p in PRODUCTS, t in MONTHS} ProcessingCost[c] * ProcessTime[c,p] * Produce[p,t]
    # 製品在庫保管費
    - sum{p in PRODUCTS, t in MONTHS} ProdHoldingCost[p] * ProdInv[p,t]
    # 材料在庫保管費
    - sum{m in MATERIALS, t in MONTHS} MatHoldingCost[m] * MatInv[m,t];


# --- 制約条件 ---
# 1. 初期在庫の設定
subject to Initial_Product_Inventory{p in PRODUCTS}:
    ProdInv[p, 0] = InitialProdInv[p];

subject to Initial_Material_Inventory{m in MATERIALS}:
    MatInv[m, 0] = InitialMatInv[m];

# 2. 製品の在庫バランス
subject to Product_Balance{p in PRODUCTS, t in MONTHS}:
    ProdInv[p, t] = ProdInv[p, t-1] + Produce[p,t] - Sell[p,t];

# 3. 原材料の在庫バランス
subject to Material_Balance{m in MATERIALS, t in MONTHS}:
    MatInv[m, t] = MatInv[m, t-1] + Procurement[m,t] - (sum{p in PRODUCTS} MaterialNeed[m,p] * Produce[p,t]);

# 4. 機械の稼働時間制約
subject to Machine_Capacity{c in MACHINES, t in MONTHS}:
    sum{p in PRODUCTS} ProcessTime[c,p] * Produce[p,t] <= MachineHours[c];

# 5. 販売量は需要を超えない
subject to Meet_Demand{p in PRODUCTS, t in MONTHS}:
    Sell[p,t] <= Demand[p,t];

# 6. 期末の目標在庫量を満たす
subject to Target_Product_Inventory{p in PRODUCTS}:
    ProdInv[p, num_months] >= TargetProdInv[p];


# -----------------------------------------------
# データ (multi_period_planning.dat)
# -----------------------------------------------
data;

set PRODUCTS  := 部品X 部品Y;
set MATERIALS := 材料P 材料Q;
set MACHINES  := 機械A 機械B;
param num_months := 3;

# 製品データ
param: SalesPrice ProdHoldingCost InitialProdInv TargetProdInv :=
  部品X   5000       50               50             80
  部品Y   6500       60               30             50;

param Demand :   1    2    3 :=
  部品X            700  600  900
  部品Y            500  700  650;

# 原材料データ
param: MaterialPrice MatHoldingCost InitialMatInv :=
  材料P   300           10               200
  材料Q   500           15               300;

param Procurement : 1    2    3 :=
  材料P              3000 2000 3000
  材料Q              4000 4000 4000;

param MaterialNeed : 部品X 部品Y :=
  材料P                   2.0   1.0
  材料Q                   1.5   3.0;

# 工程・機械データ
param: MachineHours ProcessingCost :=
  機械A     400          2000
  機械B     350          3000;

param ProcessTime : 部品X 部品Y :=
  機械A                 0.15  0.20
  機械B                 0.25  0.30;

# -----------------------------------------------
# 実行と結果表示
# -----------------------------------------------
option solver highs;
solve;

display TotalProfit, Produce, Sell, ProdInv, MatInv;

HiGHS 1.10.0: optimal solution; objective 8579246.667
18 simplex iterations
0 barrier iterations
TotalProfit = 8579250

:       Produce  Sell   ProdInv   MatInv     :=
材料P 0      .       .       .       200
材料P 1      .       .       .      1275
材料P 2      .       .       .      1408.33
材料P 3      .       .       .      2418
材料Q 0      .       .       .       300
材料Q 1      .       .       .      1450
材料Q 2      .       .       .      2550
材料Q 3      .       .       .      3756
部品X 0      .       .     50          .
部品X 1   650       700     0          .
部品X 2   600       600     0          .
部品X 3   706       626    80          .
部品Y 0      .       .     30          .
部品Y 1   625       500   155          .
部品Y 2   666.667   700   121.667      .
部品Y 3   578.333   650    50          .
;



## 6. 新製品発売における広告キャンペーン最適化

**1. 背景と課題**

ある消費財メーカーのマーケティング部門は、新製品「スマートクリーンX」の発売にあたり、大規模な広告キャンペーンを計画しています。キャンペーンの総予算は1,000万円と限られており、この予算内で製品の認知度と初期売上を最大化するために、どの広告媒体にどれだけの予算を配分するかが重要な課題となっています。

担当者は、テレビCM、ラジオCM、新聞広告、Web広告（リスティング広告とSNS広告）、屋外広告など、複数の広告媒体を検討していますが、各媒体は費用、期待できる効果（ターゲット層へのリーチ数）、そして利用上の制約（最低出稿量や最大枠）が異なります。勘や経験だけに頼るのではなく、データに基づいて最適な予算配分を決定し、キャンペーン効果を最大化したいと考えています。

**2. 目標**

*   与えられた総予算1,000万円の範囲内で、広告キャンペーンによる**総リーチ数（想定される接触人数）を最大化**する。

**3. 利用可能な広告媒体とデータ**

マーケティング部門は、過去のデータや媒体社からの情報に基づき、以下のデータを収集しました。

| 広告媒体         | 略称        | 費用単位               | 単位費用(円) | 単位あたりリーチ数(人) | 備考/制約                                     |
| :--------------- | :---------- | :--------------------- | :----------- | :--------------------- | :-------------------------------------------- |
| テレビCM         | TV          | 1回 (15秒スポット)     | 500,000      | 80,000                 | 最大10回まで                                  |
| ラジオCM         | Radio       | 1回 (20秒スポット)     | 100,000      | 15,000                 | 最低5回以上出稿                               |
| 新聞広告         | Newspaper   | 1回 (全国紙半5段)    | 200,000      | 25,000                 | 最大5回まで                                   |
| Web広告(リスティング) | Web_Lis     | 10,000円予算あたり     | 10,000       | 1,000                  | 予算上限 200万円                              |
| Web広告(SNS)     | Web_SNS     | 15,000円予算あたり     | 15,000       | 800                    | 予算上限 250万円                              |
| 屋外広告(主要駅) | OOH         | 1箇所/月               | 1,000,000    | 50,000                 | 最大2箇所まで                                 |

**追加の制約:**

*   **Web広告比率:** Web広告（リスティングとSNSの合計）への予算配分は、総予算の少なくとも30%（300万円）以上とする。

<!-- **4. 数理最適化モデル**

この問題を解決するために、線形計画問題（一部整数計画）としてモデル化します。

*   **決定変数:**
    *   `x_TV`: テレビCMの出稿回数 (0以上の整数)
    *   `x_Radio`: ラジオCMの出稿回数 (0以上の整数)
    *   `x_Newspaper`: 新聞広告の出稿回数 (0以上の整数)
    *   `b_Web_Lis`: Webリスティング広告に投下する予算 (0以上の連続値)
    *   `b_Web_SNS`: Web SNS広告に投下する予算 (0以上の連続値)
    *   `x_OOH`: 屋外広告の出稿箇所数 (0以上の整数)

*   **目的関数 (総リーチ数の最大化):**
    `Maximize:`
    `80000 * x_TV + 15000 * x_Radio + 25000 * x_Newspaper`
    `+ (b_Web_Lis / 10000) * 1000 + (b_Web_SNS / 15000) * 800`
    `+ 50000 * x_OOH`

    *(整理すると)*
    `Maximize:`
    `80000*x_TV + 15000*x_Radio + 25000*x_Newspaper`
    `+ 0.1 * b_Web_Lis + (800/15000) * b_Web_SNS`  *(約 0.0533 * b_Web_SNS)*
    `+ 50000 * x_OOH`

*   **制約条件:**
    1.  **総予算制約:**
        `500000*x_TV + 100000*x_Radio + 200000*x_Newspaper`
        `+ b_Web_Lis + b_Web_SNS + 1000000*x_OOH <= 10000000`
    2.  **テレビCM上限:** `x_TV <= 10`
    3.  **ラジオCM下限:** `x_Radio >= 5`
    4.  **新聞広告上限:** `x_Newspaper <= 5`
    5.  **Webリスティング予算上限:** `b_Web_Lis <= 2000000`
    6.  **Web SNS予算上限:** `b_Web_SNS <= 2500000`
    7.  **屋外広告上限:** `x_OOH <= 2`
    8.  **Web広告比率下限:** `b_Web_Lis + b_Web_SNS >= 3000000`
    9.  **非負制約:**
        `x_TV, x_Radio, x_Newspaper, x_OOH >= 0` (整数)
        `b_Web_Lis, b_Web_SNS >= 0` (連続値)

**5. 期待されるアウトプット (ソルバーによる結果例)**

この数理モデルを最適化ソルバーに入力して解くと、以下の情報が得られます。

*   **最適な予算配分:**
    *   テレビCM (x_TV): 10 回 (予算: 500万円)
    *   ラジオCM (x_Radio): 5 回 (予算: 50万円)
    *   新聞広告 (x_Newspaper): 0 回 (予算: 0円)
    *   Webリスティング広告 (b_Web_Lis): 200万円
    *   Web SNS広告 (b_Web_SNS): 250万円
    *   屋外広告 (x_OOH): 0 箇所 (予算: 0円)
    *   **合計使用予算:** 500 + 50 + 0 + 200 + 250 + 0 = 1000万円 (予算上限ぴったり)
*   **最大期待リーチ数:**
    *   (80000 * 10) + (15000 * 5) + (25000 * 0) + (0.1 * 2000000) + (0.0533 * 2500000) + (50000 * 0)
    *   = 800,000 + 75,000 + 0 + 200,000 + 133,250 + 0 = **1,208,250 人**

**(※上記は仮の最適解です。実際の解はソルバーで計算する必要があります。制約や効果のわずかな違いで最適配分は変わります。例えば、Web広告の効果がもっと高ければ、TVの回数が減る可能性もあります。) ** -->

**4. ケーススタディの意義と実務への応用**

このケーススタディは、数理最適化が広告キャンペーン計画において以下の価値を提供することを示しています。

*   **データに基づいた意思決定:** 経験や勘だけでなく、費用対効果や各種制約を定量的に評価し、最適な予算配分を導き出す。
*   **ROI (投資対効果) の最大化:** 限られた予算内で、キャンペーン目標（ここではリーチ数）を最大化する具体的な計画を立案できる。
*   **複雑な制約の考慮:** 予算上限、媒体ごとの出稿制限、媒体ミックスの比率など、複数の制約条件を同時に満たす解を見つけ出す。
*   **What-if 分析:** もし予算が変わったら？特定の媒体の効果が予測より高かったら/低かったら？といったシナリオ分析を容易に行い、計画の柔軟性を高める。
*   **コミュニケーションツール:** 最適化結果とその根拠を明確に示すことで、関係部署（経営層、営業部門など）との合意形成を円滑にする。


#| hide

### 問題の定義と定式化

#### 1. 問題の定義

ある企業が、新製品の広告キャンペーンのために1,000万円の予算を持っています。複数の広告媒体（テレビ、ラジオ、新聞、Web、屋外広告）を利用可能で、それぞれの媒体は費用、期待できるリーチ数、および出稿量の制約（最小・最大量）が異なります。また、Web広告には総予算の30%以上を割り当てるという追加の制約があります。
この問題の目的は、与えられた制約条件の中で、キャンペーン全体の総リーチ数を最大化する各媒体への最適な予算配分（出稿量）を決定することです。

#### 2. 数理モデルの定式化

この問題を数理最適化問題として定式化します。

**集合とインデックス**

* $M$: 広告媒体の集合 (e.g., TV, Radio, ...)
* $M_{web} \subset M$: Web広告媒体の集合
* $m \in M$: 各広告媒体を指すインデックス

**パラメータ**

* $B$: キャンペーンの総予算 (円)
* $B_{web\_min}$: Web広告に割り当てるべき最小予算 (円)
* $c_m$: 媒体 $m$ の単位あたりの費用 (円/単位)
* $r_m$: 媒体 $m$ の単位あたりのリーチ数 (人/単位)
* $L_m$: 媒体 $m$ の最小出稿量 (単位)
* $U_m$: 媒体 $m$ の最大出稿量 (単位)

※「単位」は媒体ごとに異なります。テレビCMなら「回」、Web広告なら「1万円分の予算」など。

**決定変数**

* $X_m$: 媒体 $m$ の出稿量（単位）。$L_m \le X_m \le U_m$ の範囲の非負の数値です。

**目的関数**

総リーチ数を最大化します。
$$\text{Maximize} \quad \sum_{m \in M} r_m X_m$$

**制約条件**

1.  **総予算制約:**
    キャンペーンの総費用は、与えられた総予算 $B$ を超えてはなりません。
    $$\sum_{m \in M} c_m X_m \le B$$

2.  **Web広告予算比率制約:**
    Web広告への支出合計は、指定された最小Web広告予算 $B_{web\_min}$ 以上でなければなりません。
    $$\sum_{m \in M_{web}} c_m X_m \ge B_{web\_min}$$

3.  **出稿量制約（変数の上下限）:**
    各媒体の出稿量は、それぞれの最小量と最大量の範囲内でなければなりません。これは決定変数の定義に含まれます。
    $$L_m \le X_m \le U_m \quad (\forall m \in M)$$

   

In [10]:
#| hide

%%ampl_eval
reset;
model;

# ------------------------------------------------
# 集合 (Sets)
# ------------------------------------------------
# 広告媒体の集合
set MEDIA;
# Web広告媒体の集合 (MEDIA の部分集合)
set WEB_MEDIA within MEDIA;

# ------------------------------------------------
# パラメータ (Parameters)
# ------------------------------------------------
# キャンペーンの総予算
param total_budget > 0;
# Web広告への最小投資額
param min_web_budget >= 0;

# 各媒体の単位あたりの費用
param unit_cost{MEDIA} >= 0;
# 各媒体の単位あたりのリーチ数
param unit_reach{MEDIA} >= 0;

# 各媒体の最小出稿量 (デフォルトは0)
param min_usage{m in MEDIA} >= 0 default 0;
# 各媒体の最大出稿量 (デフォルトは上限なし)
param max_usage{m in MEDIA} >= min_usage[m] default Infinity;

# ------------------------------------------------
# 変数 (Variables)
# ------------------------------------------------
# 各媒体の出稿量を決定する変数
# 上下限制約は変数宣言に含める
var X{m in MEDIA} >= min_usage[m], <= max_usage[m];

# ------------------------------------------------
# 目的関数 (Objective Function)
# ------------------------------------------------
# 総リーチ数を最大化する
maximize Total_Reach:
  sum{m in MEDIA} unit_reach[m] * X[m];

# ------------------------------------------------
# 制約条件 (Constraints)
# ------------------------------------------------
# 1. 総予算制約
#    広告費の合計は総予算を超えてはならない
subject to Budget_Constraint:
  sum{m in MEDIA} unit_cost[m] * X[m] <= total_budget;

# 2. Web広告への予算比率制約
#    Web広告への支出は指定額以上でなければならない
subject to Web_Budget_Ratio:
  sum{m in WEB_MEDIA} unit_cost[m] * X[m] >= min_web_budget;
  
data;

# 集合の定義
set MEDIA := TV Radio Newspaper Web_Lis Web_SNS OOH;
set WEB_MEDIA := Web_Lis Web_SNS;

# パラメータの定義
param total_budget := 10000000;
param min_web_budget := 3000000; # 総予算の30%

# 各媒体のパラメータ (リスト形式)
# param := medium1 value1 medium2 value2 ...;
param unit_cost :=
  TV        500000
  Radio     100000
  Newspaper 200000
  Web_Lis    10000
  Web_SNS    15000
  OOH       1000000;

param unit_reach :=
  TV        80000
  Radio     15000
  Newspaper 25000
  Web_Lis    1000
  Web_SNS     800
  OOH       50000;

# 最小出稿量 (指定がない媒体はモデルのdefault=0が適用される)
param min_usage :=
  Radio       5;

# 最大出稿量 (指定がない媒体はモデルのdefault=Infinityが適用される)
# Web広告は予算上限を単位費用で割って算出 (200万/1万=200, 250万/1.5万=166.66...)
param max_usage :=
  TV          10
  Newspaper    5
  Web_Lis    200
  Web_SNS    166.66666666
  OOH          2;
  
  # ソルバーの指定、問題の解決、結果の表示
option solver highs;
solve;
display X;

HiGHS 1.10.0: optimal solution; objective 1353333.333
0 simplex iterations
0 barrier iterations
X [*] :=
Newspaper    0
      OOH    0
    Radio   20
       TV   10
  Web_Lis  200
  Web_SNS   66.6667
;



## 7. 地域プロバスケットボールリーグの試合スケジュール最適化

**1. 背景と課題**

地域プロバスケットボールリーグ「リージョナル・フープス」の運営事務局は、来シーズンの試合スケジュール作成に取り組んでいます。リーグには4つのチーム（チームA, チームB, チームC, チームD）が所属しており、ファンやチームからの要望に応え、公平で魅力的なスケジュールを作成する必要があります。

**課題:**

*   **公平性の確保:** 各チームのホーム/アウェイ試合数のバランスを取り、特定のチームに有利/不利が生じないようにしたい。
*   **複雑な制約:** チームの都合（アリーナが他のイベントで使用されるなど）、会場の共有、選手のコンディションを考慮した連戦の回避など、多くの制約を満たす必要がある。
*   **手作業の限界:** これらの条件を手作業で調整して最適なスケジュールを見つけるのは非常に時間がかかり、ミスも発生しやすい。また、本当に「最適」あるいは「公平」なスケジュールになっているか客観的な評価が難しい。

そこで、数理最適化を用いて、全ての制約を満たし、かつ公平性を考慮した試合スケジュールを自動生成することを目指します。

**2. 目標**

*   リーグ戦（ホーム＆アウェイ総当たり）の全試合を、与えられた期間内（6週間）で実施する。
*   全ての運用上の制約（チーム可用性、会場制約、連戦制限）を遵守する。
*   各チームの**ホーム試合数とアウェイ試合数を均等（各3試合**にする。

**3. リーグ設定とデータ**

*   **リーグ構成:** 4チーム (T1, T2, T3, T4)
*   **期間:** 6週間 (W1, W2, W3, W4, W5, W6)
*   **試合形式:** ホーム＆アウェイ方式による総当たり戦。
    *   各チームは他の全チームとホームで1回、アウェイで1回、合計2回対戦する。
    *   総試合数 = 4チーム * 3対戦相手 * 2 (H&A) / 2 = 12試合。
    *   各チームの総試合数 = 6試合。
    *   期間が6週間なので、各チームは毎週1試合を行う。

*   **制約条件:**
    1.  **チーム可用性:** チームT1は、Week 3にホームアリーナが使用できないため、ホームゲームを開催できない。
    2.  **会場共有:** チームT3とチームT4は同じアリーナを共有しているため、同じ週に両チームがホームゲームを行うことはできない。
    3.  **連戦制限:** 選手の負担を考慮し、どのチームも3週連続でホームゲーム、または3週連続でアウェイゲームにならないようにする。

<!-- **4. 数理最適化モデル**

このスケジューリング問題を整数計画問題としてモデル化します。

*   **集合:**
    *   `T`: チームの集合 {T1, T2, T3, T4}
    *   `W`: 週の集合 {1, 2, 3, 4, 5, 6}

*   **決定変数:**
    *   `x[h, a, w]` : 週 `w` ∈ `W` に、チーム `h` ∈ `T` (ホーム) がチーム `a` ∈ `T` (アウェイ, `h ≠ a`) と対戦する場合に 1、そうでない場合に 0 をとるバイナリ変数。

*   **目的関数:**
    全ての制約を満たす実行可能なスケジュールを一つ見つけることが主目的のため、ダミーの目的関数を設定する。
    `Maximize 0`
    *(より高度なモデルでは、移動距離の最小化や特定カード間の間隔調整などを目的関数に含めることも可能)*

*   **制約条件:**
    1.  **各チームは各週にちょうど1試合:**
        全てのチーム `t` ∈ `T`, 全ての週 `w` ∈ `W` について、
        `sum(a ∈ T, a ≠ t) x[t, a, w] + sum(h ∈ T, h ≠ t) x[h, t, w] = 1`
        (チーム `t` がホームで行う試合数 + アウェイで行う試合数 = 1)

    2.  **ホーム＆アウェイ総当たり実施:**
        全ての異なるチームペア `{h, a}` ⊂ `T` について、
        `sum(w ∈ W) x[h, a, w] = 1`
        (チーム `h` がホームでチーム `a` と対戦するのは、期間中にちょうど1回)
        *(これにより、`x[a, h, w]` と合わせ、各ペアの対戦がH&Aで1回ずつ、計2回行われることが保証される)*

    3.  **チームT1のホーム不可制約:**
        `sum(a ∈ T, a ≠ T1) x[T1, a, 3] = 0`
        (Week 3 における T1 のホームゲーム数は 0)

    4.  **会場共有制約 (T3とT4):**
        全ての週 `w` ∈ `W` について、
        `sum(a ∈ T, a ≠ T3) x[T3, a, w] + sum(a ∈ T, a ≠ T4) x[T4, a, w] <= 1`
        (Week `w` における T3 のホームゲーム数と T4 のホームゲーム数の合計は 1 以下)

    5.  **連続ホーム/アウェイ制限:**
        全てのチーム `t` ∈ `T`, 全ての開始週 `w` ∈ {1, 2, 3, 4} について、
        *   (連続ホーム制限) `sum(a ∈ T, a ≠ t) (x[t, a, w] + x[t, a, w+1] + x[t, a, w+2]) <= 2`
        *   (連続アウェイ制限) `sum(h ∈ T, h ≠ t) (x[h, t, w] + x[h, t, w+1] + x[h, t, w+2]) <= 2`
        (任意の3連続週において、ホーム試合数またはアウェイ試合数は2以下)

    6.  **ホーム/アウェイ数の均等化:**
        全てのチーム `t` ∈ `T` について、
        `sum(a ∈ T, a ≠ t) sum(w ∈ W) x[t, a, w] = 3`
        (各チームの総ホーム試合数はちょうど 3 回)
        *(総試合数が6試合のため、これによりアウェイ試合数も自動的に3回となる)*

**5. 期待されるアウトプット (ソルバーによる結果例)**

この数理モデルを最適化ソルバーに入力して解くと、制約をすべて満たす `x[h, a, w] = 1` となる組み合わせ、すなわち具体的な試合スケジュールが出力されます。

**出力例 (スケジュール表):**

| 週   | 試合1 (Home vs Away) | 試合2 (Home vs Away) |
| :--- | :------------------- | :------------------- |
| W1   | T1 vs T2             | T3 vs T4             |
| W2   | T2 vs T3             | T4 vs T1             |
| W3   | T2 vs T4             | T3 vs T1 (T1はアウェイ) |
| W4   | T1 vs T3             | T4 vs T2             |
| W5   | T3 vs T2             | T1 vs T4             |
| W6   | T2 vs T1             | T4 vs T3             |

**(※上記は制約を満たす可能性のある一例です。実際の解はソルバーが出力します。複数の実行可能解が存在する場合もあります。)**

このスケジュールでは、
*   全12試合が6週間で行われている。
*   各ペアがホーム＆アウェイで1回ずつ対戦している。
*   W3でT1はホームで試合していない。
*   各週でT3とT4が同時にホームになっていない。
*   どのチームも3週連続ホーム/アウェイになっていない。
*   各チームのホーム試合数が3回、アウェイ試合数が3回になっている。
    (例: T1のホームは W1, W4, W5。アウェイは W2, W3, W6) -->

**4. ケーススタディの意義と実務への応用**

このケーススタディは、数理最適化がスポーツスケジューリングにおいて以下の価値を提供することを示しています。

*   **公平性の保証:** ホーム/アウェイ数などの公平性に関する基準を明確な制約や目的関数として組み込み、それを満たすスケジュールを作成できる。
*   **複雑な制約の処理:** チーム、会場、連戦など、多岐にわたる複雑なルールや制約を矛盾なく考慮したスケジュールを生成できる。
*   **効率化と客観性:** スケジュール作成にかかる時間と労力を大幅に削減し、属人的な判断ではなくデータとルールに基づいた客観的な結果を得られる。
*   **多様なシナリオ検討:** 新チームの加入、期間の変更、制約の追加・緩和などが発生した場合でも、モデルを修正して迅速に新しいスケジュール案を作成できる。
*   **関係者への説明責任:** 最適化プロセスとその結果は、なぜそのスケジュールになったのかを論理的に説明する根拠となり、チームやファンへの説明責任を果たしやすくなる。



## 8. 日次電力供給計画のコスト最小化

**1. 背景と課題**

地域電力供給会社「エリアパワー株式会社」は、管轄エリア内の家庭や企業への安定的な電力供給を使命としています。同社は複数の発電プラント（火力、水力、太陽光）を保有・運用しており、日々の電力需要は時間帯によって大きく変動します。

**課題:**

*   **コスト効率:** 各発電プラントは発電コスト（燃料費など）や運用特性（出力調整のしやすさ、最低稼働レベルなど）が異なります。コストを抑えつつ電力を供給する必要があります。
*   **需要変動への対応:** 刻々と変わる電力需要を正確に予測し、それに対して過不足なく発電量を調整しなければ、供給不安や電力余剰による損失が発生します。
*   **再生可能エネルギーの活用:** 環境負荷低減のため太陽光発電を導入していますが、その出力は天候や時間帯に左右されるため、他の電源との最適な組み合わせ（ミックス）が求められます。
*   **プラント制約:** 各プラントには最大出力容量があり、火力発電所などでは安定運用のための最低出力レベルも考慮する必要があります。

これらの複雑な要因を考慮し、翌日の各時間帯における最適な発電計画（どのプラントでどれだけ発電するか）を策定し、**総発電コストを最小化**しつつ、**電力需要を確実に満たす**ことが喫緊の課題となっています。

**2. 目標**

*   翌日（24時間）の電力需要予測に基づき、全ての時間帯で需要を満たす。
*   各発電プラントの運用制約（容量、最低出力、太陽光の出力変動）を守る。
*   上記を満たした上で、**1日の総発電コスト（主に燃料費）を最小化**する。

**3. 発電プラントとデータ**

エリアパワー社が運用する発電プラントと、翌日の計画に必要なデータは以下の通りです。
(簡単のため、1日を3つの時間帯：深夜早朝(T1: 0-8時)、日中(T2: 8-16時)、夕夜間(T3: 16-24時)に分割します)

| プラントID | タイプ   | 最大容量(MW) | 最低出力(MW) | 発電コスト(円/MWh) | 備考                                   |
| :--------- | :------- | :----------- | :----------- | :----------------- | :------------------------------------- |
| PL1        | 火力(LNG) | 500          | 100          | 8,000              | 比較的起動・停止・出力調整が容易         |
| PL2        | 火力(石炭)| 800          | 250          | 5,000              | 起動・停止に時間がかかるがコストは安い    |
| PL3        | 水力     | 300          | 0            | 1,000              | 貯水量による制約は今回考慮しない         |
| PL4        | 太陽光   | 200          | 0            | 0                  | 出力は時間帯・天候依存 (後述)          |

**時間帯別データ:**

| 時間帯 | ID | 電力需要予測(MW) | 太陽光(PL4)最大出力(MW) |
| :----- | :- | :--------------- | :---------------------- |
| 深夜早朝 | T1 | 900              | 0                       |
| 日中   | T2 | 1500             | 180                     |
| 夕夜間 | T3 | 1200             | 20                      |

<!-- **4. 数理最適化モデル**

この問題を線形計画問題（一部整数計画）としてモデル化します。

*   **添え字:**
    *   `p`: 発電プラントのインデックス {PL1, PL2, PL3, PL4}
    *   `t`: 時間帯のインデックス {T1, T2, T3}

*   **パラメータ:**
    *   `Cost[p]`: プラントpの発電コスト (円/MWh)
    *   `Capacity[p]`: プラントpの最大発電容量 (MW)
    *   `MinOutput[p]`: プラントpの最低出力 (MW)
    *   `Demand[t]`: 時間帯tの電力需要 (MW)
    *   `SolarMax[t]`: 時間帯tの太陽光プラント(PL4)の最大出力 (MW)

*   **決定変数:**
    *   `Power[p, t]`: プラント `p` が時間帯 `t` に発電する電力量 (MW) (連続変数)
    *   `IsOn[p, t]` : プラント `p` が時間帯 `t` に稼働（発電）している場合に 1、していない場合に 0 をとるバイナリ変数。（最低出力制約のため）

*   **目的関数 (総発電コスト最小化):**
    `Minimize sum(p in {PL1, PL2, PL3, PL4}) sum(t in {T1, T2, T3}) Cost[p] * Power[p, t]`

*   **制約条件:**
    1.  **需要充足:** 各時間帯 `t` において、総発電量は需要以上であること。
        `sum(p in {PL1, PL2, PL3, PL4}) Power[p, t] >= Demand[t]` (for all t ∈ {T1, T2, T3})

    2.  **最大容量制約:** 各プラントの発電量は、その最大容量以下であること。
        `Power[p, t] <= Capacity[p] * IsOn[p, t]` (for all p, t)

    3.  **最低出力制約:** 稼働している場合、発電量は最低出力以上であること。
        `Power[p, t] >= MinOutput[p] * IsOn[p, t]` (for p ∈ {PL1, PL2}) (for all t)
        *(水力、太陽光は最低出力0なので、この形式で `MinOutput=0` とするか、対象プラントを限定する)*
        *(PL3, PL4については、 `Power[p, t] >= 0` で十分)*

    4.  **太陽光出力上限:** 太陽光プラントの発電量は、その時間帯の最大出力以下であること。
        `Power[PL4, t] <= SolarMax[t]` (for all t ∈ {T1, T2, T3})

    5.  **非負制約:** 発電量は0以上。
        `Power[p, t] >= 0` (for all p, t)

    6.  **バイナリ変数定義:** `IsOn[p, t]` は 0 または 1。

**5. 期待されるアウトプット (ソルバーによる結果例)**

このモデルを最適化ソルバーに入力して解くと、以下の情報が得られます。

*   **各時間帯の発電計画:**
    *   例:
        *   **T1 (深夜早朝, 需要900MW):** PL2(石炭) 700MW, PL1(LNG) 100MW, PL3(水力) 100MW, PL4(太陽光) 0MW (合計900MW)。 コストが安いPL2を最大限使い、最低出力に合わせてPL1も稼働、残りを水力で補う。
        *   **T2 (日中, 需要1500MW):** PL2 800MW(最大), PL1 370MW, PL3 150MW, PL4 180MW(最大) (合計1500MW)。 太陽光を最大活用し、安価なPL2も最大出力。残りをPL1と水力で賄う。
        *   **T3 (夕夜間, 需要1200MW):** PL2 800MW(最大), PL1 250MW, PL3 130MW, PL4 20MW(最大) (合計1200MW)。 太陽光は減るが、引き続きPL2を最大活用し、残りを調整可能なPL1と水力で賄う。
*   **最小総発電コスト:** 上記計画を実行した場合の1日の総コスト(円)。
*   **プラント稼働状況:** 各プラントがどの時間帯に稼働しているか (`IsOn[p, t]` の値)。
*   **(感度分析など)** 需要予測の変動や燃料価格の変動が、最適な発電計画とコストにどう影響するかを分析することも可能。

**(※上記は、制約とコスト構造に基づいた典型的な結果の例です。実際の最適解はソルバーによる計算が必要です。最低出力制約などがなければ、より単純にコストの安い順に使われる傾向になります。)** -->

**4. ケーススタディの意義と実務への応用**

このケーススタディは、数理最適化が電力供給計画において以下の価値を提供することを示しています。

*   **コスト削減:** 燃料費や運用コストを考慮し、最も経済的な発電所の組み合わせと出力を決定することで、大幅なコスト削減を実現できる。
*   **安定供給の確保:** 需要予測とプラントの能力・制約に基づいて計画を立てることで、電力不足のリスクを低減し、安定供給を支援する。
*   **再生可能エネルギーの統合:** 出力が不安定な再生可能エネルギー源を、他の調整可能な電源と組み合わせて最大限有効活用する計画を立案できる。
*   **複雑な運用制約への対応:** 最大/最小出力、起動停止時間（より高度なモデルで考慮）、送電網の制約（これも高度なモデルで考慮）など、現実の複雑な制約下での最適運用計画を作成できる。
*   **迅速な計画修正:** 需要予測やプラントの状態が変化した場合でも、モデルを更新して迅速に最適な計画を再計算できる。

電力系統の運用計画（ユニットコミットメント、経済負荷配分など）は、数理最適化が極めて重要かつ効果的に活用されている分野の一つであり、エネルギーコストの削減と供給信頼性の向上に不可欠なツールとなっています。


In [ ]:
#| hide

### 問題の定義

地域電力供給会社が、複数の特性が異なる発電プラント（火力、水力、太陽光）を運用し、変動する電力需要に応えるための翌日の発電計画を最適化する問題です。

**目標:**
各発電プラントの運用制約を満たしつつ、1日（24時間）の総発電コストを最小化します。

**課題:**
1.  **コスト効率:** 発電コストが異なるプラントを最適に組み合わせる必要があります。
2.  **需要変動:** 1日を3つの時間帯（深夜早朝、日中、夕夜間）に分け、それぞれの需要予測を満たす必要があります。
3.  **プラント制約:** 各プラントの最大/最小出力容量や、天候に依存する太陽光発電の出力変動を考慮しなければなりません。

### 定式化

この問題を数理最適化モデルとして定式化します。

**1. 集合 (Sets)**

* $P$: 発電プラントの集合 (例: PL1, PL2, PL3, PL4)
* $T$: 時間帯の集合 (例: T1, T2, T3)
* $P_{thermal} \subseteq P$: 最低出力制約のある火力プラントの集合 (例: PL1, PL2)

**2. パラメータ (Parameters)**

* $demand_t$: 時間帯 $t \in T$ における電力需要予測 (MW)
* $max\_capacity_p$: プラント $p \in P$ の最大発電容量 (MW)
* $min\_output_p$: 火力プラント $p \in P_{thermal}$ の最低出力 (MW)
* $cost_p$: プラント $p \in P$ の発電コスト (円/MWh)
* $solar\_max\_output_t$: 時間帯 $t \in T$ における太陽光プラントの最大出力 (MW)
* $hours\_per\_step$: 各時間帯の長さ (時間)

**3. 決定変数 (Variables)**

* $Generate_{p,t} \ge 0$: プラント $p \in P$ が時間帯 $t \in T$ に発電する量 (MW)
* $IsOn_{p,t} \in \{0, 1\}$: 時間帯 $t \in T$ に火力プラント $p \in P_{thermal}$ が稼働しているか否かを表すバイナリ変数 (1: 稼働, 0: 停止)

**4. 目的関数 (Objective Function)**

1日の総発電コストを最小化します。

$$\text{Minimize} \quad \text{TotalCost} = \sum_{p \in P} \sum_{t \in T} (cost_p \times Generate_{p,t} \times hours\_per\_step)$$

**5. 制約条件 (Constraints)**

1.  **需要供給バランス:** 各時間帯において、総発電量は需要予測以上でなければなりません。
    $$
    \sum_{p \in P} Generate_{p,t} \ge demand_t \quad (\forall t \in T)
    $$
2.  **火力プラントの出力範囲:** 火力プラントは、稼働している場合（$IsOn_{p,t}=1$）は最低出力と最大容量の間で発電し、停止している場合（$IsOn_{p,t}=0$）は発電量が0になります。
    $$
    min\_output_p \times IsOn_{p,t} \le Generate_{p,t} \le max\_capacity_p \times IsOn_{p,t} \quad (\forall p \in P_{thermal}, \forall t \in T)
    $$
3.  **太陽光プラントの出力上限:** 太陽光プラントの発電量は、時間帯ごとの最大出力予測を超えることはできません。（問題文より太陽光プラントは'PL4'とします）
    $$
    Generate_{\text{'PL4'},t} \le solar\_max\_output_t \quad (\forall t \in T)
    $$
4.  **その他プラントの出力上限:** 上記以外のプラント（水力など）は、それぞれの最大容量を超えて発電できません。
    $$
    Generate_{p,t} \le max\_capacity_p \quad (\forall p \in P \setminus (P_{thermal} \cup \{\text{'PL4'}\}), \forall t \in T)
    $$


In [13]:
#| hide

%%ampl_eval
reset;
model;

# --- 1. 集合の定義 ---

# 全ての発電プラント
set PLANTS;

# 計画対象の時間帯
set TIMESTEPS;

# 最低出力制約を持つ火力プラント (PLANTSの部分集合)
set THERMAL_PLANTS within PLANTS;


# --- 2. パラメータの定義 ---

# 時間帯ごとの電力需要 (MW)
param demand{TIMESTEPS} >= 0;

# 各プラントの最大容量 (MW)
param max_capacity{PLANTS} >= 0;

# 火力プラントの最低出力 (MW)
param min_output{THERMAL_PLANTS} >= 0;

# 各プラントの発電コスト (円/MWh)
param cost{PLANTS} >= 0;

# 太陽光の時間帯別最大出力 (MW)
param solar_max_output{TIMESTEPS} >= 0;

# 各時間帯の長さ (時間)
param hours_per_step > 0;


# --- 3. 変数の定義 ---

# Generate[p,t]: プラントpが時間帯tに発電する量 (MW)
var Generate{p in PLANTS, t in TIMESTEPS} >= 0;

# IsOn[p,t]: 時間帯tに火力プラントpが稼働しているか (1:稼働, 0:停止)
var IsOn{p in THERMAL_PLANTS, t in TIMESTEPS} binary;


# --- 4. 目的関数の定義 ---

# 総発電コストを最小化する
minimize TotalCost:
  hours_per_step * sum{p in PLANTS, t in TIMESTEPS} cost[p] * Generate[p, t];


# --- 5. 制約条件の定義 ---

# 制約1: 需要供給バランス
# 各時間帯で、総発電量が需要を上回ること
subject to Meet_Demand{t in TIMESTEPS}:
  sum{p in PLANTS} Generate[p, t] >= demand[t];

# 制約2: 火力プラントの出力制約
# 上限制約：稼働していなければ発電量は0、稼働していれば最大容量以下
subject to Thermal_Capacity_Upper{p in THERMAL_PLANTS, t in TIMESTEPS}:
  Generate[p, t] <= max_capacity[p] * IsOn[p, t];

# 下限制約：稼働していれば最低出力以上
subject to Thermal_Capacity_Lower{p in THERMAL_PLANTS, t in TIMESTEPS}:
  Generate[p, t] >= min_output[p] * IsOn[p, t];

# 制約3: 太陽光プラント('PL4')の出力制約
# 時間帯ごとの最大出力予測を超えられない
subject to Solar_Capacity_Upper{t in TIMESTEPS}:
  Generate['PL4', t] <= solar_max_output[t];

# 制約4: その他のプラント(水力)の出力制約
# THERMAL_PLANTS と 'PL4' 以外のプラント
subject to Other_Capacity_Upper{p in PLANTS diff THERMAL_PLANTS diff {'PL4'}, t in TIMESTEPS}:
  Generate[p, t] <= max_capacity[p];


################################################################################
data;

# --- 集合のデータ ---
set PLANTS := PL1 PL2 PL3 PL4;
set TIMESTEPS := T1 T2 T3;
set THERMAL_PLANTS := PL1 PL2;


# --- パラメータのデータ ---
param hours_per_step := 8;

# 電力需要予測 (MW)
param demand :=
  T1   900
  T2  1500
  T3  1200;

# 最大容量 (MW)
param max_capacity :=
  PL1 500
  PL2 800
  PL3 300
  PL4 200;

# 最低出力 (MW)
param min_output :=
  PL1 100
  PL2 250;

# 発電コスト (円/MWh) (太陽光の燃料費は0と仮定)
param cost :=
  PL1 8000
  PL2 5000
  PL3 1000
  PL4 0;

# 太陽光の最大出力 (MW)
param solar_max_output :=
  T1   0
  T2 180
  T3  20;

################################################################################
option solver highs;
solve;

# --- 結果の表示 ---
display Generate, IsOn, TotalCost;

HiGHS 1.10.0: optimal solution; objective 114880000
1 simplex iterations
1 branching nodes
:      Generate IsOn    :=
PL1 T1      0     0
PL1 T2    220     1
PL1 T3    100     1
PL2 T1    600     1
PL2 T2    800     1
PL2 T3    780     1
PL3 T1    300      .
PL3 T2    300      .
PL3 T3    300      .
PL4 T1      0      .
PL4 T2    180      .
PL4 T3     20      .
;

TotalCost = 114880000



## 9. 農場の季節別作物栽培計画による収益最大化

**1. 背景と課題**

ある家族経営の農場「グリーンフィールド・ファーム」は、100ヘクタールの耕作可能な土地を所有しています。農場主は、来シーズンに向けてどの作物をどれだけの面積で作付けし、どの程度の資源（水、肥料、労働力）を投入すれば、農場全体の収益を最大化できるか悩んでいます。

**課題:**

*   **作物選択:** 栽培可能な作物は複数（例: トマト、小麦、トウモロコシ）あり、それぞれ市場価格、収穫量、必要な資源（土地、水、肥料、労働時間）、栽培期間が異なります。
*   **資源制約:** 利用可能な土地面積、水利権に基づく月間取水量、購入可能な肥料の量、そして家族や臨時雇いを含めた月間の総労働時間には限りがあります。
*   **季節性・天候:** 作物ごとに最適な作付け時期と収穫時期があり、また月ごとの天候（主に降水量）が水の必要量に影響します。豊作/不作による収穫量や価格の変動リスクもあります（今回は簡略化のため、平均的な値を使用）。
*   **輪作計画（簡略化）:** 特定の作物を連作することによる土壌への影響も考慮する必要がありますが、ここでは特定の組み合わせを避ける簡単な制約のみ考えます。

農場主は、これらの要因を総合的に考慮し、データに基づいて収益を最大化する**年間（または季節別）の作物栽培計画**を立てたいと考えています。

**2. 目標**

*   限られた土地、水、肥料、労働力の中で、年間を通じて栽培する作物の種類と面積を決定する。
*   各資源の制約と作物の栽培要件を満たす。
*   上記を満たした上で、**年間の総粗利益（総売上 - 総変動費）を最大化**する。

**3. 作物と資源データ**

ここでは、簡単のため主要な3つの作物と、春・夏・秋の3つの栽培期間を考えます。

**作物データ (1ヘクタールあたり):**

| 作物     | 略称  | 期間 | 予想収量(トン) | 市場価格(円/トン) | 水必要量(m³/月) | 肥料必要量(kg) | 労働時間(時間/月) | 備考                 |
| :------- | :---- | :--- | :------------- | :---------------- | :-------------- | :------------- | :---------------- | :------------------- |
| トマト   | TOM   | 春夏 | 50             | 30,000            | 春:300, 夏:400  | 100            | 春:50, 夏:80     | 連作不可 (前年考慮) |
| 小麦     | WHT   | 春   | 6              | 200,000           | 春:200          | 150            | 春:30            |                      |
| トウモロコシ | COR   | 夏秋 | 10             | 80,000            | 夏:350, 秋:250  | 120            | 夏:40, 秋:60     | 小麦の後作は推奨     |

**資源制約:**

| 資源       | 単位       | 春の上限 | 夏の上限 | 秋の上限 | 変動費        |
| :--------- | :--------- | :------- | :------- | :------- | :------------ |
| 土地       | ヘクタール | 100      | 100      | 100      | (作付で消費)  |
| 水         | m³         | 30,000   | 35,000   | 25,000   | 20 円/m³      |
| 肥料       | kg         | 12,000   | 12,000   | 12,000   | 150 円/kg     |
| 労働力     | 時間       | 4,000    | 5,000    | 4,500    | 1,500 円/時間 |

**その他の制約:**

*   **連作回避:** トマトは、前年にトマトを栽培した区画では栽培できない。（簡単のため、今回は全ての土地で前年はトマト以外だったと仮定）
*   **後作推奨:** 小麦を春に栽培した土地では、夏にトウモロコシを栽培することを推奨する（が、必須ではない）。

<!-- **4. 数理最適化モデル**

この問題を線形計画問題としてモデル化します。

*   **添え字:**
    *   `c`: 作物のインデックス {TOM, WHT, COR}
    *   `t`: 期間のインデックス {Spring, Summer, Autumn}

*   **パラメータ (作物c, 期間tあたり):**
    *   `Yield[c]`: 作物cの収量 (トン/ha)
    *   `Price[c]`: 作物cの市場価格 (円/トン)
    *   `WaterNeed[c, t]`: 作物cの期間tにおける水必要量 (m³/ha/月)
    *   `FertNeed[c]`: 作物cの肥料必要量 (kg/ha) *(期間によらないと仮定)*
    *   `LaborNeed[c, t]`: 作物cの期間tにおける労働時間 (時間/ha/月)
    *   `LandLimit`: 利用可能な総土地面積 (ha)
    *   `WaterLimit[t]`: 期間tの利用可能な水の上限 (m³)
    *   `FertLimit[t]`: 期間tの利用可能な肥料の上限 (kg)
    *   `LaborLimit[t]`: 期間tの利用可能な労働時間の上限 (時間)
    *   `WaterCost`: 水の単価 (円/m³)
    *   `FertCost`: 肥料の単価 (円/kg)
    *   `LaborCost`: 労働力の単価 (円/時間)
    *   `CropDuration[c]`: 作物cが土地を占有する期間の集合 (例: `CropDuration[TOM] = {Spring, Summer}`)

*   **決定変数:**
    *   `Area[c]`: 作物 `c` の作付面積 (ヘクタール) (0以上の連続値)

*   **目的関数 (総粗利益の最大化):**
    `Maximize:`
    `sum(c) ( Yield[c] * Price[c] * Area[c] )`  *(総売上)*
    `- sum(c) sum(t in CropDuration[c]) ( WaterNeed[c, t] * WaterCost * Area[c] )` *(総水コスト)*
    `- sum(c) ( FertNeed[c] * FertCost * Area[c] )` *(総肥料コスト)*
    `- sum(c) sum(t in CropDuration[c]) ( LaborNeed[c, t] * LaborCost * Area[c] )` *(総労働コスト)*

*   **制約条件:**
    1.  **土地制約:** 各期間 `t` において、土地を使用している作物の総面積は上限以下。
        `sum(c | t ∈ CropDuration[c]) Area[c] <= LandLimit` (for all t ∈ {Spring, Summer, Autumn})

    2.  **水制約:** 各期間 `t` において、必要な水の総量は上限以下。
        `sum(c | t ∈ CropDuration[c]) WaterNeed[c, t] * Area[c] <= WaterLimit[t]` (for all t)

    3.  **肥料制約:** 各期間 `t` において、必要な肥料の総量は上限以下 (肥料は作付け時に投入すると仮定し、最初の期間で制約)。
        `sum(c | t_start ∈ CropDuration[c]) FertNeed[c] * Area[c] <= FertLimit[t_start]` (for each starting period t_start)
        *(より正確には、施肥タイミングに応じて制約期間を設定)*

    4.  **労働力制約:** 各期間 `t` において、必要な労働時間の総量は上限以下。
        `sum(c | t ∈ CropDuration[c]) LaborNeed[c, t] * Area[c] <= LaborLimit[t]` (for all t)

    5.  **非負制約:** `Area[c] >= 0` (for all c)

    **(連作回避や後作推奨の制約は、より詳細な区画ごとのモデルや整数変数を使えば追加可能だが、ここでは簡略化)**

**5. 期待されるアウトプット (ソルバーによる結果例)**

このモデルを線形計画ソルバーに入力して解くと、以下の情報が得られます。

*   **最適な作付面積:**
    *   トマト (Area[TOM]): 例えば 40 ヘクタール
    *   小麦 (Area[WHT]): 例えば 60 ヘクタール
    *   トウモロコシ (Area[COR]): 例えば 50 ヘクタール
    *(注意: 小麦とトウモロコシは期間が異なるため、合計が100haを超えることがある。土地制約は期間ごとにチェックされる)*
*   **最大期待粗利益:** 上記の作付け計画によって得られる年間の総粗利益(円)。
*   **資源利用状況:** 各期間における土地、水、肥料、労働力の利用率。どの資源がボトルネック（制約上限に達しているか）になっているかがわかる。例えば、夏の水や労働力が不足しがちであることが判明するかもしれない。
*   **(感度分析):** 特定の作物の市場価格が変動した場合や、資源（水など）の利用可能量が変わった場合に、最適な作付け計画と収益がどのように変化するかを分析できる。 -->

**4. ケーススタディの意義と実務への応用**

このケーススタディは、数理最適化が農業経営において以下の価値を提供することを示しています。

*   **収益性の向上:** 市場価格、コスト、収量、資源制約を総合的に考慮し、最も収益性の高い作物ミックスと作付面積を決定できる。
*   **資源の効率的利用:** 土地、水、肥料、労働力といった限られた資源を最も効果的に配分し、無駄を削減する。ボトルネックとなっている資源を特定し、改善策（例: 節水技術の導入、労働力の確保）の検討に繋げる。
*   **リスク管理:** 天候不順や価格変動のリスクを考慮したシナリオ分析を行い、より頑健な生産計画を立てるための情報を提供する（感度分析）。
*   **計画策定の高度化:** 経験や勘に頼るだけでなく、データに基づいた客観的で合理的な栽培計画を迅速に作成できる。
*   **持続可能性への貢献:** 水や肥料などの投入量を最適化することで、環境負荷の低減にも貢献できる可能性がある。

農業生産計画は、天候などの不確実性も伴う複雑な問題ですが、数理最適化を用いることで、より科学的で収益性の高い、持続可能な農業経営を目指すための強力なツールとなります。


#| hide

### 1. 問題の定義と定式化

この問題は、限られた資源（土地、水、肥料、労働力）のもとで、年間の総粗利益を最大化する作物の作付面積を決定する線形計画問題として定式化できます。

#### 1.1. 集合とインデックス
-   $C$: 作物の集合 (e.g., TOM: トマト, WHT: 小麦, COR: トウモロコシ)
-   $P$: 栽培期間の集合 (e.g., SPR: 春, SUM: 夏, AUT: 秋)
-   $R$: 資源の集合 (e.g., WATER: 水, FERTILIZER: 肥料, LABOR: 労働力)

#### 1.2. パラメータ
-   $\text{land\_limit}$: 利用可能な総土地面積 (ヘクタール)
-   $\text{yield}_c$: 作物 $c \in C$ の1ヘクタールあたりの収量 (トン)
-   $\text{price}_c$: 作物 $c \in C$ の市場価格 (円/トン)
-   $\text{resource\_cost}_r$: 資源 $r \in R$ の単位あたりの変動費 (円/m³, 円/kg, 円/時間)
-   $\text{resource\_limit}_{r,p}$: 期間 $p \in P$ における資源 $r \in R$ の利用上限
-   $\text{resource\_consumption}_{c,r,p}$: 作物 $c \in C$ を1ヘクタール栽培するのに必要な、期間 $p \in P$ における資源 $r \in R$ の量
-   $\text{occupies}_{c,p}$: 作物 $c \in C$ が期間 $p \in P$ に土地を占有する場合 1、そうでなければ 0

#### 1.3. 決定変数
-   $\text{CultivateArea}_c \ge 0$: 作物 $c \in C$ の作付面積 (ヘクタール)

#### 1.4. 目的関数
年間の総粗利益（総売上 - 総変動費）を最大化します。
-   **総売上**: $\sum_{c \in C} (\text{yield}_c \times \text{price}_c \times \text{CultivateArea}_c)$
-   **総変動費**: $\sum_{c \in C} \sum_{r \in R} \sum_{p \in P} (\text{resource\_consumption}_{c,r,p} \times \text{resource\_cost}_r \times \text{CultivateArea}_c)$

これをまとめると、以下の目的関数となります。
$$
\text{Maximize} \quad \sum_{c \in C} \text{CultivateArea}_c \left( \text{yield}_c \cdot \text{price}_c - \sum_{p \in P} \sum_{r \in R} \text{resource\_consumption}_{c,r,p} \cdot \text{resource\_cost}_r \right)
$$

#### 1.5. 制約条件
1.  **土地制約**: 各期間において、作付けされている総面積は土地の上限を超えてはなりません。
    $$
    \forall p \in P, \quad \sum_{c \in C} (\text{occupies}_{c,p} \times \text{CultivateArea}_c) \le \text{land\_limit}
    $$
2.  **資源制約**: 各期間、各資源において、総消費量は利用上限を超えてはなりません。
    $$
    \forall r \in R, \forall p \in P, \quad \sum_{c \in C} (\text{resource\_consumption}_{c,r,p} \times \text{CultivateArea}_c) \le \text{resource\_limit}_{r,p}
    $$
3.  **非負制約**: 作付面積は0以上でなければなりません。
    $$
    \forall c \in C, \quad \text{CultivateArea}_c \ge 0
    $$


In [11]:
#| hide

%%ampl_eval
reset;
model;

# 1. 集合 (Sets)
set CROPS;      # 作物の集合
set PERIODS;    # 期間の集合
set RESOURCES;  # 資源の集合

# 2. パラメータ (Parameters)
param land_limit > 0;                                 # 利用可能な総土地面積 (ha)
param yield{CROPS} >= 0;                              # 作物ごとの収量 (トン/ha)
param price{CROPS} >= 0;                              # 作物ごとの市場価格 (円/トン)
param resource_cost{RESOURCES} >= 0;                  # 資源ごとの変動費 (円/単位)
param resource_limit{RESOURCES, PERIODS} >= 0;        # 期間・資源ごとの利用上限

# 作物cが期間pに土地を占有するか (1=yes, 0=no)
param occupies{CROPS, PERIODS} binary default 0;

# 作物cが期間pで資源rを消費する量 (単位/ha)
param resource_consumption{CROPS, RESOURCES, PERIODS} >= 0 default 0;

# 3. 決定変数 (Variables)
var CultivateArea{c in CROPS} >= 0; # 作物cの作付面積 (ha)

# 4. 目的関数 (Objective Function)
# 総粗利益 = 総売上 - 総変動費
maximize GrossProfit:
    sum{c in CROPS} CultivateArea[c] * (
        yield[c] * price[c] -
        sum{p in PERIODS, r in RESOURCES} resource_consumption[c, r, p] * resource_cost[r]
    );

# 5. 制約条件 (Constraints)
# 土地制約: 各期間で使用される土地の合計は上限以下
subject to LandConstraint{p in PERIODS}:
    sum{c in CROPS} occupies[c, p] * CultivateArea[c] <= land_limit;

# 資源制約: 各期間・各資源の総消費量は上限以下
subject to ResourceConstraint{r in RESOURCES, p in PERIODS}:
    sum{c in CROPS} resource_consumption[c, r, p] * CultivateArea[c] <= resource_limit[r, p];

# --- データ部 ---
data;

# 集合の定義
set CROPS := TOM WHT COR;
set PERIODS := SPR SUM AUT;
set RESOURCES := WATER FERTILIZER LABOR;

# パラメータ値の定義
param land_limit := 100;

param yield :=
    TOM 50
    WHT 6
    COR 10 ;

param price :=
    TOM 30000
    WHT 200000
    COR 80000 ;

param resource_cost :=
    WATER 20
    FERTILIZER 150
    LABOR 1500 ;

# 期間ごとの資源利用上限
param resource_limit :=
    WATER      SPR   30000
    WATER      SUM   35000
    WATER      AUT   25000
    FERTILIZER SPR   12000
    FERTILIZER SUM   12000
    FERTILIZER AUT   12000
    LABOR      SPR   4000
    LABOR      SUM   5000
    LABOR      AUT   4500 ;

# 作物が期間に土地を占有するかのフラグ
param occupies :=
    TOM SPR 1
    TOM SUM 1
    WHT SPR 1
    COR SUM 1
    COR AUT 1 ;

# 作物ごとの資源消費量 (1ヘクタールあたり)
param resource_consumption :=
    # トマト
    TOM WATER      SPR 300
    TOM WATER      SUM 400
    TOM FERTILIZER SPR 100
    TOM LABOR      SPR 50
    TOM LABOR      SUM 80
    # 小麦
    WHT WATER      SPR 200
    WHT FERTILIZER SPR 150
    WHT LABOR      SPR 30
    # トウモロコシ
    COR WATER      SUM 350
    COR WATER      AUT 250
    COR FERTILIZER SUM 120
    COR LABOR      SUM 40
    COR LABOR      AUT 60 ;

# --- 実行部 ---
option solver highs;
solve;

# 結果の表示
display CultivateArea;
display GrossProfit;

HiGHS 1.10.0: optimal solution; objective 148235208.3
4 simplex iterations
0 barrier iterations
CultivateArea [*] :=
COR  75
TOM  21.875
WHT  65.4167
;

GrossProfit = 148235000



## 10. 国内線航空会社のフリートアサインメントと基本スケジュール最適化

**1. 背景と課題**

国内線を運航する航空会社「スカイウィング航空」は、競争激化と燃料費高騰の中で、収益性を改善する必要に迫られています。現在、保有する複数のタイプの航空機（異なる座席数、燃費、航続距離）を、様々な国内路線に割り当てていますが、その割り当てと基本的な運航スケジュールは、過去の経験や部分的な調整に基づいており、必ずしもコスト効率や収益性が最大化されているとは言えません。

**課題:**

*   **コスト効率の最適化:** どの路線にどの機材タイプを投入すれば、燃料費や運航コスト全体を最小化できるか。
*   **需要と供給のマッチング:** 各路線の時間帯別需要予測に対して、適切なサイズの機材を割り当て、空席を減らしつつ機会損失（需要を取りこぼすこと）も最小限に抑えたい。
*   **機材繰りの複雑さ:** 機材は空港間を移動するため、ある便の到着機が次の便の出発機となる「機材繰り」を考慮する必要がある。効率的なローテーションを組まなければ、機材の遊休時間が増えたり、必要な場所に必要な機材がないという事態が発生する。
*   **空港制約:** 主要空港では離着陸可能な便数（スロット）が限られており、特に混雑する時間帯では制約が厳しい。

これらの課題に対し、数理最適化を用いて、**総運航コストを最小化**しつつ、**需要を満たし、各種制約を遵守する**実現可能な**機材割り当て計画と基本的な運航スケジュール**を作成することを目指します。

**2. 目標**

*   翌日の全ての計画フライト（主要路線）に対して、最適な航空機タイプを割り当てる。
*   各路線の旅客需要予測を可能な限り満たす。
*   機材の連続的な運用（フロー）を保証し、機材不足や矛盾がないようにする。
*   空港の離着陸スロット制約を遵守する。
*   上記を満たした上で、**1日の総運航コスト（主に燃料費ベースの変動費）を最小化**する。

**3. 設定とデータ**

*   **対象空港 (A):** {HND (東京羽田), ITM (大阪伊丹), CTS (札幌千歳), FUK (福岡)}
*   **航空機タイプ (P):**
    *   Type A (中型機): 座席数 150席, 運航コスト 50万円/飛行時間
    *   Type B (大型機): 座席数 250席, 運航コスト 80万円/飛行時間
*   **保有 機材数:** Type A: 10機, Type B: 5機
*   **フライト候補 (F):** 主要路線における翌日の運航候補便。各フライト `f` は (出発空港, 到着空港, 出発時刻, 到着時刻, 飛行時間) を持つ。
    (例: f1: HND→ITM 08:00発-09:00着, 飛行時間1h; f2: ITM→FUK 10:00発-11:15着, 飛行時間1.25h ...)
*   **路線需要 (Demand):** 各路線（例: HND-ITM）、時間帯ごとの旅客需要予測。フライト候補 `f` がカバーすべき需要 `Demand[f]` として簡略化。
    (例: f1(HND-ITM 08:00発)の需要は 180人)
*   **空港スロット (Slot):** 各空港 `a`、各時間帯（例: 1時間ごと）`t` の離着陸可能回数 `SlotLimit[a, t]`。
    (例: HNDの08:00-08:59の離着陸枠は合計 40回)
*   **機材の初期配置:** 運用開始時点（例: 早朝）での各機材タイプの各空港における待機機数。

<!-- **4. 数理最適化モデル**

この問題を整数計画問題（フリートアサインメント問題）としてモデル化します。

*   **決定変数:**
    *   `x[f, p]` : フライト候補 `f` ∈ `F` を機材タイプ `p` ∈ `P` で運航する場合に 1、そうでない場合に 0 をとるバイナリ変数。
    *   `Ground[p, a, t]`: 機材タイプ `p` が、空港 `a` ∈ `A` で時刻 `t` において地上待機している機数（非負整数）。

*   **目的関数 (総運航コスト最小化):**
    `Minimize sum(f ∈ F) sum(p ∈ P) FlightCost(f, p) * x[f, p]`
    *(FlightCost(f, p) はフライトfの飛行時間と機材pのコスト/時間から計算)*

*   **制約条件:**
    1.  **各フライトは高々1つの機材タイプで運航される:**
        全てのフライト `f` ∈ `F` について、
        `sum(p ∈ P) x[f, p] <= 1`
        *(需要を満たせない場合は運航しない選択肢も含む場合。必ず運航する場合は '= 1' とするが、機材不足などで実行不可能になる可能性)*

    2.  **需要充足制約 (簡略化):** 運航する場合、機材容量が需要以上であることが望ましい。
        全てのフライト `f` ∈ `F`, 機材タイプ `p` ∈ `P` について、
        `SeatCapacity[p] * x[f, p] >= Demand[f] * x[f, p]`
        *(より厳密には、空席コストや逸失利益を目的関数に入れるか、運航しない場合のペナルティを設定する)*
        *(簡単のため、ここでは運航する場合は容量が足りる機材を選ぶという制約として表現)*
        `sum(p ∈ P | SeatCapacity[p] >= Demand[f]) x[f, p] = sum(p ∈ P) x[f, p]`

    3.  **機材フロー保存制約:** 各空港 `a`、各時刻 `t` (離散時間ステップ)、各機材タイプ `p` について、空港に流入する機材数と流出する機材数がバランスする。
        `Ground[p, a, t-1]`  *(前の時刻からの待機機数)*
        `+ sum(f ∈ F | ArrivalAirport(f)=a, ArrivalTime(f)=t) x[f, p]` *(時刻tに到着した機数)*
        `= Ground[p, a, t]` *(時刻tの待機機数)*
        `+ sum(f ∈ F | DepartureAirport(f)=a, DepartureTime(f)=t) x[f, p]` *(時刻tに出発する機数)*

    4.  **総機材数制約:** 全ての時刻 `t` において、飛行中と地上待機中の機材の合計が保有数を超えない。
        全ての機材タイプ `p` ∈ `P`, 全ての時刻 `t` について、
        `sum(a ∈ A) Ground[p, a, t] + sum(f ∈ F | DepartureTime(f) <= t < ArrivalTime(f)) x[f, p] <= TotalAircraft[p]`

    5.  **空港スロット制約:** 各空港 `a`、各時間帯 `t_slot` において、離着陸する便数の合計が上限以下。
        `sum(p ∈ P) sum(f ∈ F | DepartureAirport(f)=a, DepartureTime(f) in t_slot) x[f, p]` *(離陸便数)*
        `+ sum(p ∈ P) sum(f ∈ F | ArrivalAirport(f)=a, ArrivalTime(f) in t_slot) x[f, p]` *(着陸便数)*
        `<= SlotLimit[a, t_slot]`

**5. 期待されるアウトプット (ソルバーによる結果例)**

このモデルを最適化ソルバーで解くと、以下の情報が得られます。

*   **最適フリートアサインメント:** 各フライト `f` にどの機材タイプ `p` を割り当てるかの決定 (`x[f, p] = 1` となるもの)。
    *   例: 需要の多い幹線（HND-FUK午前便）には大型機(Type B)を、需要の少ない地方路線や時間帯には中型機(Type A)を割り当てる。
*   **基本的な運航スケジュール:** 上記のアサインメントに基づいた具体的な運航便リスト。
*   **機材の運用計画 (フロー):** 各機材がどのフライトを担当し、どの空港で待機するかの流れがわかる (`Ground` 変数の値や `x` 変数の繋がりから追跡可能)。
*   **最小化された総運航コスト:** 最適化された計画における1日の変動費。
*   **ボトルネック情報:** どの空港のどの時間帯のスロットが逼迫しているか、どの機材タイプが不足しがちかなどの情報が得られる（制約の双対変数などから分析可能）。-->

**4. ケーススタディの意義と実務への応用** 

このケーススタディは、数理最適化が航空会社の運行計画において以下の価値を提供することを示しています。

*   **大幅なコスト削減:** 燃料効率の良い機材を適切に割り当て、不要な大型機の投入を避けることで、運航コスト（特に燃料費）を大幅に削減できる。
*   **収益機会の最大化:** 需要予測に基づいて適切な座席数を供給することで、逸失利益を減らし、収益を最大化する。
*   **機材稼働率の向上:** 効率的な機材繰りを計画することで、機材の遊休時間を減らし、投資対効果を高める。
*   **複雑な制約下での意思決定:** 空港スロット、機材数、需要など、多数の制約が絡み合う中で、実行可能で最適な計画を迅速に作成できる。
*   **運航の安定性向上:** 事前に矛盾のない計画を立てることで、当日のイレギュラー発生リスクを低減する（ただし、実際の不規則運航への対応はさらに別の最適化問題となる）。
*   **戦略的意思決定支援:** 新しい機材の導入や路線の開設/廃止が、全体のコストや効率にどう影響するかをシミュレーションし、経営判断を支援する。

航空会社のフリートアサインメントとスケジューリングは、オペレーションズ・リサーチの古典的かつ重要な応用分野であり、数理最適化技術によって日々莫大な経済的価値が生み出されています。


#| hide

## 航空会社の機材割り当て最適化問題

### 1. 問題の定義

国内線を運航する航空会社が、翌日の運航計画において、総運航コストを最小化するための最適な機材割り当てを決定する問題です。この計画では、各フライトの旅客需要を満たし、保有する機材の数や空港の離着陸能力（スロット）といった物理的制約、さらには機材が空港間を連続して運航するためのフロー（機材繰り）の整合性を保つ必要があります。

この問題は、どのフライトにどの機材を割り当てるかという組み合わせ最適化問題であり、特に混合整数計画問題（MIP）として定式化することができます。

### 2. 定式化

#### 集合とインデックス

* $A$: 空港の集合 ($a \in A$)
* $P$: 航空機タイプの集合 ($p \in P$)
* $F$: 運航候補フライトの集合 ($f \in F$)
* $T$: 1日の運用を離散化した時間帯の集合 ($t \in T$)

#### パラメータ

* $Seat_p$: 機材タイプ $p$ の座席数
* $NumPlanes_p$: 機材タイプ $p$ の保有数
* $HourlyCost_p$: 機材タイプ $p$ の時間当たり運航コスト
* $Demand_f$: フライト $f$ の予測需要
* $Duration_f$: フライト $f$ の飛行時間
* $DepAP_f, ArrAP_f$: フライト $f$ の出発空港と到着空港
* $DepTime_f, ArrTime_f$: フライト $f$ の出発時刻と到着時刻
* $InitPlanes_{ap}$: 空港 $a$ における機材タイプ $p$ の初期配置数
* $SlotLimit_{at}$: 空港 $a$、時間帯 $t$ での離着陸スロット上限

#### 計算パラメータ

* $FlightCost_{fp} = HourlyCost_p \times Duration_f$: フライト $f$ を機材 $p$ で運航するコスト

#### 決定変数

* $Assign_{fp} \in \{0, 1\}$: フライト $f$ に機材タイプ $p$ を割り当てる場合に1、そうでない場合に0をとるバイナリ変数。

#### 目的関数

総運航コストを最小化します。

$$\text{minimize} \quad \sum_{f \in F} \sum_{p \in P} FlightCost_{fp} \cdot Assign_{fp}$$

#### 制約条件

1.  **一便一機材:** 各フライトには、必ずいずれか1つの機材タイプが1機だけ割り当てられます。


    $$\sum_{p \in P} Assign_{fp} = 1, \quad \forall f \in F$$

3.  **需要充足:** 割り当てられた機材の座席数は、フライトの予測需要以上でなければなりません。


     $$\sum_{p \in P} Seat_p \cdot Assign_{fp} \ge Demand_f, \quad \forall f \in F$$

4.  **機材数上限:** 各時刻において、飛行中の各機材タイプの総数は、そのタイプの保有総数を超えてはなりません。


     $$\sum_{f \in F: DepTime_f \le t \text{ and } ArrTime_f > t} Assign_{fp} \le NumPlanes_p, \quad \forall p \in P, \forall t \in T$$

5.  **機材フロー保存:** 各空港、各時刻において、その時刻までに到着した機材の累積数（初期配置を含む）は、その時刻までに出発した機材の累積数以上でなければなりません。これにより、存在しない機材が出発することを防ぎます。


     $$\sum_{f \in F: ArrTime_f \le t, ArrAP_f = a} Assign_{fp} + InitPlanes_{ap} \ge \sum_{f \in F: DepTime_f \le t, DepAP_f = a} Assign_{fp}, \quad \forall a \in A, \forall p \in P, \forall t \in T$$

6.  **空港スロット:** 各空港、各時間帯において、離陸する便と着陸する便の合計数は、空港の処理能力（スロット）の上限を超えてはなりません。

$$\sum_{f \in F: \lfloor DepTime_f \rfloor = t, DepAP_f = a} \sum_{p \in P} Assign_{fp} + \sum_{f \in F: \lfloor ArrTime_f \rfloor = t, ArrAP_f = a} \sum_{p \in P} Assign_{fp} \le SlotLimit_{at}, \quad \forall a \in A, \forall t \in T$$



In [3]:
#| hide
%%ampl_eval

# -----------------------------------------------
# モデル (fleet_assignment.mod)
# -----------------------------------------------
reset; model;

# --- 集合 ---
set AIRPORTS;      # 空港
set PLANE_TYPES;   # 航空機タイプ
set FLIGHTS;       # フライト候補
param T_end integer > 0;
set TIMES = 0..T_end; # 時間帯 (0時台から23時台)

# --- パラメータ ---
# 機材スペック
param seats{PLANE_TYPES} > 0;             # 座席数
param hourly_cost{PLANE_TYPES} > 0;       # 時間当たり運航コスト
param num_planes{PLANE_TYPES} >= 0;       # 保有 機材数

# フライト情報
param demand{FLIGHTS} >= 0;                       # 予測需要
param flight_duration{FLIGHTS} > 0;               # 飛行時間
param departure_airport{FLIGHTS} symbolic in AIRPORTS; # 出発空港
param arrival_airport{FLIGHTS} symbolic in AIRPORTS;   # 到着空港
param departure_time{FLIGHTS} >= 0;               # 出発時刻
param arrival_time{FLIGHTS} >= 0;                 # 到着時刻

# 制約関連パラメータ
param initial_planes{AIRPORTS, PLANE_TYPES} >= 0 default 0; # 初期配置
param slot_limit{AIRPORTS, TIMES} >= 0 default 999;       # 空港スロット上限 (既定値は事実上無限)

# 計算パラメータ
param flight_cost{f in FLIGHTS, p in PLANE_TYPES} = hourly_cost[p] * flight_duration[f]; # 各フライトのコスト

# --- 変数 ---
var Assign{FLIGHTS, PLANE_TYPES} binary; # フライトfに機材pを割り当てるか (0 or 1)

# --- 目的関数 ---
# 総運航コストの最小化
minimize TotalCost:
    sum{f in FLIGHTS, p in PLANE_TYPES} flight_cost[f, p] * Assign[f, p];

# --- 制約条件 ---
# 1. 各フライトには1つの機材タイプのみを割り当てる
subject to OnePlanePerFlight{f in FLIGHTS}:
    sum{p in PLANE_TYPES} Assign[f, p] = 1;

# 2. 割り当てられた機材の座席数は需要を満たす
subject to MeetDemand{f in FLIGHTS}:
    sum{p in PLANE_TYPES} seats[p] * Assign[f, p] >= demand[f];

# 3. 各時刻において、運航中の機材数は保有数を超えない
subject to FleetSize{p in PLANE_TYPES, t in TIMES}:
    sum{f in FLIGHTS: departure_time[f] <= t and arrival_time[f] > t} Assign[f, p] <= num_planes[p];

# 4. 機材のフロー保存（機材繰りの整合性）
subject to PlaneFlow{a in AIRPORTS, p in PLANE_TYPES, t in TIMES}:
    sum{f in FLIGHTS: arrival_time[f] <= t and arrival_airport[f] = a} Assign[f,p]
    + initial_planes[a,p]
    >=
    sum{f in FLIGHTS: departure_time[f] <= t and departure_airport[f] = a} Assign[f,p];

# 5. 空港の離着陸スロット制約
subject to AirportSlot{a in AIRPORTS, t in TIMES}:
    sum{f in FLIGHTS: floor(departure_time[f]) = t and departure_airport[f] = a} sum{p in PLANE_TYPES} Assign[f, p]
    +
    sum{f in FLIGHTS: floor(arrival_time[f]) = t and arrival_airport[f] = a} sum{p in PLANE_TYPES} Assign[f, p]
    <= slot_limit[a,t];

# -----------------------------------------------
# データ (fleet_assignment.dat)
# -----------------------------------------------
data;

set AIRPORTS := HND ITM CTS FUK;
set PLANE_TYPES := TypeA TypeB;

param T_end := 23;

# 機材スペック
param seats :=
    TypeA 150
    TypeB 250;

param hourly_cost :=
    TypeA 50
    TypeB 80;

param num_planes :=
    TypeA 10
    TypeB 5;

# フライト候補
set FLIGHTS :=
    F1 F2 F3 F4 F5 F6 F7 F8;

# フライトごとのパラメータ
param: departure_airport arrival_airport flight_duration departure_time arrival_time demand :=
    F1  HND ITM 1.0  8  9 130
    F2  ITM HND 1.0 10 11 140
    F3  HND FUK 2.0  9 11 220
    F4  FUK HND 2.0 12 14 240
    F5  HND CTS 1.5 10 11.5 120
    F6  CTS HND 1.5 13 14.5 160
    F7  ITM FUK 1.2 12 13.2 100
    F8  FUK ITM 1.2 15 16.2 180;

# 初期配置 (疎なデータ)
param initial_planes :=
    HND TypeA 8
    HND TypeB 4
    ITM TypeA 2
    ITM TypeB 1;

# 空港スロット制約 (特定の時間帯のみ)
param slot_limit :=
    HND 8 2
    HND 9 1
    HND 10 2
    ITM 10 1
    ITM 11 1
    FUK 11 1
    FUK 12 1;

# -----------------------------------------------
# 実行と結果表示
# -----------------------------------------------
option solver highs;
solve;

display Assign;

HiGHS 1.10.0: optimal solution; objective 852
0 simplex iterations
0 branching nodes
Assign :=
F1 TypeA   1
F1 TypeB   0
F2 TypeA   1
F2 TypeB   0
F3 TypeA   0
F3 TypeB   1
F4 TypeA   0
F4 TypeB   1
F5 TypeA   0
F5 TypeB   1
F6 TypeA   0
F6 TypeB   1
F7 TypeA   0
F7 TypeB   1
F8 TypeA   0
F8 TypeB   1
;



## 11. 小規模航空路線網における乗務ペアリング最適化

**1. 背景と課題**

地域航空会社「コネクトエア」は、限られた機材と乗務員で効率的な運航を目指しています。翌日に運航予定の主要フライトレグ（区間飛行）が6便確定しました。これらをカバーするために、事前にいくつかの実行可能な「乗務ペアリング」（1人の乗務員が担当する一連のフライトで、拠点空港から出発し拠点空港に戻るもの）候補が作成されています。

**課題:**

*   全てのフライトレグを、いずれかの乗務ペアリングによって担当させる必要がある。
*   各フライトレグは、重複なくちょうど1つのペアリングによってカバーされなければならない。
*   各ペアリングには、移動費や宿泊費などを反映した推定コストがかかる。
*   これらの条件を満たし、かつ**総ペアリングコストを最小化**するペアリングの組み合わせを選定したい。

**2. 目標**

*   確定した6便のフライトレグ全てをカバーする。
*   各フライトレグがちょうど1つのペアリングに含まれるようにする。
*   総ペアリングコストを最小化するペアリングの組み合わせを選択する。

**3. データ**

*   **対象フライトレグ (L):** {L1, L2, L3, L4, L5, L6}
    *   L1: HND (羽田) → ITM (伊丹)
    *   L2: ITM (伊丹) → FUK (福岡)
    *   L3: FUK (福岡) → HND (羽田)
    *   L4: HND (羽田) → CTS (千歳)
    *   L5: CTS (千歳) → ITM (伊丹)
    *   L6: ITM (伊丹) → HND (羽田)
    *(簡単のため、出発/到着時刻や具体的な機材はここでは考慮しない)*

*   **生成されたペアリング候補 (P) とコスト:** (コストは仮の単位)

    | ペアリングID | カバーするフライトレグ       | コスト (Cost[p]) |
    | :----------- | :--------------------------- | :--------------- |
    | P1           | {L1, L2, L3}                 | 15               |
    | P2           | {L4, L5, L6}                 | 12               |
    | P3           | {L1, L6}                     | 5                |
    | P4           | {L4, L5}                     | 9                |
    | P5           | {L2, L3}                     | 8                |
    | P6           | {L1, L2, L5, L6}             | 20               |
    | P7           | {L3, L4}                     | 10               |

*   **カバー情報 (Covers[p, l]):** (ペアリングpがレグlを含むなら1)

    |       | L1 | L2 | L3 | L4 | L5 | L6 |
    | :---- | :-:| :-:| :-:| :-:| :-:| :-:|
    | **P1**  | 1  | 1  | 1  | 0  | 0  | 0  |
    | **P2**  | 0  | 0  | 0  | 1  | 1  | 1  |
    | **P3**  | 1  | 0  | 0  | 0  | 0  | 1  |
    | **P4**  | 0  | 0  | 0  | 1  | 1  | 0  |
    | **P5**  | 0  | 1  | 1  | 0  | 0  | 0  |
    | **P6**  | 1  | 1  | 0  | 0  | 1  | 1  |
    | **P7**  | 0  | 0  | 1  | 1  | 0  | 0  |

<!-- **4. 数理最適化モデル (集合分割モデル)**

*   **決定変数:**
    *   `x[p]` : ペアリング `p` ∈ {P1, P2, ..., P7} を採用する場合に 1、採用しない場合に 0 をとるバイナリ変数。

*   **目的関数 (総ペアリングコスト最小化):**
    `Minimize:`
    `15*x[P1] + 12*x[P2] + 5*x[P3] + 9*x[P4] + 8*x[P5] + 20*x[P6] + 10*x[P7]`

*   **制約条件 (各フライトレグをちょうど1回カバー):**
    *   **L1:** `x[P1] + x[P3] + x[P6] = 1`
    *   **L2:** `x[P1] + x[P5] + x[P6] = 1`
    *   **L3:** `x[P1] + x[P5] + x[P7] = 1`
    *   **L4:** `x[P2] + x[P4] + x[P7] = 1`
    *   **L5:** `x[P2] + x[P4] + x[P6] = 1`
    *   **L6:** `x[P2] + x[P3] + x[P6] = 1`
    *   `x[p]` は 0 または 1

**5. 期待されるアウトプット (ソルバーによる結果)**

このモデルを整数計画ソルバーで解くと、以下の解が得られます。

*   **採用されるペアリング:**
    *   `x[P3] = 1`
    *   `x[P4] = 1`
    *   `x[P5] = 1`
    *   (`x[P1], x[P2], x[P6], x[P7]` はすべて 0)
*   **選択されたペアリング:** P3, P4, P5
*   **最小総コスト:** 5 (P3) + 9 (P4) + 8 (P5) = **22**

**解の検証:**
選択されたペアリング P3, P4, P5 で全てのフライトレグがちょうど1回カバーされているか確認します。
*   L1: P3 がカバー (OK)
*   L2: P5 がカバー (OK)
*   L3: P5 がカバー (OK)
*   L4: P4 がカバー (OK)
*   L5: P4 がカバー (OK)
*   L6: P3 がカバー (OK)
全てのレグが重複なくカバーされています。

**比較:** もし別の組み合わせ、例えば P1 と P2 を選択した場合、全てのレグはカバーされますが、総コストは 15 + 12 = 27 となり、P3+P4+P5 の組み合わせ (コスト22) よりも高くなります。最適化ソルバーは、可能な全ての組み合わせを効率的に探索し、最小コストの解を見つけ出します。 -->

**4. ケーススタディの意義と実務への応用**

この簡略化されたケーススタディでも、数理最適化が乗務員スケジューリングにおいて以下の価値を提供することがわかります。

*   **コスト効率の追求:** 複数の選択肢（ペアリング）の中から、コストを最小化する最適な組み合わせを客観的に見つけ出すことができます。この例では、コスト27の案よりコスト22の案が優れていることを示しました。
*   **制約遵守の保証:** 全てのフライトレグを確実にカバーするという基本的な制約を厳密に満たすスケジュールを作成できます。現実の問題では、これに加えて勤務時間や休息時間などの複雑な制約も組み込まれます。
*   **計画の自動化と迅速化:** 複雑な組み合わせ問題を自動で解くことにより、計画作成にかかる時間と労力を大幅に削減できます。
*   **透明性と客観性:** なぜそのペアリングの組み合わせが選ばれたのかが、コストと制約に基づいて明確に示されます。



In [14]:
#| hide

%%ampl_eval

# -----------------------------------------------
# モデル (crew_rostering.mod)
# -----------------------------------------------
reset; model;

# --- 集合 ---
set CREW;       # 乗務員
set PAIRINGS;   # ペアリング
set SCHEDULES;  # 全てのスケジュール候補

# --- パラメータ ---
# ペアリングの必要人数
param RequiredCrew{PAIRINGS} >= 0;

# 各スケジュールの費用
param Cost{SCHEDULES} >= 0;

# 各スケジュールがどの乗務員のものか
param ScheduleOwner{SCHEDULES} symbolic in CREW;

# スケジュールがペアリングを含むか (疎なデータなのでdefault 0を指定)
param ScheduleIncludesPairing{SCHEDULES, PAIRINGS} binary default 0;


# --- 変数 ---
# Select[s] = 1 ならば、スケジュールsを採用する
var Select{SCHEDULES} binary;


# --- 目的関数 ---
# 総費用の最小化
minimize TotalCost:
    sum{s in SCHEDULES} Cost[s] * Select[s];


# --- 制約条件 ---
# 1. 各ペアリングを、ちょうど必要人数でカバーする
subject to CoverPairing{p in PAIRINGS}:
    sum{s in SCHEDULES} ScheduleIncludesPairing[s, p] * Select[s] = RequiredCrew[p];

# 2. 各乗務員は最大で1つのスケジュールしか担当しない
subject to AssignOneSchedulePerCrew{c in CREW}:
    sum{s in SCHEDULES: ScheduleOwner[s] = c} Select[s] <= 1;


# -----------------------------------------------
# データ (crew_rostering.dat)
# -----------------------------------------------
data;

set CREW := PilotA PilotB PilotC;
set PAIRINGS := P1 P2 P3 P4;
set SCHEDULES := S_A1 S_A2 S_B1 S_B2 S_C1 S_C2;

# ペアリングの必要人数
param RequiredCrew :=
    P1 2
    P2 1
    P3 1
    P4 2;

# スケジュールと費用のデータ
param: ScheduleOwner  Cost :=
    S_A1   PilotA     100
    S_A2   PilotA     120
    S_B1   PilotB     110
    S_B2   PilotB      70
    S_C1   PilotC     150
    S_C2   PilotC      90;

# スケジュールがどのペアリングを含むか (値が1のものだけを記述)
param ScheduleIncludesPairing :=
    S_A1 P1 1    S_A1 P3 1
    S_A2 P2 1    S_A2 P4 1
    S_B1 P1 1    S_B1 P4 1
    S_B2 P2 1
    S_C1 P1 1    S_C1 P2 1    S_C1 P3 1
    S_C2 P4 1;

# -----------------------------------------------
# 実行と結果表示
# -----------------------------------------------
option solver highs;
solve;

printf "\n--- 最適なスケジュール割り当て ---\n";
printf "総費用: %d\n\n", TotalCost;
printf "%-12s -> %s\n", "乗務員", "担当スケジュールID";
printf "----------------------------------\n";
for {c in CREW} {
    printf "%-12s -> ", c;
    for {s in SCHEDULES: ScheduleOwner[s] = c and Select[s] > 0.5} {
        printf "%s (費用: %d)\n", s, Cost[s];
    }
}

HiGHS 1.10.0: infeasible problem
0 simplex iterations
0 branching nodes

--- 最適なスケジュール割り当て ---
総費用: 340

乗務員    -> 担当スケジュールID
----------------------------------
PilotA       -> PilotB       -> S_B1 (費用: 110)
PilotC       -> 

## 12. 大学学部の授業教室・時間割割り当て最適化

**1. 背景と課題**

ある大学の〇〇学部では、来学期の授業スケジュールと教室割り当ての計画を進めています。学部には複数の授業があり、それぞれ担当教員、予想される受講者数、必要な設備（プロジェクターなど）が異なります。一方、利用可能な教室数には限りがあり、各教室の収容人数や設備も様々です。

**課題:**

*   **教室不足とミスマッチ:** 特定の時間帯に授業が集中し、必要な大きさや設備の教室が不足することがある。逆に、少人数の授業に大きな教室が割り当てられ、非効率になる場合もある。
*   **時間割の衝突:** 同じ教員が同時刻に複数の授業を担当したり、学生が必修科目を同時に履修できなくなったりするような時間割の衝突を避けなければならない。
*   **制約の複雑さ:** 教員の担当可能時間、教室の設備要件、学生の履修パターン（特定の科目は連続で受けたい等、今回は簡略化）など、多くの条件を考慮する必要がある。
*   **手作業の限界:** これらの複雑な条件を考慮しながら、手作業で最適な（あるいは実行可能な）時間割と教室割り当てを作成するのは非常に困難で時間がかかる。

そこで、数理最適化を用いて、全ての制約を満たし、かつ効率的な**授業の時間割と教室割り当て**を自動生成することを目指します。

**2. 目標**

*   全ての授業を、指定された時間割の枠（例: 月曜～金曜の1限～5限）と利用可能な教室に割り当てる。
*   以下の**必須制約**を全て満たす：
    *   同じ教室・同じ時間帯に複数の授業が入らない（教室衝突回避）。
    *   同じ教員が同じ時間帯に複数の授業を担当しない（教員衝突回避）。
    *   授業の受講者数が、割り当てられた教室の収容人数を超えない（収容人数制約）。
    *   授業に必要な設備が、割り当てられた教室に備わっている（設備制約）。
    *   教員の担当可能時間外には授業を割り当てない（教員可用性）。
*   上記を満たした上で、**総割り当てコスト（ペナルティ）を最小化**する。（例：希望時間帯以外への割り当て、必要最低限より大きい教室への割り当て、などにペナルティを設定）
    *(今回はシンプルに、必須制約を満たす実行可能な解を見つけることを主目的とする)*

**3. 設定とデータ**

*   **時間割枠 (T):** 月曜～金曜の各1限～3限 (簡単のため 5曜日 * 3時限 = 15スロット)
    *   t1: 月1限, t2: 月2限, ..., t15: 金3限
*   **授業リスト (J):**

    | 授業ID | 科目名         | 教員ID | 受講者数 | 要プロジェクター | 備考/制約                      |
    | :----- | :------------- | :----- | :------- | :------------- | :----------------------------- |
    | C1     | 最適化入門     | ProfA  | 40       | Yes            |                              |
    | C2     | 線形代数       | ProfB  | 60       | Yes            |                              |
    | C3     | プログラミング基礎 | ProfC  | 50       | Yes            |                              |
    | C4     | データ構造     | ProfC  | 45       | Yes            |                              |
    | C5     | 統計学         | ProfB  | 55       | No             |                              |
    | C6     | 経済学原論     | ProfA  | 70       | Yes            | 月曜午前(t1, t2)は担当不可 |

*   **教室リスト (R):**

    | 教室ID | 収容人数 | プロジェクター |
    | :----- | :------- | :------------- |
    | R101   | 50       | Yes            |
    | R102   | 75       | Yes            |
    | R201   | 60       | No             |
    | R202   | 80       | Yes            |

*   **教員可用性:**
    *   ProfA: 月曜午前(t1, t2) は担当不可。
    *   その他は全時間帯で担当可能と仮定。

<!-- **4. 数理最適化モデル**

*   **決定変数:**
    *   `x[j, r, t]` : 授業 `j` ∈ `J` を、教室 `r` ∈ `R` で、時間 `t` ∈ `T` に割り当てる場合に 1、そうでない場合に 0 をとるバイナリ変数。

*   **目的関数:**
    全ての制約を満たす実行可能な解を見つけることを主目的とするため、ダミーの目的関数を設定。
    `Maximize 0`
    *(より高度なモデルでは、部屋サイズと人数の差の最小化、希望時間への割り当て最大化などを目的関数にできる)*

*   **制約条件:**
    1.  **各授業はちょうど1回割り当てられる:**
        全ての授業 `j` ∈ `J` について、
        `sum(r ∈ R) sum(t ∈ T) x[j, r, t] = 1`

    2.  **教室衝突回避:** 同じ教室・同じ時間に複数の授業を割り当てない。
        全ての教室 `r` ∈ `R`, 全ての時間 `t` ∈ `T` について、
        `sum(j ∈ J) x[j, r, t] <= 1`

    3.  **教員衝突回避:** 同じ教員が同じ時間に複数の授業を担当しない。
        全ての教員 `p` (ProfA, B, C), 全ての時間 `t` ∈ `T` について、
        `sum(j ∈ J | Teacher(j)=p) sum(r ∈ R) x[j, r, t] <= 1`
        *(Teacher(j) は授業jの担当教員を示す)*

    4.  **収容人数制約:** 授業の受講者数が、割り当てられた教室の収容人数を超えない。
        全ての授業 `j` ∈ `J`, 全ての教室 `r` ∈ `R`, 全ての時間 `t` ∈ `T` について、
        `Enrollment(j) * x[j, r, t] <= Capacity(r) * x[j, r, t]`
        これは自明なので、より実質的には以下のように書ける:
        もし `Enrollment(j) > Capacity(r)` ならば、`x[j, r, t] = 0` とする。（あるいは、制約として）
        `sum(t ∈ T) x[j, r, t] = 0` (if `Enrollment(j) > Capacity(r)`)

    5.  **設備制約:** プロジェクターが必要な授業は、プロジェクターがある教室にしか割り当てられない。
        全ての授業 `j` ∈ `J` (ただし `NeedProj(j)=Yes`), 全ての教室 `r` ∈ `R` (ただし `HasProj(r)=No`), 全ての時間 `t` ∈ `T` について、
        `x[j, r, t] = 0`

    6.  **教員可用性制約:**
        `sum(r ∈ R) x[C6, r, t1] = 0` (ProfAは月1限(t1)不可)
        `sum(r ∈ R) x[C6, r, t2] = 0` (ProfAは月2限(t2)不可)

**5. 期待されるアウトプット (ソルバーによる結果例)**

このモデルを整数計画ソルバーで解くと、制約を満たす `x[j, r, t] = 1` となる割り当てが出力されます。これは具体的な時間割表となります。

**出力例 (部分的な時間割):**

| 時間 | 教室 R101 (50/Y) | 教室 R102 (75/Y) | 教室 R201 (60/N) | 教室 R202 (80/Y) |
| :--- | :--------------- | :--------------- | :--------------- | :--------------- |
| 月1 (t1)| C3 (ProfC, 50)   |                  | C5 (ProfB, 55)   | C2 (ProfB, 60)   | ←教員衝突！(ProfB)
| 月2 (t2)| C4 (ProfC, 45)   | C1 (ProfA, 40)   |                  | C6 (ProfA, 70)   | ←教員衝突！(ProfA), C6はProfA不可
| 月3 (t3)|                  | C6 (ProfA, 70)   |                  | C4 (ProfC, 45)   | ←教員衝突！(ProfC)
| ...  | ...              | ...              | ...              | ...              |

**(※上記は制約違反を含む悪い例。ソルバーは衝突のない割り当てを見つけ出すか、実行不可能な場合はその旨を報告する)**

**正しい出力例 (一部):**

| 時間 | 教室 R101 (50/Y) | 教室 R102 (75/Y) | 教室 R201 (60/N) | 教室 R202 (80/Y) |
| :--- | :--------------- | :--------------- | :--------------- | :--------------- |
| 月1 (t1)| C3 (ProfC, 50)   | C2 (ProfB, 60)   |                  |                  |
| 月2 (t2)| C4 (ProfC, 45)   |                  | C5 (ProfB, 55)   |                  |
| 月3 (t3)| C1 (ProfA, 40)   |                  |                  | C6 (ProfA, 70)   |
| ...  | ...              | ...              | ...              | ...              | -->

**4. ケーススタディの意義と実務への応用**

このケーススタディは、数理最適化が教室割り当てとスケジューリングにおいて以下の価値を提供することを示しています。

*   **衝突のないスケジュールの保証:** 複雑な条件（教室、教員、設備、収容人数）を考慮し、ルール違反のない実行可能な時間割を確実に作成できます。
*   **リソース利用の効率化:** 教室の収容人数や設備を考慮して授業を割り当てることで、教室の無駄遣いを減らし、利用率を向上させることができます（目的関数に組み込めばより明確に）。
*   **計画作成の自動化と迅速化:** 面倒で時間のかかる割り当て作業を自動化し、担当者の負担を大幅に軽減します。急な変更（教員の都合、教室の変更など）にも迅速に対応できます。
*   **公平性と透明性:** 特定の教員への負担集中などを避ける制約を追加したり、割り当て結果の根拠を明確に示したりすることで、公平性を高め、関係者への説明責任を果たしやすくなります。
*   **What-if 分析:** 新しい授業の追加、教室の改修、履修人数の変動などが、時間割全体にどのような影響を与えるかを事前にシミュレーションできます。

大学や学校における時間割・教室割り当ては、多くの関係者の要求と物理的な制約が絡み合う典型的な組合せ最適化問題であり、数理最適化の導入効果が高い分野の一つです。


#| hide

## 大学の授業スケジューリングと教室割り当ての最適化

提示された課題は、多くの制約条件を考慮しながら、授業を時間割と教室に効率的に割り当てる最適化問題です。ここでは、数理最適化モデリング言語AMPLを用いてこの問題を解決するアプローチを示します。

### 1. 問題の定義と定式化

この問題は、どの授業を、どの時間枠に、どの教室で実施するかを決定する「割り当て問題」として定式化できます。目標は、物理的、人的、設備的な制約をすべて満たす実行可能な時間割を作成することです。

#### 集合とインデックス
* $J$: 授業の集合 ($j \in J$)
* $T$: 時間割枠（タイムスロット）の集合 ($t \in T$)
* $R$: 教室の集合 ($r \in R$)
* $P$: 教員の集合 ($p \in P$)
* $U \subseteq P \times T$: 教員が担当**不可能**なスロットの集合 ($(p, t) \in U$)

#### パラメータ
* $Teacher_j$: 授業 $j$ の担当教員
* $Enrollment_j$: 授業 $j$ の予想受講者数
* $ReqProj_j$: 授業 $j$ がプロジェクターを必要とするか (Yes=1, No=0)
* $Capacity_r$: 教室 $r$ の収容人数
* $HasProj_r$: 教室 $r$ にプロジェクターがあるか (Yes=1, No=0)

#### 決定変数
* $Assign_{j,t,r}$: 授業 $j$ を時間 $t$ に教室 $r$ で実施する場合に 1、そうでない場合に 0 をとるバイナリ変数。

#### 目的関数
今回はすべての必須制約を満たす**実行可能な解**を見つけることが主目的であるため、目的関数はダミーとして定数0の最小化を設定します。
$$\text{minimize} \quad \text{DummyObjective}: 0$$

#### 制約条件
1.  **授業割り当て制約:** 全ての授業は、ただ1つの時間と教室の組み合わせに割り当てられなければなりません。
    $$\sum_{t \in T} \sum_{r \in R} Assign_{j,t,r} = 1 \quad (\forall j \in J)$$
2.  **教室の衝突回避:** 各時間、各教室には、最大で1つの授業しか割り当てられません。
    $$\sum_{j \in J} Assign_{j,t,r} \le 1 \quad (\forall t \in T, \forall r \in R)$$
3.  **教員の衝突回避:** 各時間、各教員は、最大で1つの授業しか担当できません。
    $$\sum_{j \in J : Teacher_j=p} \sum_{r \in R} Assign_{j,t,r} \le 1 \quad (\forall p \in P, \forall t \in T)$$
4.  **収容人数・設備・教員可用性の制約:** 割り当ては、収容人数、設備、教員の担当可能時間の条件を満たす組み合わせでのみ可能です。これは、変数を定義するインデックス集合を、予めこれらの条件を満たすものに限定することで効率的に表現します。
    $$Assign_{j,t,r} = 0 \quad \text{if} \begin{cases} Enrollment_j > Capacity_r \\ \text{or} \quad (ReqProj_j = 1 \text{ and } HasProj_r = 0) \\ \text{or} \quad (Teacher_j, t) \in U \end{cases}$$

### 2. AMPLモデルとデータ

上記の定式化に基づき、AMPLのモデルとデータを作成します。ハードな制約（収容人数、設備、教員可用性）を満たさない割り当ては、実行不可能な組み合わせとして予め除外するために `POSSIBLE_ASSIGNMENTS` という補助的な集合を定義します。これにより、モデルがより効率的になります。


In [4]:
#| hide
%%ampl_eval

# ----------------------------------------------------
# モデルファイル (timetable.mod)
# ----------------------------------------------------
reset; model;

# --- 集合 ---
set COURSES;    # 授業
set TIMES;      # 時間割枠
set ROOMS;      # 教室
set TEACHERS;   # 教員

# --- パラメータ ---
param Teacher{COURSES} symbolic in TEACHERS;   # 担当教員
param Enrollment{COURSES} >= 0;                # 受講者数
param Requires_Projector{COURSES} binary;      # プロジェクター要否 (1=Yes, 0=No)

param Capacity{ROOMS} >= 0;                    # 収容人数
param Has_Projector{ROOMS} binary;             # プロジェクター有無 (1=Yes, 0=No)

# 教員の担当不可能なスロット
set UNAVAILABLE_SLOTS within TEACHERS cross TIMES;

# --- 実行可能な割り当ての集合を事前に定義 ---
# ハードな制約（収容人数、設備、教員可用性）を満たす組み合わせのみを変数として考慮
set POSSIBLE_ASSIGNMENTS = {
    j in COURSES, t in TIMES, r in ROOMS:
        Enrollment[j] <= Capacity[r] and 
        (Requires_Projector[j] = 0 or Has_Projector[r] = 1) and
        (Teacher[j], t) not in UNAVAILABLE_SLOTS
};

# --- 変数 ---
var Assign{POSSIBLE_ASSIGNMENTS} binary;

# --- 目的関数 ---
# 今回は制約を満たす解を見つけるのが目的なので、ダミーの目的関数を設定
minimize DummyObjective: 0;

# --- 制約条件 ---
# 1. 全ての授業は、ただ1つの時間・教室に割り当てる
subject to CourseScheduleOnce{j in COURSES}:
    sum{(j, t, r) in POSSIBLE_ASSIGNMENTS} Assign[j, t, r] = 1;

# 2. 各時間・各教室には最大1つの授業
subject to RoomConflict{t in TIMES, r in ROOMS}:
    sum{j in COURSES: (j, t, r) in POSSIBLE_ASSIGNMENTS} Assign[j, t, r] <= 1;

# 3. 各時間・各教員は最大1つの授業
subject to TeacherConflict{p in TEACHERS, t in TIMES}:
    sum{j in COURSES, r in ROOMS: Teacher[j] = p and (j, t, r) in POSSIBLE_ASSIGNMENTS} 
    Assign[j, t, r] <= 1;

# ----------------------------------------------------
# データファイル (timetable.dat)
# ----------------------------------------------------
data;

set COURSES  := C1 C2 C3 C4 C5 C6;
set TIMES    := t1 t2 t3 t4 t5 t6 t7 t8 t9 t10 t11 t12 t13 t14 t15; # Mon1-Fri5
set ROOMS    := R101 R102 R201 R202;
set TEACHERS := ProfA ProfB ProfC;

# 授業データ
param: Teacher Enrollment Requires_Projector :=
    C1  ProfA   40      1
    C2  ProfB   60      1
    C3  ProfC   50      1
    C4  ProfC   45      1
    C5  ProfB   55      0
    C6  ProfA   70      1;

# 教室データ
param: Capacity Has_Projector :=
    R101  50      1
    R102  75      1
    R201  60      0
    R202  80      1;

# 教員の担当不可データ
set UNAVAILABLE_SLOTS :=
    ProfA t1
    ProfA t2 ;

# ----------------------------------------------------
# 実行と結果表示
# ----------------------------------------------------
option solver highs;
solve;

# 割り当て結果を見やすく表示
printf "\n--- 授業時間割と教室割り当て結果 ---\n";
printf "%-10s %-20s %-10s %-10s %-10s\n", "授業ID", "科目名", "担当教員", "時間", "教室";
printf "------------------------------------------------------------------\n";
for {j in COURSES} {
    for {(j, t, r) in POSSIBLE_ASSIGNMENTS: Assign[j,t,r] > 0.5} {
        printf "%-10s %-20s %-10s %-10s %-10s\n", j, j, Teacher[j], t, r;
    }
}

HiGHS 1.10.0: optimal solution; objective 0
6 simplex iterations
1 branching nodes

--- 授業時間割と教室割り当て結果 ---
授業ID   科目名            担当教員 時間     教室    
------------------------------------------------------------------
C1         C1                   ProfA      t13        R102      
C2         C2                   ProfB      t14        R202      
C3         C3                   ProfC      t12        R102      
C4         C4                   ProfC      t10        R101      
C5         C5                   ProfB      t9         R201      
C6         C6                   ProfA      t4         R102      


## 13. 森林公園における希少動物生息地の監視センサーネットワーク設計

**1. 背景と課題**

広大な森林公園「ミドリの森国立公園」の管理事務所は、公園内に生息する希少動物（例：特別天然記念物のニホンカモシカ）の生態調査と保護のため、主要な生息候補地や水飲み場を監視するセンサーネットワークの導入を計画しています。しかし、設置できるセンサーの数や性能、設置・運用にかかる予算は限られています。

**課題:**

*   **効果的なカバレッジ:** 限られた予算内で、できるだけ多くの重要監視ポイントをカバーしたい。特に、過去の目撃情報が多いポイントや、密猟のリスクがあるポイントは優先的に監視したい。
*   **センサー選択:** 性能（検知範囲）とコスト（設置費、通信費）が異なる複数のセンサータイプから、最適な組み合わせを選ぶ必要がある。
*   **設置場所の選定:** センサーを設置できる候補地点はいくつかあるが、地形やアクセスの制約がある。どの候補地点にどのセンサーを置くのが最も効率的か。
*   **コスト管理:** 設置費用だけでなく、データ通信にかかるランニングコストも考慮し、予算内に収める必要がある。

管理事務所は、これらの課題を解決するため、数理最適化を用いて、**予算内で最も効果的なカバレッジ（重要度を考慮）を達成するセンサーの配置計画**を策定したいと考えています。

**2. 目標**

*   与えられた総予算内で、センサーの設置場所とタイプを決定する。
*   各監視ターゲットポイントの重要度を考慮し、**総カバレッジスコア（カバーされたターゲットの重要度の合計）を最大化**する。
*   設置場所の制約（各候補地に設置できるセンサーは1つまで）を守る。

**3. 設定とデータ**

*   **センサー設置候補地点 (L):** 公園内の5箇所 {L1, L2, L3, L4, L5}
*   **監視ターゲットポイント (T):** 動物の生息・活動が予想される10箇所 {T1, T2, ..., T10}
*   **センサータイプ (S):**
    *   `SensorA` (標準型): 設置コスト 5万円, 通信コスト 1万円/年, 検知範囲 1km
    *   `SensorB` (広範囲型): 設置コスト 12万円, 通信コスト 2万円/年, 検知範囲 2km
*   **ターゲットポイントの重要度 (Importance[j]):** (1～5の5段階評価)

    | ターゲット | T1 | T2 | T3 | T4 | T5 | T6 | T7 | T8 | T9 | T10 |
    | :--------- | :-:| :-:| :-:| :-:| :-:| :-:| :-:| :-:| :-:| :--:|
    | 重要度     | 5  | 4  | 5  | 3  | 2  | 3  | 4  | 2  | 3   | 5   |

*   **設置候補地点からターゲットポイントまでの距離 (km) とカバレッジ:**
    (距離が検知範囲以下ならカバー可能 (1)、超えるなら不可 (0))

    **SensorA (範囲1km) でのカバー可否 Covers[l, SensorA, j]:**

    |      | T1 | T2 | T3 | T4 | T5 | T6 | T7 | T8 | T9 | T10 |
    | :--- | :-:| :-:| :-:| :-:| :-:| :-:| :-:| :-:| :-:| :--:|
    | L1   | 1  | 1  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0   |
    | L2   | 0  | 1  | 1  | 1  | 0  | 0  | 0  | 0  | 0  | 0   |
    | L3   | 0  | 0  | 0  | 1  | 1  | 1  | 0  | 0  | 0  | 0   |
    | L4   | 0  | 0  | 0  | 0  | 0  | 1  | 1  | 1  | 0  | 0   |
    | L5   | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 1  | 1  | 1   |

    **SensorB (範囲2km) でのカバー可否 Covers[l, SensorB, j]:**

    |      | T1 | T2 | T3 | T4 | T5 | T6 | T7 | T8 | T9 | T10 |
    | :--- | :-:| :-:| :-:| :-:| :-:| :-:| :-:| :-:| :-:| :--:|
    | L1   | 1  | 1  | 1  | 1  | 0  | 0  | 0  | 0  | 0  | 0   |
    | L2   | 1  | 1  | 1  | 1  | 1  | 1  | 0  | 0  | 0  | 0   |
    | L3   | 0  | 1  | 1  | 1  | 1  | 1  | 1  | 1  | 0  | 0   |
    | L4   | 0  | 0  | 0  | 1  | 1  | 1  | 1  | 1  | 1  | 1   |
    | L5   | 0  | 0  | 0  | 0  | 0  | 0  | 1  | 1  | 1  | 1   |

*   **総予算 (Budget):** 40万円 (設置コスト+初年度通信コストの合計)

<!-- **4. 数理最適化モデル**

*   **決定変数:**
    *   `x[l, s]` : 地点 `l` ∈ {L1..L5} にセンサータイプ `s` ∈ {SensorA, SensorB} を設置する場合に 1、しない場合に 0 (バイナリ変数)。
    *   `y[j]` : ターゲット `j` ∈ {T1..T10} が少なくとも1つのセンサーでカバーされる場合に 1、されない場合に 0 (バイナリ変数)。

*   **目的関数 (総重要度加重カバレッジ最大化):**
    `Maximize:`
    `5*y[T1] + 4*y[T2] + 5*y[T3] + 3*y[T4] + 2*y[T5] + 3*y[T6] + 4*y[T7] + 2*y[T8] + 3*y[T9] + 5*y[T10]`

*   **制約条件:**
    1.  **カバレッジ達成制約:** 各ターゲット `j` がカバーされる(`y[j]=1`)のは、それをカバー可能なセンサー(`x[l,s]=1`)が設置された場合のみ。
        (例: ターゲット T1 について)
        `x[L1, SensorA] + x[L1, SensorB] + x[L2, SensorB] >= y[T1]`
        (例: ターゲット T4 について)
        `x[L2, SensorA] + x[L2, SensorB] + x[L3, SensorA] + x[L3, SensorB] + x[L4, SensorB] >= y[T4]`
        ... (全てのターゲット T1～T10 について同様の制約を作成)

    2.  **設置場所制約:** 各地点 `l` には高々1つのセンサーしか設置できない。
        `x[L1, SensorA] + x[L1, SensorB] <= 1`
        `x[L2, SensorA] + x[L2, SensorB] <= 1`
        `x[L3, SensorA] + x[L3, SensorB] <= 1`
        `x[L4, SensorA] + x[L4, SensorB] <= 1`
        `x[L5, SensorA] + x[L5, SensorB] <= 1`

    3.  **予算制約:** 総コスト（設置＋初年度通信）は40万円以下。
        `sum(l,s) (InstallCost[s] * x[l, s]) + sum(l,s) (CommCost[s] * x[l, s]) <= 400000`
        展開すると:
        `(50000+10000)*x[L1,A] + (120000+20000)*x[L1,B] + ... + (120000+20000)*x[L5,B] <= 400000`
        `60000*x[L1,A] + 140000*x[L1,B] + 60000*x[L2,A] + 140000*x[L2,B] + ... + 140000*x[L5,B] <= 400000`

    4.  変数定義: `x[l, s]`, `y[j]` は 0 または 1 のバイナリ変数。

**5. 期待されるアウトプット (ソルバーによる結果例)**

このモデルを整数計画ソルバーで解くと、以下の情報が得られます。

*   **最適なセンサー配置計画 (`x[l, s] = 1` となるもの):**
    *   例:
        *   L1 に SensorA を設置 (`x[L1, SensorA] = 1`)
        *   L3 に SensorB を設置 (`x[L3, SensorB] = 1`)
        *   L5 に SensorA を設置 (`x[L5, SensorA] = 1`)
        *   (L2, L4 には設置しない)
*   **総コスト:**
    *   設置コスト: 5 (L1A) + 12 (L3B) + 5 (L5A) = 22万円
    *   通信コスト: 1 (L1A) + 2 (L3B) + 1 (L5A) = 4万円
    *   合計コスト: 22 + 4 = 26万円 (予算40万円以内)
*   **カバーされるターゲット (`y[j] = 1` となるもの) と最大カバレッジスコア:**
    *   L1Aがカバー: T1, T2
    *   L3Bがカバー: T2, T3, T4, T5, T6, T7, T8
    *   L5Aがカバー: T8, T9, T10
    *   カバーされるターゲット: T1, T2, T3, T4, T5, T6, T7, T8, T9, T10 (全て)
    *   最大カバレッジスコア: 5+4+5+3+2+3+4+2+3+5 = 36 (全ターゲットの重要度合計)
    *(この例では予算内で全ターゲットをカバーできたが、予算がもっと厳しい場合や、より広範囲な問題では、重要度の低いターゲットがカバーされない解が最適となる可能性がある)*

**(※上記は仮の最適解です。実際の解はソルバーで計算する必要があり、制約やパラメータによって結果は変わります。例えば、広範囲型センサーBのコストがもっと高ければ、標準型Aを複数設置する解が最適になるかもしれません。)** -->

**4. ケーススタディの意義と実務への応用**

このケーススタディは、数理最適化がセンサーネットワーク設計において以下の価値を提供することを示しています。

*   **コスト効率の最大化:** 限られた予算内で、カバレッジや性能といった目標を最大化する最適なセンサーの種類と配置を決定できる。
*   **客観的な意思決定:** どのターゲットを優先的にカバーするか（重要度）、どのセンサータイプがコスト対効果で優れているかなどをデータに基づいて判断できる。
*   **複雑なトレードオフの考慮:** センサー性能（範囲）、コスト（設置、通信）、カバレッジ要件、予算などの間の複雑なトレードオフを考慮した上で、全体として最適な解を見つけ出す。
*   **計画の柔軟性:** 予算の増減、新しいセンサー技術の登場、監視対象エリアの変更などがあった場合に、モデルを更新して迅速に再計画できる。
*   **多様な分野への応用:** 環境モニタリングだけでなく、防犯・防災、スマート農業、工場自動化、交通システム、インフラ監視など、センサーネットワークが利用されるあらゆる分野の設計最適化に応用可能。

センサーネットワーク設計は、モノのインターネット (IoT) の普及に伴い、ますます重要性が高まっている分野であり、数理最適化はその効率性と効果を最大化するための重要なツールとなります。


#| hide

## センサーネットワーク配置の最適化問題

### 1. 問題の定義と定式化

この問題は、限られた予算内でセンサーの設置場所と種類を最適に選択し、監視対象エリアのカバレッジを最大化する、典型的な**最大被覆問題（Maximal Covering Location Problem）**の一種です。特に、各ターゲットの重要度を考慮に入れるため、重み付きの最大被覆問題として定式化します。

#### 集合とインデックス
* $L$: センサー設置候補地点の集合 ($l \in L$)
* $S$: センサータイプの集合 ($s \in S$)
* $J$: 監視ターゲットポイントの集合 ($j \in J$)

#### パラメータ
* $InstallCost_s$: センサータイプ $s$ の設置コスト
* $CommCost_s$: センサータイプ $s$ の（初年度）通信コスト
* $Budget$: 利用可能な総予算
* $Importance_j$: ターゲットポイント $j$ の重要度
* $Covers_{lsj}$: 候補地点 $l$ にセンサー $s$ を設置した際に、ターゲット $j$ がカバーされるか否かを示すバイナリ値（カバーされるなら1、されないなら0）

#### 決定変数
* $Place_{ls} \in \{0, 1\}$: 候補地点 $l$ にセンサータイプ $s$ を設置する場合に1、そうでない場合に0をとるバイナリ変数。
* $IsCovered_j \in \{0, 1\}$: ターゲットポイント $j$ がいずれかのセンサーによってカバーされる場合に1、そうでない場合に0をとるバイナリ変数。

#### 目的関数
カバーされた各ターゲットの重要度の合計（総カバレッジスコア）を最大化します。
$$\text{maximize} \quad \sum_{j \in J} Importance_j \cdot IsCovered_j$$

#### 制約条件
1.  **設置場所制約:** 各設置候補地点には、最大でも1つのセンサーしか設置できません。
    $$\sum_{s \in S} Place_{ls} \le 1 \quad (\forall l \in L)$$
2.  **予算制約:** 設置する全てのセンサーの総コスト（設置費＋通信費）は、総予算を超えてはなりません。
    $$\sum_{l \in L} \sum_{s \in S} (InstallCost_s + CommCost_s) \cdot Place_{ls} \le Budget$$
3.  **カバレッジ定義:** あるターゲットポイント $j$ がカバーされる（$IsCovered_j = 1$ となる）のは、そのターゲットを監視可能なセンサーが少なくとも1つ設置された場合に限ります。この関係は以下の制約で表現できます。
    $$IsCovered_j \le \sum_{l \in L} \sum_{s \in S, \text{ s.t. } Covers_{lsj}=1} Place_{ls} \quad (\forall j \in J)$$
    この制約により、ターゲット $j$ をカバーするセンサーが一つも設置されない場合（右辺が0）、$IsCovered_j$ は0にならざるを得ません。目的関数が $IsCovered_j$ の最大化を目指すため、右辺が1以上になれば $IsCovered_j$ は1となり、正しくカバレッジを表現できます。


In [5]:
%%ampl_eval

# -----------------------------------------------
# モデル (sensor_placement.mod)
# -----------------------------------------------
reset; model;

# --- 集合 ---
set LOCATIONS;      # センサー設置候補地点
set TARGETS;        # 監視ターゲットポイント
set SENSOR_TYPES;   # センサータイプ

# --- パラメータ ---
# センサーのコスト
param InstallCost{SENSOR_TYPES} >= 0;
param CommCost{SENSOR_TYPES} >= 0;

# 予算
param Budget >= 0;

# ターゲットの重要度
param Importance{TARGETS} >= 0;

# カバー可能かを示すバイナリパラメータ (疎なデータなのでdefault 0を指定)
param Covers{LOCATIONS, SENSOR_TYPES, TARGETS} binary default 0;


# --- 変数 ---
# Place[l,s]=1 ならば、地点lにセンサーsを設置する
var Place{LOCATIONS, SENSOR_TYPES} binary;

# IsCovered[j]=1 ならば、ターゲットjがカバーされている
var IsCovered{TARGETS} binary;


# --- 目的関数 ---
# カバーされたターゲットの重要度の合計（カバレッジスコア）を最大化
maximize TotalCoverageScore:
    sum{j in TARGETS} Importance[j] * IsCovered[j];


# --- 制約条件 ---
# 1. 各設置候補地には最大1つのセンサーしか置けない
subject to OneSensorPerLocation{l in LOCATIONS}:
    sum{s in SENSOR_TYPES} Place[l, s] <= 1;

# 2. 総コストは予算内に収める
subject to BudgetConstraint:
    sum{l in LOCATIONS, s in SENSOR_TYPES} (InstallCost[s] + CommCost[s]) * Place[l, s] <= Budget;

# 3. ターゲットjがカバーされているか否かを定義
subject to DefineCoverage{j in TARGETS}:
    IsCovered[j] <= sum{l in LOCATIONS, s in SENSOR_TYPES: Covers[l,s,j] = 1} Place[l,s];


# -----------------------------------------------
# データ (sensor_placement.dat)
# -----------------------------------------------
data;

set LOCATIONS := L1 L2 L3 L4 L5;
set TARGETS   := T1 T2 T3 T4 T5 T6 T7 T8 T9 T10;
set SENSOR_TYPES := SensorA SensorB;

# センサーコスト
param: InstallCost CommCost :=
    SensorA   5  1
    SensorB  12  2;

# 総予算
param Budget := 40;

# ターゲットの重要度
param Importance :=
    T1 5  T2 4  T3 5  T4 3  T5 2
    T6 3  T7 4  T8 2  T9 3  T10 5;

# カバー可能か (値が1のものだけを記述)
param Covers :=
    # SensorA
    L1 SensorA T1 1  L1 SensorA T2 1
    L2 SensorA T2 1  L2 SensorA T3 1  L2 SensorA T4 1
    L3 SensorA T4 1  L3 SensorA T5 1  L3 SensorA T6 1
    L4 SensorA T6 1  L4 SensorA T7 1  L4 SensorA T8 1
    L5 SensorA T8 1  L5 SensorA T9 1  L5 SensorA T10 1
    # SensorB
    L1 SensorB T1 1  L1 SensorB T2 1  L1 SensorB T3 1  L1 SensorB T4 1
    L2 SensorB T1 1  L2 SensorB T2 1  L2 SensorB T3 1  L2 SensorB T4 1  L2 SensorB T5 1  L2 SensorB T6 1
    L3 SensorB T2 1  L3 SensorB T3 1  L3 SensorB T4 1  L3 SensorB T5 1  L3 SensorB T6 1  L3 SensorB T7 1  L3 SensorB T8 1
    L4 SensorB T4 1  L4 SensorB T5 1  L4 SensorB T6 1  L4 SensorB T7 1  L4 SensorB T8 1  L4 SensorB T9 1  L4 SensorB T10 1
    L5 SensorB T8 1  L5 SensorB T9 1  L5 SensorB T10 1;

# -----------------------------------------------
# 実行と結果表示
# -----------------------------------------------
option solver highs;
solve;

printf "\n--- 最適なセンサー配置計画 ---\n";
printf "%-12s %-12s %-12s\n", "設置場所", "センサー", "総コスト";
printf "--------------------------------------\n";
for {l in LOCATIONS, s in SENSOR_TYPES: Place[l,s] > 0.5} {
    printf "%-12s %-12s %-12.1f 万円\n", l, s, InstallCost[s] + CommCost[s];
}
printf "--------------------------------------\n";
printf "使用総コスト: %.1f 万円 (予算: %.1f 万円)\n", sum{l in LOCATIONS, s in SENSOR_TYPES} (InstallCost[s] + CommCost[s]) * Place[l,s], Budget;
printf "達成カバレッジスコア: %d\n\n", TotalCoverageScore;

printf "--- カバーされたターゲット ---\n";
for {j in TARGETS: IsCovered[j] > 0.5} {
    printf "%s (重要度: %d)\n", j, Importance[j];
}
printf "\n";

HiGHS 1.10.0: optimal solution; objective 36
4 simplex iterations
1 branching nodes

--- 最適なセンサー配置計画 ---
設置場所 センサー 総コスト
--------------------------------------
L1           SensorB      14.0         万円
L4           SensorB      14.0         万円
--------------------------------------
使用総コスト: 28.0 万円 (予算: 40.0 万円)
達成カバレッジスコア: 36

--- カバーされたターゲット ---
T1 (重要度: 5)
T2 (重要度: 4)
T3 (重要度: 5)
T4 (重要度: 3)
T5 (重要度: 2)
T6 (重要度: 3)
T7 (重要度: 4)
T8 (重要度: 2)
T9 (重要度: 3)
T10 (重要度: 5)



## 14. 時間帯ごとに複数の仕事をもつシフトスケジューリング問題

**背景:**
「ミニマート・サポート」は、小規模なオンラインストアの顧客対応を3名のスタッフで行っています。効率的な運営のため、3日間のシフトを計画する必要があります。

**目的:**
3名のスタッフを、3日間のシフトに割り当てる。その際、各シフトの必要人数を満たし、特定の勤務ルールを守りつつ、スタッフ間の勤務時間の公平性をできるだけ保つ。

**登場要素:**

**1. 対象期間:**

   1日目、2日目、3日目の3日間。

**2. スタッフ:**

   | スタッフID | 氏名   |
   | :--------- | :----- |
   | S1         | 山田太郎 |
   | S2         | 佐藤花子 |
   | S3         | 田中一郎 |

**3. 勤務時間帯:**

   *   朝 (Morning)
   *   昼 (Day)
   *   夜 (Night)
   *(簡単化のため、全日共通の3時間帯とします)*

**4. 仕事の種類:**

   *   受付 (Reception)
   *   対応 (Support)
   *(簡単化のため、2種類の仕事とします)*

**5. シフトの定義と必要人数:**
   「日・時間帯・仕事」の組み合わせを「シフト」とします。

   | 日   | 時間帯 | 仕事     | シフトID | 必要人数 |
   | :--- | :----- | :------- | :------- | :------- |
   | 1日目| 朝     | 受付     | D1MA_Rec | 1人      |
   | 1日目| 朝     | 対応     | D1MA_Sup | 1人      |
   | 1日目| 昼     | 受付     | D1MD_Rec | 1人      |
   | 1日目| 昼     | 対応     | D1MD_Sup | 1人      |
   | 1日目| 夜     | 対応     | D1MN_Sup | 1人      |
   | 2日目| 朝     | 受付     | D2MA_Rec | 1人      |
   | 2日目| 朝     | 対応     | D2MA_Sup | 1人      |
   | 2日目| 昼     | 受付     | D2MD_Rec | 1人      |
   | 2日目| 昼     | 対応     | D2MD_Sup | 1人      |
   | 2日目| 夜     | 対応     | D2MN_Sup | 1人      |
   | 3日目| 朝     | 受付     | D3MA_Rec | 1人      |
   | 3日目| 朝     | 対応     | D3MA_Sup | 1人      |
   | 3日目| 昼     | 受付     | D3MD_Rec | 1人      |
   | 3日目| 昼     | 対応     | D3MD_Sup | 1人      |
   | 3日目| 夜     | 対応     | D3MN_Sup | 1人      |

**6. スタッフのスキルと割り当て可能性:**
   各スタッフが担当可能な仕事。

   | スタッフID | 担当可能な仕事 |
   | :--------- | :------------- |
   | S1 (山田)  | 受付, 対応     |
   | S2 (佐藤)  | 対応           |
   | S3 (田中)  | 受付           |

**7. シフト割り当てに関する制約・要望:**
   （元の問題から主要なものを抜粋・単純化）

   1.  **必要人数の充足 (厳守):** 各シフトには、表5で定められた「必要人数」のスタッフが必ず割り当てられること。
   2.  **夜勤明けの朝勤回避 (厳守):** 前日の「夜(Night)」の時間帯に勤務したスタッフは、翌日の「朝(Morning)」の時間帯の勤務はできない。
   3.  **連続夜勤日数制限 (厳守):** いかなるスタッフも、「夜(Night)」勤務が2日以上連続しないこと（最大1日まで）。 (元の4日から大幅に短縮)
   4.  **作業量の公平性 (努力目標):** 各スタッフの3日間における総勤務シフト数ができるだけ均等になるようにする。
        *   具体的には、全スタッフの総勤務シフト数のうち、最大値と最小値の差をできるだけ小さくしたい。

**解決したい問題:**

「ミニマート・サポート」の管理者は、3名のスタッフ（山田、佐藤、田中）を上記の条件を考慮しながら、3日間の全シフトに割り当てる必要があります。特に、1～3の条件は必ず守り、その上で4の作業量の公平性をできるだけ達成したいと考えています。

**期待されるアウトプット:**
*   3日間の各シフトに、どのスタッフが割り当てられるかの具体的なシフト表。
*   各スタッフの総勤務シフト数、およびその最大と最小の差。


<!-- 某社では、社内の各部署のシフトの最適化を考えている。シフトは、朝、昼、夜の3つの時間帯（休日は2つの時間帯）だが、時間帯ごとに複数の仕事があり、
またスタッフによって可能な仕事が限定されている。２週間（実際には１ヶ月）のシフトに30名のスタッフを割り振りたいが、以下の条件が付加される。

1. 日・時間帯・仕事の組をシフトとし、必要な人数は予め与えられている。なお、実際にはシフト数は15程度ある。
2. 前日の夜に仕事をした場合には、翌日の朝の勤務は避ける。
3. 週に夜と昼の勤務がなるべく重ならないようにする。
4. 4日連続の夜勤は避ける。
5. 各スタッフの作業量はできるだけ公平になるようにする。 -->


#| hide

## スタッフスケジューリングの最適化問題

提示された課題は、複数の制約を遵守しながら、スタッフの勤務公平性を最大化するようなシフト割り当てを決定する、典型的な数理最適化問題です。AMPLを用いて、この問題を混合整数計画問題としてモデル化し、解決します。

### 1. 問題の定義と定式化

この問題は、どのスタッフをどのシフト（日・時間帯・仕事の組み合わせ）に割り当てるかを決定する問題です。厳守すべき複数のルール（必要人数、スキル、勤務パターン）を満たした上で、全スタッフの総勤務時間（シフト数）のばらつきを最小化することを目的とします。

#### 集合とインデックス
* $S$: スタッフの集合 ($s \in S$)
* $D$: 日の集合 ($d \in D$)
* $T$: 時間帯の集合 ($t \in T$)
* $J$: 仕事の種類の集合 ($j \in J$)
* $K \subseteq D \times T \times J$: 存在するシフトの集合 ($k=(d,t,j) \in K$)
* $Skill_{sj} \subseteq S \times J$: スタッフ $s$ が仕事 $j$ を担当可能であることを示すペアの集合

#### パラメータ
* $Req_{k}$: シフト $k$ に必要なスタッフの人数

#### 決定変数
* $Assign_{sk} \in \{0, 1\}$: スタッフ $s$ をシフト $k$ に割り当てる場合に1、そうでない場合に0をとるバイナリ変数。
* $TotalShifts_s$: スタッフ $s$ の総勤務シフト数（連続変数）。
* $MaxShifts, MinShifts$: 全スタッフの総勤務シフト数の最大値と最小値（連続変数）。

#### 目的関数
スタッフ間の総勤務シフト数の差（最大値と最小値の差）を最小化することで、勤務の公平性を目指します。
$$\text{minimize} \quad \text{FairnessGap}: MaxShifts - MinShifts$$

#### 制約条件
1.  **スキル制約:** スタッフは、担当可能な仕事のシフトにのみ割り当て可能です。これは、決定変数 $Assign_{sk}$ を、スキル上可能な組み合わせ $(s,k)$ についてのみ定義することでモデルに組み込みます。
2.  **必要人数充足:** 各シフトには、定められた必要人数のスタッフを割り当てなければなりません。
    $$\sum_{s \in S} Assign_{sk} = Req_{k} \quad (\forall k \in K)$$
3.  **夜勤明け朝勤の回避:** 前日の夜勤を担当したスタッフは、翌日の朝の勤務はできません。
    $$\sum_{j \in J, k=(d,'Night',j) \in K} Assign_{sk} + \sum_{j' \in J, k'=(d+1,'Morning',j') \in K} Assign_{sk'} \le 1 \quad (\forall s \in S, \forall d \in D \text{ s.t. } d < |D|)$$
4.  **連続夜勤の制限:** 夜勤は連続で最大1日までです（2日連続は不可）。
    $$\sum_{j \in J, k=(d,'Night',j) \in K} Assign_{sk} + \sum_{j' \in J, k'=(d+1,'Night',j') \in K} Assign_{sk'} \le 1 \quad (\forall s \in S, \forall d \in D \text{ s.t. } d < |D|)$$
5.  **公平性のための定義:** 目的関数で用いる変数を定義します。
    * $TotalShifts_s = \sum_{k \in K} Assign_{sk} \quad (\forall s \in S)$
    * $MaxShifts \ge TotalShifts_s \quad (\forall s \in S)$
    * $MinShifts \le TotalShifts_s \quad (\forall s \in S)$


In [9]:
#| hide

%%ampl_eval
reset;
model;
# ======== Sets ========
set EMPLOYEES = 1..10;  # 利用可能な従業員の集合 (最大10人)

param MaxDay = 14;
set DAYS = 1..MaxDay;  # 日の集合 
param WEEKS = 2;

param StartDay{1..WEEKS+1}; #１週間の開始日

set SHIFTS = {"Morning", "Evening", "Evening2", "Night", "Night2", "OFF"}; # シフトの種類 (OFFは休日)

#set SHIFT4E{EMPLOYEES};
#set SHIFT4D{DAYS};

set WORKING_SHIFTS = SHIFTS diff {"OFF"}; # 勤務シフトの集合

# ======== Parameters ========
param ShiftHours{SHIFTS} default 8; # 各シフトの労働時間 (OFFは0時間にすべきだが、今回は使わない)
param AverageHours = 96;            # 平均労働時間（公平性のため）

param SUNDAY integer default 7;     # 日曜日を表す数値
param SATURDAY integer default 6;   # 土曜日を表す数値

# 日ごとのシフトに必要な従業員数
param RequiredStaff{DAYS, WORKING_SHIFTS};

# 次の日を計算するためのパラメータ (ラップアラウンド対応)
param NextDay{d in DAYS} = (d mod card(DAYS)) + 1;

# ======== Variables ========
# Assign[e, d, s] = 1 iff employee e works shift s on day d
var Assign{EMPLOYEES, DAYS, SHIFTS} binary;
#スタッフごと（日ごと）に可能なシフトが異なる場合
#var Assign{e in EMPLOYEES, d in DAYS, SHIFT4E[e] inter SHIFT4D[d]} binary;

var TotalHours{EMPLOYEES} >=0;

var NightShift{EMPLOYEES, 1..WEEKS} binary;  #週に夜勤が入ったら１
var DayShift{EMPLOYEES, 1..WEEKS} binary; 

# ======== Objective Function ========
# 目的：スケジュールに必要な従業員数を最小化する
minimize Fairness:
  sum {e in EMPLOYEES} abs(AverageHours-TotalHours[e]);

# ======== Constraints ========
# 夜勤と日勤の設定
s.t. NightShiftConnect{w in 1..WEEKS, d in StartDay[w]..StartDay[w+1]-1, e in EMPLOYEES}:
    Assign[e,d,"Night"] <= NightShift[e,w];

s.t. NightShiftConnect1{w in 1..WEEKS, d in StartDay[w]..StartDay[w+1]-1, e in EMPLOYEES}:
    Assign[e,d,"Night2"] <= NightShift[e,w];

s.t. MorningShiftConnect{w in 1..WEEKS, d in StartDay[w]..StartDay[w+1]-1, e in EMPLOYEES}:
    Assign[e,d,"Morning"] <= DayShift[e,w];

#4連続夜勤の回避
s.t. ConsecutiveNight{e in EMPLOYEES, d in 1..MaxDay-4}:
    sum {t in 0..4} (Assign[e,d+t,"Night"] +Assign[e,d+t,"Night2"]) <=3;


#週ごとに夜勤と日勤が混ざるのを禁止
s.t. ShiftMix{w in 1..WEEKS, e in EMPLOYEES}:
    NightShift[e,w] + DayShift[e,w] <=1;


# --- 1. Staffing Coverage Constraint ---
# 各日の各勤務シフトで必要な人数を確保する
subject to StaffingRequirement {d in DAYS, s in WORKING_SHIFTS}:
  sum {e in EMPLOYEES} Assign[e, d, s] >= RequiredStaff[d, s];

# --- 2. One Assignment Per Day Constraint ---
# 各従業員は、各日に必ず1つの状態（勤務または休日）を持つ
subject to OneAssignmentPerDay {e in EMPLOYEES, d in DAYS}:
  sum {s in SHIFTS} Assign[e, d, s] = 1;

# --- 3. 総稼働時間の計算
subject to ComputeTotalHours{e in EMPLOYEES}:
  sum {d in DAYS, s in WORKING_SHIFTS} Assign[e, d, s] * ShiftHours[s] == TotalHours[e];

# --- 4. Minimum Rest Between Shifts Constraint (12 hours) ---
# 特定のシフトの組み合わせを禁止する
# 終了時刻と次の開始時刻の間が12時間未満になる組み合わせ:
# - Evening (ends 22:00) -> Morning (starts 6:00 next day) : 8 hours rest -> FORBIDDEN
# - Night (ends 6:00) -> Evening (starts 14:00 next day) : 8 hours rest -> FORBIDDEN
# - Morning (ends 14:00) -> Night (starts 22:00 same day) : 8 hours rest -> IMPOSSIBLE due to OneAssignmentPerDay
# - Night (ends 6:00) -> Morning (starts 6:00 next day) : 24 hours rest -> OK
# - Other combinations have >= 16 hours rest -> OK

subject to MinRest_Eve_Morn {e in EMPLOYEES, d in DAYS}:
  Assign[e, d, "Evening"] + Assign[e, NextDay[d], "Morning"] <= 1;

subject to MinRest_Ngt_Eve {e in EMPLOYEES, d in DAYS}:
  Assign[e, d, "Night"] + Assign[e, NextDay[d], "Evening"] <= 1;

data;
param StartDay := 1 1 2 8 3 15; #週の開始日

# 曜日とシフトごとの必要人数データ
param RequiredStaff : "Morning" "Evening" "Evening2" "Night" "Night2":=
  1  2 2 1 2 1  # Monday
  2  2 2 1 2 1  # Tuesday
  3  2 2 1 2 1  # Wednesday
  4  2 2 1 2 1  # Thursday
  5  2 2 1 2 1  # Friday
  6  2 2 1 2 1  # Saturday (Day 6)
  7  1 1 1 2 1  # Sunday   (Day 7)
  8  2 2 1 2 1  # Monday
  9  2 2 1 2 1  # Tuesday
  10  2 2 1 2 1  # Wednesday
  11  2 2 1 2 1  # Thursday
  12  2 2 1 2 1  # Friday
  13  2 2 1 2 1  # Saturday (Day 6)
  14  1 1 1 2 1  # Sunday   (Day 7)
;

# Specify Sunday and Saturday numbers if different from default
param SUNDAY = 7;
param SATURDAY = 6;

option solver highs;
solve;

# スケジュールを表示 (見やすいように整形)
printf "\n=== Schedule ===\n";
printf "%-10s", "スタッフ   ";
for {d in DAYS} {
  printf " %3s", d; # Display day number header
}
printf "\n";
printf "%-10s", "----------";
for {d in DAYS} {
  printf " ---";
}
printf "\n";

for {e in EMPLOYEES}{
  printf "%-10s", e;
  for {d in DAYS} {
    for {s in SHIFTS} {
      if Assign[e, d, s] > 0.5 then {
         # Display first letter of shift or OFF
         if s == "Morning" then {printf " %3s", "M"}
         if s == "Evening" then {printf " %3s", "E"}
           if s == "Evening2" then {printf " %3s", "E2"}
         if s == "Night" then {printf " %3s", "N"}
          if s == "Night2" then {printf " %3s", "N2"}
         if s == "OFF" then {printf " %3s", "OFF"};
      }
    }
  }
  printf "\n";
}

# 勤務時間や休日数の確認
display {e in EMPLOYEES} (sum {d in DAYS, s in WORKING_SHIFTS} Assign[e,d,s] * ShiftHours[s]);
display {e in EMPLOYEES} (sum {d in DAYS} Assign[e,d,"OFF"]);

HiGHS 1.10.0: optimal solution; objective 0
4834 simplex iterations
1 branching nodes

=== Schedule ===
スタッフ      1   2   3   4   5   6   7   8   9  10  11  12  13  14
---------- --- --- --- --- --- --- --- --- --- --- --- --- --- ---
1           E2  N2   E   N  N2 OFF  N2   M   M   M   M OFF   M   E
2            M OFF   M OFF   M  E2   M   N   N  E2   E   N  N2   N
3            N   N  E2   E   N  N2   N OFF   M   M   M   M OFF   M
4            E   N   N   N OFF   E   N OFF   E   N  E2   E  E2   E
5            N OFF  N2  E2   N   N OFF  E2   E   E  N2   E   N   N
6            M   E OFF   M   M   M  E2  N2  E2   N   N  E2   N OFF
7            M   M   M   E   E   E OFF   E  N2  N2   N OFF   E   N
8          OFF   M  E2   M   E   E   E   E   N OFF   N  N2   E  N2
9           N2   E   N  N2   E   N  E2   N   N OFF   E   N  N2 OFF
10           E  E2   E   E  E2   M   M   M OFF   E OFF   M   M  E2
sum{d in DAYS, s in WORKING_SHIFTS} Assign[e,d,s]*ShiftHours[s] [*] :=
 1  96
 2  96
 3  96
 4 

## 15. 均等化を考慮したチーム割当問題

**背景:**

「未来技術研究所」は、社内の多様な部門からメンバーを集め、3日間の「イノベーション・ワークショップ」を開催します。参加者は新しいアイデアを創出し、プロトタイプを開発するために複数のチームに分かれて活動します。効果的なワークショップのためには、各チームの能力を最大限に引き出し、かつチーム間の経験やスキルセットのバランスを取ることが重要です。

**目的:**

ワークショップに参加する社員を複数のチームに割り当て、以下の目標を達成する。

1.  各社員をいずれか1つのチームに割り当てる。
2.  各社員を特定のチームに割り当てた際に得られる「貢献度（利得）」の全チーム合計を最大化する。
3.  各チームの平均年齢と平均スキルポイントが、チーム間でできるだけ均等になるようにする。
4.  各チームの平均年齢と平均スキルポイントが、定められた範囲内（下限と上限）に収まるようにする。
5.  均等化の度合いは、チーム間の平均値の差の合計（重み付き）を最小化するか、あるいは年齢の均等化を優先し、次にスキルポイントの均等化を優先する（辞書的順序）形で評価する。

**登場要素:**

**1. 参加者 (社員):**

   ワークショップに参加する社員は5名です。
   
   | 社員ID | 氏名     | 年齢 ($a_i$) | スキルポイント ($b_i$) (1-10点) |
   | :----- | :------- | :----------- | :------------------------------ |
   | E001   | 田中太郎 | 30           | 7                               |
   | E002   | 佐藤花子 | 45           | 9                               |
   | E003   | 鈴木一郎 | 25           | 6                               |
   | E004   | 高橋次郎 | 38           | 8                               |
   | E005   | 伊藤三郎 | 52           | 5                               |

**2. チーム:**

   編成するチームは2チームです。
   
   *   チームA
   *   チームB

**3. チーム割り当て時の貢献度（利得） ($p_{ij}$):**
   各社員を各チームに割り当てた場合に期待される貢献度（アイデア発想力、専門性などを総合的に評価）。

   | 社員ID | チームAへの貢献度 | チームBへの貢献度 |
   | :----- | :---------------- | :---------------- |
   | E001   | 8                 | 7                 |
   | E002   | 9                 | 9                 |
   | E003   | 6                 | 7                 |
   | E004   | 7                 | 8                 |
   | E005   | 5                 | 6                 |

**4. チームの属性に関する目標範囲:**
   各チームに割り当てられたメンバーの平均年齢と平均スキルポイントについて、以下の範囲内であることが望ましい。

   | チーム   | 平均年齢下限 | 平均年齢上限 | 平均スキル下限 | 平均スキル上限 |
   | :------- | :----------- | :----------- | :------------- | :------------- |
   | チームA  | 30           | 45           | 6.0            | 8.0            |
   | チームB  | 30           | 45           | 6.0            | 8.0            |
  
**5. チーム間の均等化に関する目標:**

   *   **目標1 (貢献度最大化):** 上記3の貢献度の合計を最大にする。
   *   **目標2 (均等化 - 重み和最小化アプローチ):**
        チームAとチームBの「平均年齢の差の絶対値」と「平均スキルポイントの差の絶対値」を計算し、これらの差に適切な重みを掛けて合計した値を最小化する。
        (例: `重み年齢 * |平均年齢_A - 平均年齢_B| + 重みスキル * |平均スキル_A - 平均スキル_B|` を最小化)
        *   年齢差の重み: 1.0
        *   スキルポイント差の重み: 2.0 (スキル差の方をより重視)
        *   
   *   **目標3 (均等化 - 辞書的順序アプローチ):**
        1.  まず、チーム間の「平均年齢の差の絶対値」を最小化する。
        2.  その上で（平均年齢差が最小となる割り当ての中で）、「平均スキルポイントの差の絶対値」を最小化する。

**解決したい問題:**

「未来技術研究所」のワークショップ運営担当者は、5名の社員を2つのチーム（チームA、チームB）に割り当てる必要があります。以下の条件と目標を考慮して、最適なチーム編成を決定してください。

1.  **全員割り当て:** 各社員は、必ずチームAまたはチームBのいずれか1つのチームに割り当てられる。
2.  **貢献度の最大化:** 全体の貢献度の合計が最も高くなるようにする。
3.  **チーム属性の制約:** 各チームの平均年齢と平均スキルポイントが、表4で定められた下限と上限の範囲にできるだけ収まるようにする（努力目標、あるいは厳密な制約）。
4.  **チーム間の均等化:** 上記「5. チーム間の均等化に関する目標」で述べた、重み和アプローチまたは辞書的順序アプローチのいずれかに基づいて、チーム間の平均年齢と平均スキルポイントのバランスを取る。

**期待されるアウトプット:**

*   各社員がどのチームに割り当てられるかのリスト。
*   その割り当てによって達成される総貢献度。
*   各チームの平均年齢と平均スキルポイント、およびチーム間のこれらの値の差。
*   （均等化目標を達成するための目的関数の値）


#| hide
* $N$ 人の人を $M$ 個のチームに割り振る。
* 各人 $i (i = 1, ..., N)$ はチーム $j (i = 1, ..., M)$ へ割り振ったときの利得 $p_{ij}$ が与えられていて、
  その和を最大化したい。
* 各人 $i$  (i = 1, ..., N) は年齢 $a_i$ や能力 $b_i$ というパラメータを持ち、目標は、各チームに割り振られた人の年齢や能力の平均値が、チーム間で可能な限り均等になるようにしたい。
*   チームごとに、年齢や能力値の下限と上限が与えられている。
*   望ましい均等化はあいまいで、重み和を最小化するか、優先度（辞書的順序）に基づき計算したい。

#| hide

**定式化**

**1. 集合とパラメータ**

*   `I = {1, 2, ..., N}`: 人の集合
*   `J = {1, 2, ..., M}`: チームの集合
*   `a_i`: 人 `i` の年齢 (既知のパラメータ, `i ∈ I`)

**2. 決定変数**

*   `x_ij`: 人 `i` がチーム `j` に割り振られる場合に 1、そうでない場合に 0 をとるバイナリ変数 (`i ∈ I`, `j ∈ J`)。
    *   `x_ij ∈ {0, 1}`

**3. 計算用の中間変数 (補助変数)**

これらは厳密には決定変数ではありませんが、目的関数や制約条件を記述しやすくするために定義します。

*   `NumPeople_j`: チーム `j` に割り振られた人数
    *   `NumPeople_j = ∑_{i ∈ I} x_ij` (`j ∈ J`)
*   `TotalAge_j`: チーム `j` に割り振られた人の年齢の合計
    *   `TotalAge_j = ∑_{i ∈ I} a_i * x_ij` (`j ∈ J`)
*   `AvgAge_j`: チーム `j` の平均年齢
    *   `AvgAge_j = TotalAge_j / NumPeople_j` (`j ∈ J`)
    *   (注意: この形式は非線形であり、直接的な線形計画問題の定式化には工夫が必要です。後述の目的関数で対応します。)

**4. 制約条件**

*   **各人はちょうど1つのチームに所属する:**
    *   `∑_{j ∈ J} x_ij = 1` (すべての `i ∈ I` について)

*   **(任意) 各チームの最小/最大人数 (必要に応じて追加):**
    *   `L_j ≤ NumPeople_j ≤ U_j` (すべての `j ∈ J` について)
    *   ここで `L_j` はチーム `j` の最小人数、`U_j` は最大人数。もしチームが空になることを許容しないなら `L_j ≥ 1` とします。

*   **変数の型:**
    *   `x_ij ∈ {0, 1}` (すべての `i ∈ I`, `j ∈ J` について)

**5. 目的関数**

チーム間の平均年齢のばらつきを最小化する方法はいくつか考えられます。

**方法1: 平均年齢の最大値と最小値の差を最小化 (Min-Max)**

1.  全体の平均年齢 `A_overall` を計算します: `A_overall = (∑_{i ∈ I} a_i) / N`
2.  各チームの平均年齢 `AvgAge_j` と全体の平均年齢 `A_overall` との乖離（ずれ）を考えます。
3.  最大の乖離を最小化することを目指します。補助変数 `D_max` (最大乖離) を導入します。

*   **補助変数:**
    *   `D_max ≥ 0`: 各チームの平均年齢と全体平均年齢との間の最大の絶対差を表す連続変数。

*   **目的関数:**
    *   `Minimize D_max`

*   **追加の制約条件 (線形化):**
    `AvgAge_j` の定義 `TotalAge_j / NumPeople_j` は非線形です。これを線形化するために、平均年齢そのものではなく、「チームの合計年齢」と「(全体平均年齢) × (チーム人数)」との差を考えます。この差が 0 に近いほど、そのチームの平均年齢は全体平均年齢に近くなります。
    *   `∑_{i ∈ I} (a_i - A_overall) * x_ij ≤ D_max` (すべての `j ∈ J` について)
    *   `∑_{i ∈ I} (a_i - A_overall) * x_ij ≥ -D_max` (すべての `j ∈ J` について)

    これらの制約は、 `| ∑_{i ∈ I} a_i * x_ij - A_overall * ∑_{i ∈ I} x_ij | ≤ D_max` を線形に表現したものです。つまり、`| TotalAge_j - A_overall * NumPeople_j | ≤ D_max` を意味します。
    チーム人数 `NumPeople_j` が 0 でない限り、これを `NumPeople_j` で割ると `| AvgAge_j - A_overall | ≤ D_max / NumPeople_j` となり、間接的に平均年齢の乖離を抑えることになります。 (※もしチームサイズが大きく異なる場合、この目的関数は完全に平均年齢の差を最小化するわけではない可能性がありますが、実用上よく使われる線形化アプローチです。)

**方法2: 平均年齢の分散 (または標準偏差) を最小化 (非線形)**

1.  各チームの平均年齢 `AvgAge_j` を計算します (上記定義)。
2.  全チームの平均年齢の平均値 `μ = (∑_{j ∈ J} AvgAge_j) / M` を計算します。
3.  目的関数: `Minimize ∑_{j ∈ J} (AvgAge_j - μ)^2` (分散) または `Minimize sqrt(∑_{j ∈ J} (AvgAge_j - μ)^2 / M)` (標準偏差)

    *   注意: この目的関数は `AvgAge_j` の定義により非線形 (分数や二乗を含む) となり、混合整数非線形計画問題 (MINLP) となります。解くためには専用のソルバーが必要です。

**方法3: 合計年齢の分散を最小化 (チームサイズが均等な場合の近似)**

もし、各チームの人数 `NumPeople_j` がほぼ等しくなるように制約されているか、期待される場合、平均年齢の均等化は合計年齢 `TotalAge_j` の均等化で近似できます。

1.  目標とするチームあたりの合計年齢 `TargetTotalAge = (∑_{i ∈ I} a_i) / M` を計算します。
2.  目的関数: `Minimize ∑_{j ∈ J} (TotalAge_j - TargetTotalAge)^2`
    *   `Minimize ∑_{j ∈ J} (∑_{i ∈ I} a_i * x_ij - TargetTotalAge)^2`
    *   注意: この目的関数は決定変数 `x_ij` の二乗を含むため、混合整数二次計画問題 (MIQP) となります。MIQPソルバーで解くことができます。

---

**推奨される定式化 (MILP)**

実用上、線形計画問題として扱える **方法1** の定式化がよく用いられます。

**まとめ (方法1に基づくMILP定式化):**

*   **Sets:** `I`, `J`
*   **Parameters:** `a_i`, `N`, `M`, `A_overall = (∑ a_i) / N`
*   **Variables:** `x_ij ∈ {0, 1}`, `D_max ≥ 0`
*   **Objective:** `Minimize D_max`
*   **Constraints:**
    1.  `∑_{j ∈ J} x_ij = 1` (for all `i ∈ I`)
    2.  `∑_{i ∈ I} (a_i - A_overall) * x_ij ≤ D_max` (for all `j ∈ J`)
    3.  `∑_{i ∈ I} (a_i - A_overall) * x_ij ≥ -D_max` (for all `j ∈ J`)
    4.  (Optional) `L_j ≤ ∑_{i ∈ I} x_ij ≤ U_j` (for all `j ∈ J`)

この定式化により、混合整数線形計画 (MILP) ソルバーを用いて、チーム間の平均年齢のばらつき（具体的には、各チームの合計年齢と期待される合計年齢との間の最大絶対差）を最小にするような人の割り当てを見つけることができます。


## 16. 献立問題

**背景:**
「ぷちデリ弁当」は、小規模ながら健康志向の宅配弁当サービスです。コストを抑えつつ、栄養バランスと日々の献立の目新しさを提供することを目指しています。今回は、月曜日から水曜日までの3日間の日替わり弁当（主菜1品、副菜1品）の献立を計画します。

**目的:**
3日間の各日において、主菜と副菜を1品ずつ選び、以下の目標を達成する献立を作成する。
1.  毎日の弁当が、定められた栄養素（カロリー、塩分）の摂取基準（下限と上限）を満たす。
2.  3日間の弁当の総費用をできるだけ安く抑える。
3.  献立のバラエティを豊かにするため、同じ日の主菜と副菜で、また連続する2日間の献立で、同じ「素材」や「調理法」ができるだけ重ならないようにする（ペナルティとして評価）。

**登場要素:**

**1. 対象期間:**
   月曜日、火曜日、水曜日の3日間。

**2. 食品候補:**
   弁当に使用できる食品の候補リスト。

   *   **食品リストと基本情報:**

     
        | 食品ID | 食品名         | 主菜/副菜 | 費用(円) | 主な素材 | 主な調理法 |
        | :----- | :------------- | :-------- | :------- | :------- | :--------- |
        | M01    | 鶏の照り焼き   | 主菜      | 200      | 肉       | 焼き物     |
        | M02    | 鮭の塩焼き     | 主菜      | 220      | 魚       | 焼き物     |
        | M03    | 豚の角煮       | 主菜      | 250      | 肉       | 煮物       |
        | S01    |ほうれん草の胡麻和え| 副菜      | 80       | 野菜     | 和え物     |
        | S02    | きんぴらごぼう | 副菜      | 70       | 野菜     | 炒め物     |
        | S03    | だし巻き卵     | 副菜      | 90       | 卵       | 焼き物     |

   *   **食品ごとの栄養素含有量（1食あたり）:**

     
        | 食品ID | カロリー(kcal) | 塩分(g) |
        | :----- | :------------- | :------ |
        | M01    | 300            | 1.5     |
        | M02    | 250            | 1.2     |
        | M03    | 400            | 1.8     |
        | S01    | 70             | 0.5     |
        | S02    | 90             | 0.7     |
        | S03    | 120            | 0.6     |

**3. 栄養素の摂取基準 (1日あたり):**

   | 栄養素   | 下限  | 上限  |
   | :------- | :---- | :---- |
   | カロリー(kcal) | 400   | 600   |
   | 塩分(g)    | 1.5   | 2.5   |

**4. 素材と調理法:**

   *   **素材の分類:** 肉、魚、野菜、卵
   *   **調理法の分類:** 焼き物、煮物、和え物、炒め物

**5. 重複ペナルティの値 (1回発生するごと):**

   | 重複の種類                                     | ペナルティ(円) |
   | :--------------------------------------------- | :------------- |
   | 同じ日の主菜と副菜の「素材」が同じ              | 30             |
   | 同じ日の主菜と副菜の「調理法」が同じ            | 20             |
   | 連続する2日間の献立で同じ「素材」が使われる      | 40             |
   | 連続する2日間の献立で同じ「調理法」が使われる    | 25             |

**解決したい問題:**

「ぷちデリ弁当」の献立プランナーとして、月曜日から水曜日までの3日間の弁当献立（各日に主菜1品、副菜1品）を決定する必要があります。以下の全ての条件を満たしつつ、3日間の「総費用（材料費）＋ 重複ペナルティの合計」を最小にする献立プランを見つけてください。

1.  **主菜・副菜の選択:** 各日、表2の食品リストから、指定された主菜/副菜の区分に従い、主菜候補から1品、副菜候補から1品を選択する。
2.  **栄養基準の遵守:** 毎日、選択された主菜と副菜の栄養素（カロリー、塩分）の合計が、表3の「栄養素の摂取基準」の下限と上限の範囲内に収まるようにする。
3.  **素材・調理法の重複回避の考慮:** 表5で定義されたペナルティに基づき、素材や調理法の重複を評価する。
4.  **総コストの最小化:** 3日間の食品費用と、発生した重複ペナルティの合計額が最も少なくなるようにする。

**期待されるアウトプット:**
月曜日、火曜日、水曜日の各日について、どの食品を主菜とし、どの食品を副菜とするかの具体的な献立リスト。そして、その献立によって達成される総費用（食品費用＋ペナルティ）、および各日の栄養摂取量。

<!-- 栄養問題の多期間への拡張を考える。1週間分の給食の献立を考える。与えられた食品の候補には、含まれる栄養素（カロリー、塩分、鉄分など）と費用が
与えられている。献立は主菜、副菜に分かれており、それぞれ１品づつ選択する。食品には、肉、魚、野菜などの素材と、煮物、揚げ物、生食などの調理法が与えられている。
目的は、１つの献立、ならびに連続する２日の献立では、できるだけ同じ素材、調理法が重ならないようにすることと、毎日の食事の栄養素の下限と上限を満たす範囲で、
総費用を最小化することである。 -->

#| hide

### 問題の定義と定式化

この問題は、月曜日から水曜日までの3日間の弁当（主菜1品、副菜1品）の献立を、コストと各種ペナルティの合計を最小化するように決定する混合整数計画問題です。

#### 集合とインデックス

* $T$: 曜日の集合 (月, 火, 水)。$t \in T$
* $T_{prev}$: 連続日を評価するための曜日の集合 (月, 火)。$t \in T_{prev}$
* $F$: 食品候補の集合。$f \in F$
* $F_M$: 主菜の集合。$F_M \subset F$
* $F_S$: 副菜の集合。$F_S \subset F$
* $N$: 栄養素の集合（カロリー, 塩分）。$n \in N$
* $M$: 素材の集合（肉, 魚, 野菜, 卵）。$m \in M$
* $C$: 調理法の集合（焼き物, 煮物, 和え物, 炒め物）。$c \in C$

#### パラメータ

* $cost_f$: 食品 $f$ の費用
* $nutval_{n,f}$: 食品 $f$ の栄養素 $n$ の含有量
* $nut\_min_n, nut\_max_n$: 栄養素 $n$ の1日の摂取基準（下限・上限）
* $mat_{m,f}$: 食品 $f$ が素材 $m$ を使うなら1、そうでなければ0
* $cook_{c,f}$: 食品 $f$ が調理法 $c$ を使うなら1、そうでなければ0
* $P_{ms}$: 同じ日の主菜・副菜の素材重複ペナルティ (30円)
* $P_{mc}$: 同じ日の主菜・副菜の調理法重複ペナルティ (20円)
* $P_{ds}$: 連続する日の素材重複ペナルティ (40円)
* $P_{dc}$: 連続する日の調理法重複ペナルティ (25円)

#### 決定変数

* $x_{t,f} \in \{0, 1\}$: 曜日 $t$ に食品 $f$ を選択するなら1、しないなら0
* $y_{ms, t} \in \{0, 1\}$: 曜日 $t$ の主菜と副菜で素材が重複した場合に1
* $y_{mc, t} \in \{0, 1\}$: 曜日 $t$ の主菜と副菜で調理法が重複した場合に1
* $y_{ds, t} \in \{0, 1\}$: 曜日 $t$ と $t+1$ で素材が重複した場合に1
* $y_{dc, t} \in \{0, 1\}$: 曜日 $t$ と $t+1$ で調理法が重複した場合に1
* $u_{m, t} \in \{0, 1\}$: 曜日 $t$ に素材 $m$ が使用された場合に1
* $v_{c, t} \in \{0, 1\}$: 曜日 $t$ に調理法 $c$ が使用された場合に1

#### 定式化

**目的関数**

総コスト（食品費用＋ペナルティ合計）を最小化します。
$$\text{Minimize} \quad \sum_{t \in T} \sum_{f \in F} cost_f \cdot x_{t,f} + \sum_{t \in T} (P_{ms} \cdot y_{ms, t} + P_{mc} \cdot y_{mc, t}) + \sum_{t \in T_{prev}} (P_{ds} \cdot y_{ds, t} + P_{dc} \cdot y_{dc, t})$$

**制約条件**

1.  **主菜・副菜の選択**: 各日、主菜と副菜を1品ずつ選択する。
    $$\forall t \in T: \quad \sum_{f \in F_M} x_{t,f} = 1$$
    $$\forall t \in T: \quad \sum_{f \in F_S} x_{t,f} = 1$$

2.  **栄養基準**: 各日、栄養素の合計が基準範囲内にある。
    $$\forall t \in T, \forall n \in N: \quad nut\_min_n \le \sum_{f \in F} nutval_{n,f} \cdot x_{t,f} \le nut\_max_n$$

3.  **補助変数の定義**: その日に特定の素材・調理法が使われたかを判定する。
    $$\forall t \in T, \forall m \in M: \quad \sum_{f \in F} mat_{m,f} \cdot x_{t,f} \le 2 \cdot u_{m, t}$$
    $$\forall t \in T, \forall m \in M: \quad \sum_{f \in F} mat_{m,f} \cdot x_{t,f} \ge u_{m, t}$$
    $$\forall t \in T, \forall c \in C: \quad \sum_{f \in F} cook_{c,f} \cdot x_{t,f} \le 2 \cdot v_{c, t}$$
    $$\forall t \in T, \forall c \in C: \quad \sum_{f \in F} cook_{c,f} \cdot x_{t,f} \ge v_{c, t}$$

4.  **ペナルティ変数の関連付け**:
    * 同じ日の重複:
        $$\forall t \in T, \forall m \in M: \quad \sum_{f \in F_M} mat_{m,f} \cdot x_{t,f} + \sum_{f \in F_S} mat_{m,f} \cdot x_{t,f} \le 1 + y_{ms, t}$$
        $$\forall t \in T, \forall c \in C: \quad \sum_{f \in F_M} cook_{c,f} \cdot x_{t,f} + \sum_{f \in F_S} cook_{c,f} \cdot x_{t,f} \le 1 + y_{mc, t}$$
    * 連続する日の重複:
        $$\forall t \in T_{prev}, \forall m \in M: \quad u_{m, t} + u_{m, t+1} \le 1 + y_{ds, t}$$
        $$\forall t \in T_{prev}, \forall c \in C: \quad v_{c, t} + v_{c, t+1} \le 1 + y_{dc, t}$$


In [18]:
#| hide
%%ampl_eval

reset;
model;

# 集合の定義
set DAYS := {1, 2, 3};
set FOODS;
set MAIN_DISHES within FOODS;
set SIDE_DISHES within FOODS;
set INGREDIENTS;
set COOKING_METHODS;

# パラメータの定義
param cost{FOODS};
param calories{FOODS};
param salt{FOODS};
param food_ingredient{FOODS, INGREDIENTS} binary;
param food_cooking{FOODS, COOKING_METHODS} binary;

# 栄養素の基準
param min_calories := 400;
param max_calories := 600;
param min_salt := 1.5;
param max_salt := 2.5;

# ペナルティの値
param same_day_ingredient_penalty := 30;
param same_day_cooking_penalty := 20;
param consecutive_ingredient_penalty := 40;
param consecutive_cooking_penalty := 25;

# 決定変数
var select_main{DAYS, MAIN_DISHES} binary;
var select_side{DAYS, SIDE_DISHES} binary;

# ペナルティ変数
var same_day_ingredient_overlap{DAYS, INGREDIENTS} binary;
var same_day_cooking_overlap{DAYS, COOKING_METHODS} binary;
var consecutive_ingredient_overlap{1..2, INGREDIENTS} binary;
var consecutive_cooking_overlap{1..2, COOKING_METHODS} binary;

# 制約条件
s.t. select_one_main{d in DAYS}:
    sum{m in MAIN_DISHES} select_main[d,m] = 1;

s.t. select_one_side{d in DAYS}:
    sum{s in SIDE_DISHES} select_side[d,s] = 1;

s.t. calories_min{d in DAYS}:
    sum{m in MAIN_DISHES} calories[m] * select_main[d,m] + 
    sum{s in SIDE_DISHES} calories[s] * select_side[d,s] >= min_calories;

s.t. calories_max{d in DAYS}:
    sum{m in MAIN_DISHES} calories[m] * select_main[d,m] + 
    sum{s in SIDE_DISHES} calories[s] * select_side[d,s] <= max_calories;

s.t. salt_min{d in DAYS}:
    sum{m in MAIN_DISHES} salt[m] * select_main[d,m] + 
    sum{s in SIDE_DISHES} salt[s] * select_side[d,s] >= min_salt;

s.t. salt_max{d in DAYS}:
    sum{m in MAIN_DISHES} salt[m] * select_main[d,m] + 
    sum{s in SIDE_DISHES} salt[s] * select_side[d,s] <= max_salt;

s.t. same_day_ingredient_check{d in DAYS, i in INGREDIENTS}:
    same_day_ingredient_overlap[d,i] >= 
    sum{m in MAIN_DISHES} food_ingredient[m,i] * select_main[d,m] + 
    sum{s in SIDE_DISHES} food_ingredient[s,i] * select_side[d,s] - 1;

s.t. same_day_cooking_check{d in DAYS, c in COOKING_METHODS}:
    same_day_cooking_overlap[d,c] >= 
    sum{m in MAIN_DISHES} food_cooking[m,c] * select_main[d,m] + 
    sum{s in SIDE_DISHES} food_cooking[s,c] * select_side[d,s] - 1;

s.t. consecutive_ingredient_check{d in 1..2, i in INGREDIENTS}:
    consecutive_ingredient_overlap[d,i] >= 
    (sum{m in MAIN_DISHES} food_ingredient[m,i] * select_main[d,m] + 
     sum{s in SIDE_DISHES} food_ingredient[s,i] * select_side[d,s]) +
    (sum{m in MAIN_DISHES} food_ingredient[m,i] * select_main[d+1,m] + 
     sum{s in SIDE_DISHES} food_ingredient[s,i] * select_side[d+1,s]) - 1;

s.t. consecutive_cooking_check{d in 1..2, c in COOKING_METHODS}:
    consecutive_cooking_overlap[d,c] >= 
    (sum{m in MAIN_DISHES} food_cooking[m,c] * select_main[d,m] + 
     sum{s in SIDE_DISHES} food_cooking[s,c] * select_side[d,s]) +
    (sum{m in MAIN_DISHES} food_cooking[m,c] * select_main[d+1,m] + 
     sum{s in SIDE_DISHES} food_cooking[s,c] * select_side[d+1,s]) - 1;

# 目的関数：総費用の最小化
minimize total_cost:
    sum{d in DAYS, m in MAIN_DISHES} cost[m] * select_main[d,m] +
    sum{d in DAYS, s in SIDE_DISHES} cost[s] * select_side[d,s] +
    sum{d in DAYS, i in INGREDIENTS} same_day_ingredient_penalty * same_day_ingredient_overlap[d,i] +
    sum{d in DAYS, c in COOKING_METHODS} same_day_cooking_penalty * same_day_cooking_overlap[d,c] +
    sum{d in 1..2, i in INGREDIENTS} consecutive_ingredient_penalty * consecutive_ingredient_overlap[d,i] +
    sum{d in 1..2, c in COOKING_METHODS} consecutive_cooking_penalty * consecutive_cooking_overlap[d,c];

data;

# 食品リスト
set FOODS := M01 M02 M03 S01 S02 S03;
set MAIN_DISHES := M01 M02 M03;
set SIDE_DISHES := S01 S02 S03;
set INGREDIENTS := meat fish vegetable egg;
set COOKING_METHODS := grilled simmered mixed stir_fried;

# 食品費用
param cost := 
    M01 200  M02 220  M03 250
    S01 80   S02 70   S03 90;

# 栄養素データ
param calories :=
    M01 300  M02 250  M03 400
    S01 70   S02 90   S03 120;

param salt :=
    M01 1.5  M02 1.2  M03 1.8
    S01 0.5  S02 0.7  S03 0.6;

# 食品の素材マッピング
param food_ingredient: meat fish vegetable egg :=
    M01  1    0    0       0
    M02  0    1    0       0
    M03  1    0    0       0
    S01  0    0    1       0
    S02  0    0    1       0
    S03  0    0    0       1;

# 食品の調理法マッピング
param food_cooking: grilled simmered mixed stir_fried :=
    M01  1       0        0     0
    M02  1       0        0     0
    M03  0       1        0     0
    S01  0       0        1     0
    S02  0       0        0     1
    S03  1       0        0     0;

option solver highs;
solve;

# 結果の表示
printf "=== 最適献立プラン ===\n";
for {d in DAYS} {
    printf "Day %d: ", d;
    for {m in MAIN_DISHES: select_main[d,m] = 1} {
        printf "Main=%s ", m;
    }
    for {s in SIDE_DISHES: select_side[d,s] = 1} {
        printf "Side=%s ", s;
    }
    printf "\n";
}

printf "\n=== 栄養情報 ===\n";
for {d in DAYS} {
    printf "Day %d: Calories=%.0f kcal, Salt=%.1f g\n", d,
        sum{m in MAIN_DISHES} calories[m] * select_main[d,m] + 
        sum{s in SIDE_DISHES} calories[s] * select_side[d,s],
        sum{m in MAIN_DISHES} salt[m] * select_main[d,m] + 
        sum{s in SIDE_DISHES} salt[s] * select_side[d,s];
}

printf "\n=== コスト内訳 ===\n";
printf "食品費用: %.0f円\n",
    sum{d in DAYS, m in MAIN_DISHES} cost[m] * select_main[d,m] +
    sum{d in DAYS, s in SIDE_DISHES} cost[s] * select_side[d,s];
printf "総費用: %.0f円\n", total_cost;


HiGHS 1.10.0: optimal solution; objective 1070
16 simplex iterations
1 branching nodes
=== 最適献立プラン ===
Day 1: Main=M01 Side=S03 
Day 2: Main=M03 Side=S02 
Day 3: Main=M01 Side=S03 

=== 栄養情報 ===
Day 1: Calories=420 kcal, Salt=2.1 g
Day 2: Calories=490 kcal, Salt=2.5 g
Day 3: Calories=420 kcal, Salt=2.1 g

=== コスト内訳 ===
食品費用: 900円
総費用: 1070円


##  17. エア・フロンティア航空の月間乗務員スケジュール作成

**背景:**
エア・フロンティア航空は、ヨーロッパを拠点とする中規模航空会社です。同社は、乗務員の満足度と効率的な運航を両立させるため、乗務員の個々の事情や実行可能な勤務パターンを考慮した上で、月間の乗務スケジュール（個別月間ブロック）を作成する「乗務員勤務表問題 (Crew Rostering Problem)」に取り組んでいます。事前に、運航計画に基づいて効率的な「ペアリング」（ある出発地点（home base）から出発し，
再び同じ出発地点に戻る乗務員のスケジュール）が複数作成されており、これらのペアリングを各乗務員に適切に割り当てる必要があります。

**目的:**
全てのペアリングが必要な人数でカバーされ、各乗務員は実行可能な月間スケジュールを1つだけ担当し、会社全体としてスケジューリングのコスト（乗務員の特性や希望を反映）を最も低くすること。

**登場要素:**

**1. 乗務員**
   今回の対象となるパイロットは3名です。
   
   *   パイロットA
   *   パイロットB
   *   パイロットC

**2. ペアリング と 必要人数**
   1ヶ月間に実施すべきペアリングと、各ペアリングに必要な乗務員の数は以下の通りです。

   | ペアリングID | 必要な乗務員数 |
   | :----------- | :------------- |
   | P1           | 2人            |
   | P2           | 1人            |
   | P3           | 1人            |
   | P4           | 2人            |

**3. 乗務員ごとの実行可能な月間スケジュール候補**
   各パイロットには、実行可能な月間スケジュールの候補がいくつかあります。各スケジュールは、1つ以上のペアリングを含んでいます。

   *   **パイロットA のスケジュール候補:**

        | スケジュールID (パイロットA) | 含まれるペアリング |
        | :--------------------------- | :----------------- |
        | S_A1                         | P1, P3             |
        | S_A2                         | P2, P4             |

   *   **パイロットB のスケジュール候補:**

        | スケジュールID (パイロットB) | 含まれるペアリング |
        | :--------------------------- | :----------------- |
        | S_B1                         | P1, P4             |
        | S_B2                         | P2                 |

   *   **パイロットC のスケジュール候補:**

        | スケジュールID (パイロットC) | 含まれるペアリング |
        | :--------------------------- | :----------------- |
        | S_C1                         | P1, P2, P3         |
        | S_C2                         | P4                 |

**4. 各スケジュールの費用**

   各パイロットが特定の月間スケジュールを担当する場合の「費用」です。この費用が低いほど、会社や乗務員にとって望ましいとされます。

   | パイロット   | スケジュールID | 費用 |
   | :----------- | :------------- | :--- |
   | パイロットA  | S_A1           | 100  |
   | パイロットA  | S_A2           | 120  |
   | パイロットB  | S_B1           | 110  |
   | パイロットB  | S_B2           | 70   |
   | パイロットC  | S_C1           | 150  |
   | パイロットC  | S_C2           | 90   |

**解決したい問題:**

エア・フロンティア航空は、次の2つの条件を必ず満たした上で、全体の費用が最も少なくなるように、各パイロットにどの月間スケジュールを割り当てるか（または割り当てないか）を決めたいと考えています。


1.  **ペアリングのカバー:** 表に記載された全てのペアリング（P1からP4まで）は、それぞれ定められた「必要な乗務員数」が確保されるように、誰かしらのパイロットに担当されなければなりません。

    *   例えば、ペアリングP1は2人の乗務員が必要です。もしパイロットAがS_A1（P1を含む）を担当し、パイロットBがS_B1（P1を含む）を担当した場合、P1は2人の乗務員によってカバーされたことになります。
      
3.  **乗務員へのスケジュール割り当て:** 各パイロット（パイロットA、B、C）は、表3で示された自身の実行可能なスケジュール候補の中から、最大で1つだけしか割り当てられません。

**期待されるアウトプット:**
どのパイロットにどの月間スケジュールを割り当てるのが最適か、そしてその時の最も低い総費用。


#| hide

**集合:**

- $P$: 実行可能なペアリングの集合．ここで **ペアリング**（paring, rotation）とは，ある出発地点（home base）から出発し，
再び同じ出発地点に戻る乗務員のスケジュールを表し，1つ以上のフライトとその間の休息期間から構成される．おおむね1週間以下の乗務員のスケジュールを表す．
- $L$: 乗務員（パイロットもしくはキャビンクルー）の集合
- $S_{\ell}$: 乗務員 $\ell \in L$ に対する実行可能な個別スケジュールの集合

**パラメータ：**

- $C_s^{\ell}$: 乗務員 $\ell \in L$ がスケジュール $s$ を遂行したときの費用．選好する便や休暇を考慮して決定する。
- $n_p$: ペアリング $p$ に必要な乗務員の数
- $e_{p}^{s, \ell}$: ペアリング $p$ が乗務員 $\ell$ のスケジュール $s$ に含まれるとき $1$，それ以外のとき $0$ の定数
- 
**変数:**

- $x_s^{\ell}$: 乗務員 $\ell \in L$ がスケジュール $s \in S_{\ell}$ を遂行するとき $1$，それ以外のとき $0$


In [25]:
#| hide
%%ampl_eval
reset;
model;

# Sets
# -----------------------------------------------------------------------------
set PAIRINGS; # P: 実行可能なペアリングの集合
set CREWS;    # L: 乗務員（パイロットもしくはキャビンクルー）の集合

# 乗務員 l に対する実行可能な個別スケジュールの集合
# データファイルで crew_schedules[l] のように各乗務員に対応するスケジュールを指定
set SCHEDULES {CREWS}; # S_l

# Parameters
# -----------------------------------------------------------------------------
# C_s^l: 乗務員 l がスケジュール s を遂行したときの費用
param cost_schedule {l in CREWS, s in SCHEDULES[l]} >= 0;

# n_p: ペアリング p に必要な乗務員の数
param required_crew_for_pairing {PAIRINGS} >= 0, integer;

# e_p^{s,l}: ペアリング p が乗務員 l のスケジュール s に含まれるとき 1, それ以外のとき 0
param pairing_in_schedule {p in PAIRINGS, l in CREWS, s in SCHEDULES[l]} binary default 0;


# Variables
# -----------------------------------------------------------------------------
# x_s^l: 乗務員 l がスケジュール s を遂行するとき 1, それ以外のとき 0
var AssignSchedule {l in CREWS, s in SCHEDULES[l]} binary;

# Objective Function
# -----------------------------------------------------------------------------
minimize TotalCost:
    sum {l in CREWS, s in SCHEDULES[l]}
        cost_schedule[l,s] * AssignSchedule[l,s];

# Constraints
# -----------------------------------------------------------------------------
# 1. 各ペアリングは必要な数の乗務員によってカバーされなければならない
subject to PairingCoverage {p in PAIRINGS}:
    sum {l in CREWS, s in SCHEDULES[l]}
        pairing_in_schedule[p,l,s] * AssignSchedule[l,s]
    >= required_crew_for_pairing[p];

# 2. 各乗務員は最大1つのスケジュールを遂行する
subject to OneSchedulePerCrew {l in CREWS}:
    sum {s in SCHEDULES[l]} AssignSchedule[l,s] <= 1;

data;

set PAIRINGS := P1 P2 P3;
set CREWS := C1 C2 C3;

# SCHEDULES[l] のデータ例
set SCHEDULES[C1] := C1S1 C1S2;
set SCHEDULES[C2] := C2S1 C2S2 C2S3;
set SCHEDULES[C3] := C3S1;

param cost_schedule :=
  C1  C1S1  100  C1 C1S2 120
  C2  C2S1   90  C2 C2S2 110 C2 C2S3 130
  C3  C3S1  150;

param required_crew_for_pairing :=
  P1 1  P2 1  P3 1;

# pairing_in_schedule[p,l,s] (3次元パラメータ)
param pairing_in_schedule :=
  P1 C1 C1S1 1
  P2 C1 C1S2 1
  P1 C2 C2S1 1
  P3 C2 C2S2 1
  P2 C3 C3S1 1  
;

option solver highs;
solve;
display AssignSchedule;

Solution determined by presolve;
objective TotalCost = 360.
AssignSchedule :=
C1 C1S1   1
C1 C1S2   0
C2 C2S1   0
C2 C2S2   1
C2 C2S3   0
C3 C3S1   1
;



## 18. ケーススタディ：首都圏フーズ物流の挑戦「最適化が導く共同配送計画」

### 登場人物と背景

**佐藤さん**は、中堅食品卸「首都圏フーズ物流」の物流企画マネージャー。彼の部署は、千葉と神奈川にある基幹倉庫から、需要が集中する渋谷と新宿のエリア配送センター（卸）へ、日々商品を補充する重要な役割を担っています。

近年、燃料費の高騰、ドライバー不足、そして「物流の2024年問題」の余波が、物流コストを圧迫していました。経営陣からは「コストを5%削減しつつ、都心店舗への欠品は絶対に回避せよ」という厳しい指示が出ています。

### 直面している課題

これまでの首都圏フーズ物流のやり方は、渋谷と新宿の各センターから個別に発注依頼が来たら、その都度、近い方の倉庫からトラックを手配するという、いわば「場当たり的」なものでした。この方法にはいくつかの問題がありました。

1.  **非効率なトラック稼働:** 同じ日の午前中に千葉倉庫から渋谷へトラックが出たのに、午後にはまた千葉倉庫から少量の追加荷物を積んで渋谷へ向かう、といった無駄が発生していました。
2.  **低い積載率:** 緊急の補充依頼に応えるため、トラックの荷台が半分も埋まらないまま輸送することがあり、輸送の固定費が収益を圧迫していました。
3.  **トラック台数の制約:** 提携している運送会社との契約上、1日に全社で利用できるトラックの合計台数には上限があり、繁忙期には手配できないリスクがありました。

佐藤さんは、個別の発注に個別に対応するやり方では限界があると感じていました。「渋谷と新宿、千葉と神奈川、製品在庫と輸送タイミング。これら全てを統合的に考え、最も効率的な計画を立てる必要がある」と考えた彼は、数理最適化モデリング言語AMPLを用いて、この複雑な問題を解決することにしました。

### AMPLによる問題のモデル化

佐藤さんは、AMPLのエキスパートに相談し、自社の課題を数理モデルに落とし込んでいきました。

* **目的:** 複数日（まずは2日間の問題）にわたる「トラックの固定輸送費」と「配送センターでの在庫維持費」の合計を最小化すること。

* **決定すべきこと（決定変数）:**
    * **いつ、どの倉庫から、どの配送センターへ、どの製品を、どれだけ輸送するか？** (`Transport`)
    * その際に、**各ルートでトラックを何台使うか？** (`NumTrucks`)
    * その結果、**毎日の終わりに各センターの製品在庫はどれだけになるか？** (`Inventory`)

* **守るべきルール（制約条件）:**
    1.  **需要の充足:** 渋谷と新宿の各センターでは、日々の需要を絶対に満たさなければなりません（欠品は許されない）。これは、今日の在庫は「昨日の在庫＋今日の入荷量－今日の需要量」で決まるという**在庫バランス制約**で表現されます。
    2.  **トラックの能力:** 1台のトラックに積める量には上限があります。同時に、積載率が極端に悪い輸送をなくすため、「トラックを1台使うなら、最低でも積載可能量の20%は積む」という**トラック容量の上下限制約**を設けました。
    3.  **保管スペースの限界:** 渋谷と新宿のセンターは都心にあるため、保管スペースに限りがあります。全製品の合計在庫が、この上限を超えてはいけません（**卸在庫容量制約**）。
    4.  **車両の確保:** 運送会社との契約により、1日に千葉・神奈川の倉庫から出発できるトラックの合計台数には限りがあります（**一日あたり総トラック台数制約**）。

### 最適化の実行と得られた洞察

佐藤さんは、これらの条件をすべて盛り込んだAMPLモデルに、過去の需要データ、輸送費、在庫費などの現実の数値を入力し、ソルバーを実行しました。数秒後、コンピュータは最適な補充計画を提示しました。

その結果を見て、佐藤さんはいくつかの重要な発見をします。

* **発見1：共同配送による固定費削減**
    最適化された計画では、これまで別々に輸送していた渋谷向けと新宿向けの荷物の一部が、同じ日に千葉倉庫からまとめて輸送されていました。これにより、トラックの稼働台数が減り、固定費が大幅に削減されていました。

* **発見2：戦略的な在庫活用**
    これまでは需要に応じて輸送量を決めていましたが、計画では1日目に少し多めに輸送して在庫として確保し、トラック台数の上限が厳しい2日目の輸送量を抑える、という動きが見られました。在庫費用は少し増えますが、それを上回る輸送費の削減効果があり、トータルコストは最小化されていました。

* **発見3：非効率な輸送の撲滅**
    最低積載量のルールが効果を発揮し、少量の荷物を運ぶためだけにトラックを1台走らせる、といった最も非効率な輸送が完全に排除されていました。

### 結論

AMPLによる最適化アプローチは、佐藤さんに「コスト削減」と「安定供給」という二律背反の課題を両立させるための具体的なアクションプランを示してくれました。彼は、このモデルを毎週実行し、常に全体の状況を把握しながら最適な輸送・在庫計画を立案していくことで、会社の競争力を高められると確信したのでした。

### 新たな課題：週次計画の最適化

物流企画マネージャーの佐藤さんは、AMPLを用いた2日間の補充計画の最適化に成功し、コスト削減と安定供給の両立に手応えを感じていました。次のステップとして、彼はこのアプローチを**1週間単位の計画**に拡張することを決意します。

週末を挟むことで需要の変動はさらに大きくなります。例えば、週末前の金曜日には需要が急増し、週明けの月曜日には各センターが在庫を補充するため再び需要が高まります。一方で、土日のトラック稼働台数は平日よりも少なくなる契約です。

これらの変動をすべて見越して、週全体で最も効率的な補充計画を立てるためには、日々の場当たり的な判断では到底不可能です。佐藤さんは、1週間の需要予測データをもとに、再度AMPLで最適計画を立案することにしました。

### 1週間の計画データ

佐藤さんは、過去の販売実績から、今後1週間の製品別・センター別の需要量を予測しました。製品P1は「プレミアム緑茶」、P2は「ミネラルウォーター」です。

**表1：1週間の需要予測データ（単位：ケース）**

| 日 | 曜日 | 製品 | 渋谷センター需要 | 新宿センター需要 |
| :---: | :---: | :--- | :---: | :---: |
| 1 | 月曜 | 緑茶 | 400 | 500 |
| | | 水 | 300 | 350 |
| 2 | 火曜 | 緑茶 | 300 | 400 |
| | | 水 | 230 | 280 |
| 3 | 水曜 | 緑茶 | 330 | 450 |
| | | 水 | 200 | 300 |
| 4 | 木曜 | 緑茶 | 310 | 420 |
| | | 水 | 210 | 290 |
| 5 | 金曜 | 緑茶 | 450 | 550 |
| | | 水 | 350 | 400 |
| 6 | 土曜 | 緑茶 | 250 | 300 |
| | | 水 | 280 | 320 |
| 7 | 日曜 | 緑茶 | 150 | 200 |
| | | 水 | 180 | 210 |

また、運送会社との契約により、1日に全社で利用できるトラックの合計台数は、**平日（1〜5日目）は4台**まで、**週末（6, 7日目）は2台**までという制約があります。

### AMPLモデルと更新されたデータ

この新しい7日間計画のために、佐藤さんはAMPLモデルの定義部分（`model;`ブロック）を一切変更する必要がありませんでした。モデルは`T_max`という期間を指定するパラメータを使って汎用的に作られているため、データ部分を更新するだけで、異なる期間の問題に柔軟に対応できるのです。

#### 7日間計画から得られる新たな洞察

この1週間モデルを実行することで、佐藤さんはさらに高度な計画を立てられるようになります。例えば、トラックの台数が制限される週末に向けて、木曜日や金曜日に少し多めに輸送して在庫を積み増しておく、といった**先行補充**の判断が可能になります。また、週明けの月曜日の高い需要に応えるため、輸送コストが安いルートを使って日曜日のうちに補充を済ませておくなど、より戦略的でコスト効率の高い物流網の構築が実現できるのです。

In [ ]:
#| hide
reset;
model;

# 1. 集合 (Sets)
set PRODUCTS;
set WAREHOUSES;
set DEPOTS;

param T_max > 0 integer;
set PERIODS = 1..T_max;
set PERIODS0 = 0..T_max;

# 2. パラメータ (Parameters)
param demand{PRODUCTS, DEPOTS, PERIODS} >= 0 default 0;
param truck_capacity_max > 0;
param truck_capacity_min >= 0;
param fixed_transport_cost{WAREHOUSES, DEPOTS} >= 0;
param holding_cost{PRODUCTS, DEPOTS} >= 0;
param depot_capacity{DEPOTS} >= 0;
param initial_inventory{PRODUCTS, DEPOTS} >= 0 default 0;
param max_total_trucks_per_day{PERIODS} > 0 integer;

# 3. 決定変数 (Variables)
var Transport{PRODUCTS, WAREHOUSES, DEPOTS, PERIODS} >= 0;
var NumTrucks{WAREHOUSES, DEPOTS, PERIODS} integer >= 0;
var Inventory{PRODUCTS, DEPOTS, PERIODS0} >= 0;

# 4. 目的関数 (Objective Function)
minimize Total_Cost:
    sum{t in PERIODS, w in WAREHOUSES, d in DEPOTS}
        fixed_transport_cost[w,d] * NumTrucks[w,d,t]
    +
    sum{t in PERIODS, p in PRODUCTS, d in DEPOTS}
        holding_cost[p,d] * Inventory[p,d,t];

# 5. 制約 (Constraints)
subject to Inventory_Balance{p in PRODUCTS, d in DEPOTS, t in PERIODS}:
    Inventory[p,d,t-1] + sum{w in WAREHOUSES} Transport[p,w,d,t] - demand[p,d,t]
    = Inventory[p,d,t];

subject to Set_Initial_Inventory{p in PRODUCTS, d in DEPOTS}:
    Inventory[p,d,0] = initial_inventory[p,d];

subject to Truck_Capacity_Upper{w in WAREHOUSES, d in DEPOTS, t in PERIODS}:
    sum{p in PRODUCTS} Transport[p,w,d,t] <= truck_capacity_max * NumTrucks[w,d,t];

subject to Truck_Capacity_Lower{w in WAREHOUSES, d in DEPOTS, t in PERIODS}:
    sum{p in PRODUCTS} Transport[p,w,d,t] >= truck_capacity_min * NumTrucks[w,d,t];

subject to Depot_Capacity{d in DEPOTS, t in PERIODS}:
    sum{p in PRODUCTS} Inventory[p,d,t] <= depot_capacity[d];

subject to Total_Truck_Limit{t in PERIODS}:
    sum{w in WAREHOUSES, d in DEPOTS} NumTrucks[w,d,t] <= max_total_trucks_per_day[t];

data;

# --- 集合のデータ ---
set PRODUCTS := P1 P2; # P1:緑茶, P2:水
set WAREHOUSES := WH_CHIBA WH_KANAGAWA;
set DEPOTS := DP_SHIBUYA DP_SHINJUKU;
param T_max := 7;

# --- パラメータのデータ ---
param truck_capacity_max := 1000;
param truck_capacity_min := 200;
param max_total_trucks_per_day := 1 4, 2 4, 3 4, 4 4, 5 4, 6 2, 7 2;

param fixed_transport_cost :=
    WH_CHIBA    DP_SHIBUYA   500
    WH_CHIBA    DP_SHINJUKU  550
    WH_KANAGAWA DP_SHIBUYA   480
    WH_KANAGAWA DP_SHINJUKU  450;

param holding_cost :=
    P1 DP_SHIBUYA   2
    P2 DP_SHIBUYA   3
    P1 DP_SHINJUKU  2.2
    P2 DP_SHINJUKU  3.3;

param depot_capacity :=
    DP_SHIBUYA   1500
    DP_SHINJUKU  1800;

param initial_inventory :=
    P1 DP_SHIBUYA   50
    P2 DP_SHIBUYA   50
    P1 DP_SHINJUKU  40
    P2 DP_SHINJUKU  60;

# --- demand パラメータの最終修正箇所 ---
# [*,*,:] テンプレートを使用して、(製品, 卸)を行、(期間)を列として明示的に指定します。
param demand [*,*,:] :
          1   2   3   4   5   6   7 :=
P1 DP_SHIBUYA   400 300 330 310 450 250 150
P1 DP_SHINJUKU  500 400 450 420 550 300 200
P2 DP_SHIBUYA   300 230 200 210 350 280 180
P2 DP_SHINJUKU  350 280 300 290 400 320 210;


# --- 実行セクション ---
option solver highs;
solve;

# --- 結果表示 ---
display Transport, NumTrucks, Inventory;

#| hide

## 3段階サプライチェーン最適化問題の非線形計画法による定式化

## 1. 集合と添え字

*   `P`: プラント（工場）の集合、添え字は $p$
*   `W`: 倉庫の集合、添え字は $w$
*   `C`: 顧客の集合、添え字は $c$
*   `I`: アイテム（製品）の集合、添え字は $i$

## 2. パラメータ

*   $Prod_{pi}$: バイナリパラメータ。プラント $p$ がアイテム $i$ を生産可能なら1、そうでなければ0。
*   $μ_{ci}$: 顧客 $c$ におけるアイテム $i$ の平均需要率（単位/期間）。
*   $σ_{ci}$: 顧客 $c$ におけるアイテム $i$ の需要の標準偏差（単位/期間）。
*   $v_i$: アイテム $i$ の単位あたりの容積（容積単位/単位）。
* $h_{ci}$: 顧客 $c$ におけるアイテム $i$ の単位あたり・期間あたりの在庫保管費用（円/単位/期間）。
* $TC_{pw}$: プラント $p$ から倉庫 $w$ への容積単位あたりの輸送コスト（円/容積単位）。
*   $TC_{wc}$: 倉庫 $w$ から顧客 $c$ への容積単位あたりの輸送コスト（円/容積単位）。
*   $TC_{pc}$: プラント $p$ から顧客 $c$ へ直接輸送する場合の容積単位あたりの輸送コスト（円/容積単位）。
*   $K_{truck}$: トラック1台の容量（容積単位）。
*   $R_{wc}$: 倉庫 $w$ から顧客 $c$ への補充における固定補充時間（時間単位）。
*   $R_{pc}$: プラント $p$ から顧客 $c$ への補充における固定補充時間（時間単位）。
*   $Cap_c$: 顧客 $c$ の最大在庫容量（容積単位）。
*   $z$: 安全係数（目標サービスレベルに対応する標準正規分布のz値、例: $Φ^{-1}(サービスレベル)$、ここで $Φ$ は標準正規累積分布関数）。
*   $ε$: ゼロ除算を防ぐための小さな正の数（例: $1e-6$）。

## 3. 決定変数

*   $x_{pwi}$: プラント $p$ から倉庫 $w$ へのアイテム $i$ のフローレート（単位/期間）。$Prod_{pi} = 1$ の場合のみ定義される。
*   $x_{wci}$: 倉庫 $w$ から顧客 $c$ へのアイテム $i$ のフローレート（単位/期間）。
*   $x_{pci}$: プラント $p$ から顧客 $c$ へ直接輸送されるアイテム $i$ のフローレート（単位/期間）。$Prod_{pi} = 1$ の場合のみ定義される。

## 4. 中間表現（決定変数に依存）

*   **輸送リンク上の総物流量:**
    *   $V_{pw}$: $p$ から $w$ への総物流量。
        $V_{pw} = Σ_{i | Prod_{pi}=1} (v_i x_{pwi})$  (全ての $p ∈ P, w ∈ W$)
    *   $V_{wc}$: $w$ から $c$ への総物流量。
        $V_{wc} = Σ_i (v_i x_{wci})$  (全ての $w ∈ W, c ∈ C$)
    *   $V_{pc}$: $p$ から $c$ への総物流量。
        $V_{pc} = Σ_{i | Prod_{pi}=1} (v_i x_{pci})$  (全ての $p ∈ P, c ∈ C$)

*   **FTL (満載輸送) のサイクルタイム:**
    *   $CT_{wc}$: $w$ と $c$ 間のサイクルタイム。フロー量が正のときのみ以下のように定義する。
        $CT_{wc} = K_{truck} / (V_{wc})$  (全ての $w ∈ W, c ∈ C$)
    *   $CT_{pc}$: $p$ と $c$ 間のサイクルタイム。
        $CT_{pc} = K_{truck} / (V_{pc})$  (全ての $p ∈ P, c ∈ C$)
    *(注意: フローがゼロの場合のゼロ除算を避けるために $ε$ を加える)*

*   **顧客における補充リードタイム:**
    *   $L_{wc}$: 倉庫 $w$ から顧客 $c$ への補充リードタイム。フロー量が正のときのみ以下のように定義する。
        $L_{wc} = CT_{wc} + R_{wc} = (K_{truck} / (V_{wc})) + R_{wc}$  (全ての $w ∈ W, c ∈ C$)
    *   $L_{pc}$: プラント $p$ から顧客 $c$ への補充リードタイム。
        $L_{pc} = CT_{pc} + R_{pc} = (K_{truck} / (V_{pc})) + R_{pc}$  (全ての $p ∈ P, c ∈ C$)

*   **顧客 $c$ におけるアイテム $i$ の実効リードタイム:** (フローの割合に基づく加重平均)
    *   $L_{ci}$: 顧客 $c$ におけるアイテム $i$ の実効リードタイム。$μ_{ci} > 0$ の場合のみ定義。
        $L_{ci} = ( Σ_{w} (x_{wci} * L_{wc}) + Σ_{p | Prod_{pi}=1} (x_{pci} * L_{pc}) ) / μ_{ci}$
        *$L_{wc}$ と $L_{pc}$ を代入:*
        $L_{ci} = ( Σ_{w} x_{wci} * [(K_{truck} / (V_{wc} + ε)) + R_{wc}] + Σ_{p | Prod_{pi}=1} x_{pci} * [(K_{truck} / (V_{pc} + ε)) + R_{pc}] ) / μ_{ci}$ (全ての $c ∈ C, i ∈ I$ 但し $μ_{ci} > 0$)

*   **顧客 $c$ におけるアイテム $i$ の在庫基準（目標在庫水準/Base-Stock Level）:**
    *   $S_{ci}$: 顧客 $c$ におけるアイテム $i$ の在庫基準。
        $S_{ci} = μ_{ci} * L_{ci} + z * σ_{ci} * \sqrt{L_{ci}}$ (全ての $c ∈ C, i ∈ I$ 但し $μ_{ci} > 0$)
    *(注意: $L_{ci} ≥ 0$ が必要。$K_{truck}$, $R_{wc}$, $R_{pc}$ が正であり、$V ≥ 0$ なので、$L_{wc}$ と $L_{pc}$ は正となり、フローが非負であれば $L_{ci}$ は非負となる)*

## 5. 目的関数

総輸送コストと総在庫保管コストの合計を最小化する。

*   **総輸送コスト (TTC):**
    $TTC = Σ_{p∈P} Σ_{w∈W} (TC_{pw} * V_{pw}) + Σ_{w∈W} Σ_{c∈C} (TC_{wc} * V_{wc}) + Σ_{p∈P} Σ_{c∈C} (TC_{pc} * V_{pc})$
    *$V$ の式を代入:*
    $TTC = Σ_{p∈P} Σ_{w∈W} TC_{pw} * (Σ_{i | Prod_{pi}=1} x_{pwi} * v_i) + Σ_{w∈W} Σ_{c∈C} TC_{wc} * (Σ_i x_{wci} * v_i) + Σ_{p∈P} Σ_{c∈C} TC_{pc} * (Σ_{i | Prod_{pi}=1} x_{pci} * v_i)$

*   **総在庫保管コスト (TIC):** (コストは平均在庫量に適用されると仮定し、在庫基準 $S_{ci}$ で近似する)
    $TIC = Σ_{c∈C} Σ_{i∈I | μ_{ci}>0} (h_{ci} * S_{ci})$
    *$S_{ci}$ を代入:*
    $TIC = Σ_{c∈C} Σ_{i∈I | μ_{ci}>0} h_{ci} * (μ_{ci} * L_{ci} + z * σ_{ci} * \sqrt{L_{ci}})$
    *(ステップ4の複雑な $L_{ci}$ の式を使用)*

*   **総コスト (TC):**
    `最小化 TC = TTC + TIC`

## 6. 制約条件

*   **プラント生産ソース:** フローはそのアイテムを生産するプラントからのみ発生する。（これは $x_{pwi}$ と $x_{pci}$ を $Prod_{pi} = 1$ である $p, i$ に対してのみ定義することで暗黙的に扱われる）。
*   **倉庫フロー保存則:** 各倉庫、各アイテムについて、総流入量は総流出量に等しい。
    $Σ_{p | Prod_{pi}=1} x_{pwi} = Σ_{c∈C} x_{wci}$ (全ての $w ∈ W, i ∈ I$)
*   **顧客需要充足:** 各顧客、各アイテムについて、受け取る総フローは平均需要率に等しい。
    $Σ_{w∈W} x_{wci} + Σ_{p | Prod_{pi}=1} x_{pci} = μ_{ci}$ (全ての $c ∈ C, i ∈ I$)
*   **顧客在庫容量:** 各顧客において、平均在庫（在庫基準で近似）が占める総容積はその容量を超えてはならない。
    $Σ_{i∈I | μ_{ci}>0} (S_{ci} * v_i) ≤ Cap_c$ (全ての $c ∈ C$)
    *$S_{ci}$ を代入:*
    $Σ_{i∈I | μ_{ci}>0} [ (μ_{ci} * L_{ci} + z * σ_{ci} * \sqrt{L_{ci}}) * v_i ] ≤ Cap_c$ (全ての $c ∈ C$)
    *(ステップ4の複雑な $L_{ci}$ の式を使用)*
*   **非負条件:** 全てのフロー変数は非負でなければならない。
    $x_{pwi} ≥ 0$ (全ての $p ∈ P, w ∈ W, i ∈ I$ 但し $Prod_{pi} = 1$)
    $x_{wci} ≥ 0$ (全ての $w ∈ W, c ∈ C, i ∈ I$)
    $x_{pci} ≥ 0$ (全ての $p ∈ P, c ∈ C, i ∈ I$ 但し $Prod_{pi} = 1$)

## 7. NLP定式化の要約

**最小化:**
$$
 Σ_{p,w} TC_{pw} (Σ_{i|P_{pi}=1} x_{pwi} v_i) + Σ_{w,c} TC_{wc} (Σ_i x_{wci} v_i) + Σ_{p,c} TC_{pc} (Σ_{i|P_{pi}=1} x_{pci} v_i) ]
+ [ Σ_{c,i|μ_{ci}>0} h_{ci} (μ_{ci} L_{ci} + z σ_{ci} sqrt(L_{ci})) 
$$

**制約条件:**

1.  $Σ_{p|P_{pi}=1} x_{pwi} = Σ_c x_{wci}$ (全ての $w, i$)
2.  $Σ_w x_{wci} + Σ_{p|P_{pi}=1} x_{pci} = μ_{ci}$ (全ての $c, i$)
3.  $Σ_{i|μ_{ci}>0} [ (μ_{ci} L_{ci} + z σ_{ci} \sqrt{L_{ci}}) v_i ] ≤ Cap_c$ (全ての $c$)
4.  $x_{pwi}, x_{wci}, x_{pci} ≥ 0$ (定義された全ての変数)

**ここで:**

*   $V_{wc} = Σ_j (x_{wcj} * v_j)$
*   $V_{pc} = Σ_{j | Prod_{pj}=1} (x_{pcj} * v_j)$
*   $L_{wc} = (K_{truck} / (V_{wc} + ε)) + R_{wc}$
*   $L_{pc} = (K_{truck} / (V_{pc} + ε)) + R_{pc}$
*   $L_{ci} = ( Σ_{w} x_{wci}*L_{wc} + Σ_{p|P_{pi}=1} x_{pci}*L_{pc} ) / μ_{ci}$ ($μ_{ci}>0$ の場合)

## 8. 主な特徴

*   **非線形性:** 目的関数と容量制約（制約3）は、変数の合計（$V_{wc}$, $V_{pc}$）による除算と $\sqrt{L_{ci}}$ 項を含む $L_{ci}$ 項により、非常に非線形性が高い。
*   **非凸性の可能性:** $1/V$ 項や $\sqrt{L_{ci}}$ 項の存在により、問題は非凸である可能性が高い。これは、標準的なNLPソルバーが局所最適解を見つける可能性があり、それが必ずしも大域最適解ではないことを意味する。保証された最適性を得るためには、特殊な大域最適化ソルバーやヒューリスティクスが必要になる場合がある。
*   **大規模性:** プラント、倉庫、顧客、アイテムの数によっては、変数と制約の数が非常に大きくなる可能性がある。

---

In [ ]:
#| hide
# Suntoryの問題
%%ampl_eval
reset;
model;
# Sets
set ITEMS;  # Products
set PLANTS; # Production facilities
set WAREHOUSES; # Distribution centers
set CUSTOMERS;  # Demand points

# Parameters
param mean_demand{ITEMS, CUSTOMERS};  # Mean demand for item i at customer c
param std_demand{ITEMS, CUSTOMERS};   # Std. deviation of demand
param transport_cost_plant_warehouse{PLANTS, WAREHOUSES}; # Cost from plant to warehouse
param transport_cost_plant_customer{PLANTS, CUSTOMERS};  # Cost from plant to customer
param transport_cost_warehouse_customer{WAREHOUSES, CUSTOMERS}; # Cost from warehouse to customer
param truck_capacity_plant_customer{PLANTS, CUSTOMERS};  # Truck capacity (plant → customer)
param truck_capacity_warehouse_customer{WAREHOUSES, CUSTOMERS}; # Truck capacity (warehouse → customer)
param replenishment_time_plant;  # Constant replenishment time from plants
param replenishment_time_warehouse; # Constant replenishment time from warehouses
param holding_cost{ITEMS, CUSTOMERS}; # Inventory holding cost per unit
param z_score{ITEMS, CUSTOMERS};      # Z-score for service level
param inventory_capacity{CUSTOMERS};  # Max inventory capacity per customer
param produces{ITEMS, PLANTS} binary; # 1 if plant p produces item i

# Variables
var Y{ITEMS, CUSTOMERS, PLANTS} binary; # 1 if customer c gets item i from plant p
var Z{ITEMS, CUSTOMERS, WAREHOUSES} binary; # 1 if customer c gets item i from warehouse w
var X_plant_warehouse{ITEMS, PLANTS, WAREHOUSES} >= 0; # Flow plant → warehouse
var F_plant_customer{PLANTS, CUSTOMERS} >= 0; # Total flow plant → customer
var F_warehouse_customer{WAREHOUSES, CUSTOMERS} >= 0; # Total flow warehouse → customer
var L_plant_customer{PLANTS, CUSTOMERS} >= 0; # Leadtime plant → customer
var L_warehouse_customer{WAREHOUSES, CUSTOMERS} >= 0; # Leadtime warehouse → customer

# Objective: Minimize total cost (transportation + inventory)
minimize Total_Cost:
    sum{i in ITEMS, p in PLANTS, w in WAREHOUSES} transport_cost_plant_warehouse[p,w] * X_plant_warehouse[i,p,w] +
    sum{i in ITEMS, c in CUSTOMERS, p in PLANTS} transport_cost_plant_customer[p,c] * mean_demand[i,c] * Y[i,c,p] +
    sum{i in ITEMS, c in CUSTOMERS, w in WAREHOUSES} transport_cost_warehouse_customer[w,c] * mean_demand[i,c] * Z[i,c,w] +
    sum{i in ITEMS, c in CUSTOMERS, p in PLANTS}
        holding_cost[i,c] * mean_demand[i,c] * (L_plant_customer[p,c] + (z_score[i,c] * std_demand[i,c] / mean_demand[i,c]) * (L_plant_customer[p,c]))^(1/2) * Y[i,c,p] +
    sum{i in ITEMS, c in CUSTOMERS, w in WAREHOUSES}
        holding_cost[i,c] * mean_demand[i,c] * (L_warehouse_customer[w,c] + (z_score[i,c] * std_demand[i,c] / mean_demand[i,c]) * (L_warehouse_customer[w,c]))^(1/2) * Z[i,c,w];

# Constraints
# Single-sourcing: Each customer gets each item from exactly one source
subject to Single_Sourcing{i in ITEMS, c in CUSTOMERS}:
    sum{p in PLANTS} Y[i,c,p] + sum{w in WAREHOUSES} Z[i,c,w] = 1;

# Warehouse flow conservation（０で割っているので間違い）
subject to Warehouse_Flow{i in ITEMS, w in WAREHOUSES}:
    sum{p in PLANTS} X_plant_warehouse[i,p,w] = sum{c in CUSTOMERS} mean_demand[i,c] * Z[i,c,w];

# Leadtime definitions (nonlinear)（０で割っているので間違い）
subject to Leadtime_Plant{p in PLANTS, c in CUSTOMERS}:
    L_plant_customer[p,c] = (truck_capacity_plant_customer[p,c] / F_plant_customer[p,c])+ replenishment_time_plant;

subject to Leadtime_Warehouse{w in WAREHOUSES, c in CUSTOMERS}:
    L_warehouse_customer[w,c] = (truck_capacity_warehouse_customer[w,c] / F_warehouse_customer[w,c]) + replenishment_time_warehouse;

# Total flow definitions
subject to Flow_Plant_Customer{p in PLANTS, c in CUSTOMERS}:
    F_plant_customer[p,c] = sum{i in ITEMS} mean_demand[i,c] * Y[i,c,p];

subject to Flow_Warehouse_Customer{w in WAREHOUSES, c in CUSTOMERS}:
    F_warehouse_customer[w,c] = sum{i in ITEMS} mean_demand[i,c] * Z[i,c,w];

# Inventory capacity constraint (nonlinear)
subject to Inventory_Capacity{c in CUSTOMERS}:
    sum{i in ITEMS, p in PLANTS}
        mean_demand[i,c] * (L_plant_customer[p,c] + (z_score[i,c] * std_demand[i,c] / mean_demand[i,c]) * (L_plant_customer[p,c]))^(1/2) * Y[i,c,p] +
    sum{i in ITEMS, w in WAREHOUSES}
        mean_demand[i,c] * (L_warehouse_customer[w,c] + (z_score[i,c] * std_demand[i,c] / mean_demand[i,c]) * (L_warehouse_customer[w,c]))^(1/2) * Z[i,c,w] <= inventory_capacity[c];

# Plant production constraints
subject to Plant_Production{i in ITEMS, p in PLANTS, c in CUSTOMERS}:
    Y[i,c,p] <= produces[i,p];